# Proposition Analysis

This notebook implements Python code to apply a large language model (LLM) to the tasks of first generating a research/analysis plan and second executing that plan to generate an analysis report.
The key input is a proposition -- a statement or question -- that is to be researched.
The output is an R Markdown text file containing the report capturing the results and summarization of the research.

As the plan, a query graph is created consisting of Query nodes. Some of the nodes pose a question to the LLM, while others first pose a search query to the Serper Google Search API in order to gather current information, and then use the search results as the context for the question it poses to the LLM.

## Test of Jason Liu's `instructor` Package

This notebook began as a quick test of the `instructor` package by Jason Liu ([Instructor Documentation](https://jxnl.github.io/instructor)).  It's strengths are the following:

* Can query an LLM based off a class object (i.e., the `response_model` argument) without even having to write a custom system prompt.
  - Results in faster implementation of LLM-based applications.
* Will auto-validate the LLM's response to ensure that the results are as expected.
  - Results in more reliable, production-ready code leveraging LLM.


### See the MIT License Notice (per the QueryPlan, Query, ComputedQuery, ComputedQueryGPT4, MergedResponses classes)

The Pydantic model classes `QueryPlan`, `Query`, `ComputedQuery`, `ComputedQueryGPT4`, and `MergedResponses` are modifications of the same/similar named classes in the `instructor` package examples for query planning and task planning. Therefore, the MIT License notice appears in a Markdown chunk just above the class definitions.

These model classes are modified versions and/or extensions of those same/similar named classes in the [`instructor` package](https://github.com/jxnl/instructor) examples by Jason Liu:

* [query_planner_execution.py](https://github.com/jxnl/instructor/blob/main/examples/query_planner_execution/query_planner_execution.py)
* [task_planner_topological_sort.py](https://github.com/jxnl/instructor/blob/main/examples/task_planner/task_planner_topological_sort.py)

Therefore, the MIT License Notice below, within the "Model Class Definitions" section, is included per the instructions at the [`instructor` package's Github repository](https://github.com/jxnl/instructor/tree/main?tab=MIT-1-ov-file#readme).


### To Go Farther...

See the Google Collab notebook here: https://www.youtube.com/redirect?event=video_description&redir_token=QUFFLUhqbHZMSGlSN2lvX3h1T296Wng1dkpRaHdoWHJfd3xBQ3Jtc0tsMTB4MENQWkRDR2lxZU1ucEtZd0lBM0M3Q25aWFFvWDE5ZjU2N1M0Tjh6ZjFDRmlacTFuYUxwUGJ2aVVOQzY0aDUtU1E0SDE2Y3NoVEdfNjMwLXk0Q0lnVE93N20wZDJMN0I3M3BQRF9rYnpwODNVZw&q=https%3A%2F%2Fcolab.research.google.com%2Fdrive%2F1Hb-khdfXvYKz8zUoYy7l8yuZ2MDoMAkF%3Fusp%3Dsharing&v=b_fRz09ZP_M

* Also see ["Bridging Language Models with Python with Instructor, Pydantic, and OpenAI’s function calls."](https://medium.com/@jxnlco/bridging-language-model-with-python-with-instructor-pydantic-and-openais-function-calling-f32fb1cdb401) by Jason Liu.

# Set Up

In [ ]:
# This is how to install the latest version of the instructor package:
#!pip install git+https://github.com/jxnl/instructor.git#egg=instructor

In [ ]:
import asyncio
import enum
import instructor

from typing import Any, Dict, List, Generator, Union, Iterator
from openai import OpenAI, ChatCompletion
from pydantic import Field, BaseModel, validator, field_validator

#client = instructor.patch(OpenAI())

In [ ]:
import os
#os.environ["OPENAI_API_KEY"] = '....'
os.environ["OPENAI_API_KEY"] = '****' #PUT your API Key


In [ ]:
# This enables response_model keyword
# from client.chat.completions.create
client = instructor.patch(OpenAI())

## Search Functions

In [ ]:
from pprint import pprint
from langchain.utilities import GoogleSerperAPIWrapper
os.environ["SERPERAPI_API_KEY"] = "****" #PUT your API Key
def search_GoogleSerperAPI(topic: str, raw_results=False):
    search = GoogleSerperAPIWrapper(serper_api_key=os.environ['SERPERAPI_API_KEY'])
    results = search.results(topic)
    if raw_results:
        return results
    
    results_text = ""
    if results.get('knowledgeGraph'):
        rkb = results['knowledgeGraph']
        if rkb.get('attributes') is None:
            attr_str = ''
        else:
            attr_str = "\n" + "\n".join([f"{k}: {v}" for k,v in rkb['attributes'].items()])

        wbs = rkb.get('website')
        if wbs is None:
            results_text += f"[{rkb['title']}]({rkb['descriptionLink']}):\n{rkb['description']}" + attr_str
        else:
            results_text += f"[{rkb['title']}]({rkb['website']}):\n{rkb['description']}" + attr_str
    
    for rec in results['organic']:
        if rec.get('attributes') is None:
            attr_str = ''
        else:
            attr = rec.get('attributes')
            if attr:
                attr_str = "\n" + "\n".join([f"{k}: {v}" for k,v in attr.items()])
        results_text += f"\n* [{rec['title']}]({rec['link']}):\n{rec['snippet']}" + attr_str
        
    return results_text



In [ ]:
def summarize_raw_search(raw_search: dict) -> str:
    results_text = ""
    if raw_search.get('knowledgeGraph'):
        rkb = raw_search['knowledgeGraph']
        if rkb.get('attributes') is None:
            attr_str = ''
        else:
            attr_str = "\n" + "\n".join([f"{k}: {v}" for k,v in rkb['attributes'].items()])

        wbs = rkb.get('website')
        try:
            if wbs is None:
                results_text += f"[{rkb['title']}]({rkb['descriptionLink']}):\n{rkb['description']}" + attr_str
            else:
                results_text += f"[{rkb['title']}]({rkb['website']}):\n{rkb['description']}" + attr_str
        except:
            results_text += attr_str
    
    for rec in raw_search['organic']:
        if rec.get('attributes') is None:
            attr_str = ''
        else:
            attr_str = "\n" + "\n".join([f"{k}: {v}" for k,v in rec['attributes'].items()])
        snippet = rec.get('snippet','')
        results_text += f"\n* [{rec['title']}]({rec['link']}):\n{snippet}" + attr_str

    # Strip out the 'Missing:...' and 'Show results with:...' sections
    results_text = re.sub(r'\n*Missing:.+?([\|\n])','',results_text)
    results_text = re.sub(r'(\| )*Show results with:.+?([\|\n])',r'\2',results_text)
        
    return results_text

In [ ]:
import re
import unicodedata

def to_ascii(text):
    # Normalize the Unicode string to its closest ASCII representation
    normalized = unicodedata.normalize('NFKD', text)

    # Encode to ASCII bytes, then decode back to string, ignoring non-ASCII characters
    ascii_text = normalized.encode('ascii', 'ignore').decode('ascii')
    
    return ascii_text


In [ ]:
from pydantic import BaseModel, Field, field_validator
from typing import List,Optional
from typing_extensions import Annotated
from pydantic import AfterValidator
import re

def validate_response(v:str) -> str:
    if len(v)<1:
        raise ValueError('Response cannot be an empty string. It must be generated from the internet search results (`raw_results`) in response to the query (`query`).')
    return v

class SearchSummary(BaseModel):
    """
    The internet search query and the results of an internet search for the query.
    """
    
    context: str = Field(...,description="The context within which to answer the query. (This may be an empty string.)")
    query: str = Field(...,description="The internet search query to perform and then responded to within the given context, if any.")
    #response: str = Field(default='', validate_default=True,description="A clear concise response to the query based upon the raw results and generated within the given context.")
    response: Annotated[str, AfterValidator(validate_response)] = Field(...,description="A clear concise response to the query based upon the raw results and generated within the given context.")
    raw_results: Optional[str] = Field(default=None, validate_default=True,description="The raw results of the internet search.")

    @field_validator('raw_results')
    @classmethod
    def generate_raw_results(cls, v, validation_info, **kwargs):
        # 'values' is a dictionary containing the field values of the model
        # Here we generate the response based on the query
        values = validation_info.data
        query = values.get('query', '')
        raw_valid = "..."
        if query:
            raw_valid = search_GoogleSerperAPI(topic = query,raw_results=False)
        return raw_valid


## Model Class Definitions

These model classes are modified versions and/or extensions of those same/similar named classes in the [`instructor` package](https://github.com/jxnl/instructor) examples by Jason Liu:

* [query_planner_execution.py](https://github.com/jxnl/instructor/blob/main/examples/query_planner_execution/query_planner_execution.py)
* [task_planner_topological_sort.py](https://github.com/jxnl/instructor/blob/main/examples/task_planner/task_planner_topological_sort.py)

Therefore, the MIT License Notice below is included per the instructions at the [`instructor` package's Github repository](https://github.com/jxnl/instructor/tree/main?tab=MIT-1-ov-file#readme):

### MIT License Notice
MIT License

Copyright (c) 2023 Michael L. Thompson

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

### Knowledge Graphs


In [ ]:
from pydantic import BaseModel, Field
from typing import List, Union

class Node(BaseModel):
    """
    Concepts, entities, principles, etc. that are central to the topic of discourse.
    """
    id: int
    label: str
    color: str
    description: str

class Edge(BaseModel):
    """
    Representation of a Relationship between the source and the target.
    For example, if causal, the source is a direct cause of the target, which is its effect. Therefore, it represents a cause-effect relationship
    within a causal mechanism.
    For example, if inferential, the source is an antecedent proposition and the target is the consequent, which follows logically from the proposition.
    Therefore, inferential chains of reasoning are represented.
    Relationships can be anything meaningful in the context of the concepts represented by the source and target. They DO NOT have to be causal or
    inferential.
    """
    source: int
    target: int
    label: str
    description: str
    color: str = "black"

class KnowledgeGraph(BaseModel):
    """
    You are an expert at capturing the conceptual mapping of concepts and their relationships as a detailed intricate web of knowledge 
    known as a "Knowledge Graph".
    Create such a Knowledge Graph.
    DO NOT have disjoint/unconnected subgraphs. (Every node should be reachable from every other node along an undirected path.)
    """
    topic: str = Field(..., description="The central topic captured by the concepts (nodes) and relationships (edges) in the Knowledge Graph.")
    nodes: List[Node] = Field(..., default_factory=list)
    edges: List[Edge] = Field(..., default_factory=list)

### Query Execution

In [ ]:
# See documentation on accessing context at BaseModel initialization:
# https://docs.pydantic.dev/latest/concepts/validators/#using-validation-context-with-basemodel-initialization

from contextlib import contextmanager
from contextvars import ContextVar

_init_context_var = ContextVar('_init_context_var', default=None)

@contextmanager
def init_context(value: Dict[str, Any]) -> Iterator[None]:
    token = _init_context_var.set(value)
    try:
        yield
    finally:
        _init_context_var.reset(token)

class TokenCost(BaseModel):
    tokens: dict
    cost: dict
    
class ComputeQuery(BaseModel):
    """
    Responds to a query by answering it
    """

    task_id: str = Field(...,description="The id of the query node from which `query` came.")
    query: str
    response:  Optional[str] = Field(default=None, validate_default=True,description="The response, answer and/or search results, to the `query`.")
    
    def __init__(__pydantic_self__, **data: Any) -> None:
        __pydantic_self__.__pydantic_validator__.validate_python(
            data,
            self_instance=__pydantic_self__,
            context=_init_context_var.get(),
        )

    @field_validator('response')
    @classmethod
    def generate_response(cls, v, validation_info, **kwargs):
        # 'values' is a dictionary containing the field values of the model
        # Here we generate the response based on the query
        values = validation_info.data
        query = values.get('query', '')
        
        if v is None:
            client = validation_info.context.get('client')
            model = validation_info.context.get('model')
            if query:
                completion = client.chat.completions.create(
                    model=model, 
                    temperature=0, 
                    messages=[
                        {'role':'system',
                         'content':"""You are an expert at logical and probabilistic reasoning. 
Use step-by-step logical reasoning to answer each question posed to you.
Answer as accurately and thoroughly as possible. Base your conclusions upon fact-based data. 
Cite the data in your responses, including URLs when available."""},{'role':'user', 'content':query}], 
                    max_tokens=3000
                )
                return completion.choices[0].message.content
        return v

In [ ]:
TEST = False
if TEST:
    client = instructor.patch(OpenAI())
    with init_context({'client': client, 'model': 'gpt-4-turbo-preview'}):
        resp = ComputeQuery(task_id='root',query='Hello, how are you?')

In [ ]:
if TEST:
    print(resp)
    #tkncst = resp.completion.usage
    #print(tkncst)

In [ ]:
class MergedResponses(BaseModel):
    """
    Models a merged response of multiple queries.
    Currently we just concatenate them but we can do much more complex things.
    """

    responses: List[ComputeQuery]

In [ ]:
import pandas as pd

def markdown_table_to_dataframe(markdown_text):
    # Split the text into lines
    lines = markdown_text.strip().split('\n')
    
    # Extract the header line and data lines
    header_line = lines[0]
    data_lines = lines[2:]  # Skip the Markdown alignment row

    # Split the header and data lines into cells
    headers = header_line.split('|')[1:-1]  # Skip the outer empty cells
    headers = [txt.strip() for txt in headers]
    data = [line.split('|')[1:-1] for line in data_lines]

    # Create and return a DataFrame
    return pd.DataFrame(data, columns=headers)

# Example usage
markdown_text = """
| Column1 | Column2 | Column3 |
|---------|---------|---------|
| data1   | data2   | data3   |
| data4   | data5   | data6   |
"""
df = markdown_table_to_dataframe(markdown_text)
print(df)


In [ ]:

#        * DO NOT create a Query node with only a single dependency that performs a search. Instead, combine the question and the search query 
#          into a single Query node with an assigned `search_query` field to perform the search and `question` field to ask the question.

class Query(BaseModel):
    """
    A question in a question-answer query plan that may or may not require search. 
    If need data or current information by internet searching, specify a `search_query`:
        * Pose a valid question for the `question` field, or as a "Tabulate.." statement, if use a list of search strings for `search_query`.
        * Specify a `search_query`, if you DO need to search the internet to best answer the question.
        * Specify a list of search strings for `search_query` if you need a set of statistics, indicators, metrics on a set of entities: include a search string for each combo.
        * Leave `search_query`=[], if you DO NOT need to search the internet.
    In the case of unknown quantities in either the `question` or the `search_query`, to be supplied by answering the questions `dependencies` or searching, 
    place square brackets,'[' and ']' around a descriptive term or phrase describing the unknown.
    """

    id: str = Field(..., description="Unique id of the query")
    question: str = Field(...,description="Question we are asking.")
    dependencies: List[str] = Field(
        default_factory=list,
        description="List of at most 9 sub-questions that need to be answered or searched before we can ask the question. Use a subquery when anything may be unknown, and we need to ask multiple questions to get the answer. Dependences must only be other queries.",
    )
    search_query: Optional[List[str]] = Field(default=[],description="Search queries seeking latest information so as to provide context for and improve the answer to the `question`. If have more than one entity or indicator to search for, specify a list of search queries over all entity & indicator combinations.")
        
    @field_validator('dependencies')
    @classmethod
    def validate_dependencies(cls, v, validation_info, **kwargs):
        dependencies = v
        # 'values' is a dictionary containing the field values of the model
        values = validation_info.data
        id = values.get('id', '')
        MAX_DEPENDENCIES = 9
        if len(dependencies) > MAX_DEPENDENCIES:
            raise ValueError(f"""Field `dependencies` has more than {MAX_DEPENDENCIES}, which is the maximumn number allowed.
Fix this by doing one or both of the following:
1. Create a `Query` node with a longer list of search queries in its `search_query` field so that more entitities & statistics can be gathered into a single node, and assign a "Tabulate..." statement to its `question` field.
2. Create an intermediate layer of new `Query` nodes as dependencies:
   * Subset the current list of `dependencies` into logical sets of no more than {MAX_DEPENDENCIES} `Query` nodes each according to the `question` field of each node in the list.
   * For each subset, create a new `Query` node with its field `dependencies` set to the `Query` nodes in the subset and an appropriate `question` field.
   * Assign the list of these new set of `Query` nodes as the fixed `dependencies` field.""")
        return dependencies
     
    async def aexecute_dynamic(self, sub_answers: MergedResponses,client, model="gpt-3.5-turbo", print_level=0) -> ComputeQuery:
        """
        Executes the task by asking the question and returning the answer.
        Dynamically generates the search query for any node that has both a search_query string and at least 1 dependency.
        """
        
        def fill_in_dynamic(query,context):
            system_prompt = """You are an expert at filling in unknown entities, enclosed by square brackets '[]', with the correct values gathered from <<CONTEXT>>. 
Based upon <<CONTEXT>>, generate a <<QUERY STRING>> by replacing any unknowns in the <<PROPOSED QUERY STRING>> with the correct information drawn from <<CONTEXT>>.
The new <<QUERY STRING>> should have a resolved term or phrase in place of the unknown.
Respond with only the <<QUERY STRING>> and no other text. 
DO NOT attempt to answer the <<PROPOSED QUERY STRING>> or the new <<QUERY STRING>>.
If the <<PROPOSED QUERY STRING>> does not contain unknowns and so is perfectly fine, just respond with the <<PROPOSED QUERY STRING>>."""
            user_prompt = f"""
<<CONTEXT>>: 
{context}
---------------------------------------------------------------------------------------------------------
<<PROPOSED QUERY STRING>>: 
{query}
---------------------------------------------------------------------------------------------------------
<<QUERY STRING>>:
"""
            response = client.chat.completions.create(
                model="gpt-3.5-turbo", 
                temperature=0, 
                messages=[ {'role':'system', 'content': system_prompt},{'role':'user', 'content': user_prompt}],
                max_tokens=10000).choices[0].message.content
            
            return response


        def search_for_context(search_query_original,context='',compound=False,print_level=0):
            search_query = search_query_original
            if compound:
                if re.search(r'\[[^\]]+\]',search_query):
                    # Dynamically generate the search query by calling an LLM to construct the search query
                    if print_level > 1:
                        print(f"\t\t====== dynamic search query generation for: '{search_query_original}' ========")
                        print(f"\t\t=== using context: '{context}' ========")
                    search_query = fill_in_dynamic(query=search_query_original, context=context)
                    if print_level > 1:
                        print(f"\t\t====== dynamic search query result: '{search_query}' ========")


            # Retrieve the answer to the search query from the search_cache, if present; else perform a Google Serper API search
            search_context = search_cache.get(search_query,None)
            
            if search_context is None:
                # Perform an internet search to add context to the query.
                if print_level > 1:
                    print(f"...performing search: {search_query} ...")
                search_context = str(search_GoogleSerperAPI(topic = search_query,raw_results=True))
                search_cache[search_query] = search_context
            else:
                if print_level > 1:
                    print(f"...retrieved search from cache: {search_query} ...")

            # Augment the existing context (possibly from dependencies, else empty) with the search results `search_context`
            if compound:
                if len(search_context)>0:
                    #context += '\n\n' + search_context
                    # Must add in search_context so that context remains a valid dictionary when executing eval(context)
                    context = f"{{'merged_responses': {context},\n\n'search_results': {search_context} }}"
                if print_level > 1:
                    print('\t... done gathering subqueries *')
            else:
                context = search_context
                
            return (context,search_query)

        
        
        if print_level > 0:
            print("Executing",f"`{self.question}`")
        length_dependencies = len(sub_answers.responses)
        compound = length_dependencies > 0
        query_string = self.question
        do_search = len(self.search_query)>0
        search_context=''
        sleep_time = 1

        if compound:
            if print_level > 1:
                print("\tExecuting with", length_dependencies, "dependencies")
                print(f"\t* Gathering subqueries for {self.question}...")
            # Get the json equivalent of the MergedResponses object
            #sub_answers_text = sub_answers.model_dump_json()
            
            dict0 = sub_answers.model_dump()
            dict0['responses'] = [ {key: re.sub(r"Context: ##`.+?`##","",value,flags=re.DOTALL) for key,value in cq_dict.items() if key in ['task_id','query','response']} for cq_dict in dict0['responses'] ]
            cqs = [ ComputeQuery(**cq_dict) for cq_dict in dict0['responses'] ]

            # This should rarely happen, but... check if potentially too much context.
            total_chars = sum([ len(cq.response) for cq in cqs ])
            if total_chars > 20000:
                cqs_truncated = []
                for cq in cqs:
                    response_split = cq.response.split(' ')
                    if len(response_split) > 485: # If too long, truncate response
                        if print_level > 1:
                            print(f"\t\t... truncating id={cq.task_id} ...")
                        response_split = response_split[:485] + ['...']
                    cq.response = ' '.join(response_split)
                    cqs_truncated.append(cq)
                cqs = cqs_truncated
                
            sub_answers_text = MergedResponses(responses=cqs).model_dump_json()
            
            # Strip away the contexts of the subqueries, which often are identical or at least share a lot of text in common among all of them.
            #sub_answers_text = re.sub(r"Context: ##`.*?`##","",sub_answers_text,flags=re.DOTALL)

            # At this point context is a string that will return a valid dictionary when executing eval(context)
            context = sub_answers_text 
            sleep_time = 2
            if re.search(r'\[[^\]]+\]',query_string):
                # Dynamically generate the query string by calling an LLM to construct replace missing info between square brackets with the correct info
                if print_level > 1:
                    print(f"\t\t====== dynamic query string generation for: '{query_string}' ========")
                    print(f"\t\t=== using context: '{context}' ========")
                    query_string = fill_in_dynamic(query=query_string, context=context)
                if print_level > 1:
                    print(f"\t\t====== dynamic query string result: '{query_string}' ========")
        else:
            context = ''

        if do_search:
            if isinstance(self.search_query,str):
                context,srch_qry = search_for_context(search_query_original=self.search_query,context=context,compound=compound,print_level=print_level)
                self.search_query = srch_qry
            elif isinstance(self.search_query,list):
                context_list = []
                srch_qries = []
                for sq in self.search_query:
                    cntxt,srch_qry = search_for_context(search_query_original=sq,context=context,compound=compound,print_level=print_level)
                    context_list.append(cntxt)
                    srch_qries.append(srch_qry)
                context = f"[{','.join(context_list)}]"
                self.search_query = srch_qries
            if print_level>1:
                print(f'***********{self.search_query}************')

        # Augment the LLM prompt query with the context, if any
        if len(context)>0:
            query_string = f"{query_string}\nContext: ##`{context}`##"

        # Call the LLM to generate a response to the query
        if print_level > 1:
            print(f"\t.... **computing query** for '{self.question}' ({model}) ...")
        
        with init_context({'client': client, 'model': model}):
            resp = ComputeQuery(task_id=self.id, query=query_string)
        await asyncio.sleep(sleep_time)
        
        if print_level > 2:
            pprint(resp.model_dump())
        if print_level > 0:
            print(f"... done with '{self.question}' ({model}) ******")
        
        return resp



### Query Planning

In [ ]:
class Query0:
    def __init__(self, id, dependencies):
        self.id = id
        self.dependencies = dependencies

def dfs(query, visited, all_queries):
    visited.add(query.id)
    for dep_id in query.dependencies:
        if dep_id in all_queries and dep_id not in visited:
            dfs(all_queries[dep_id], visited, all_queries)

def find_unreachable_queries(queries):
    if not queries:
        return []

    all_queries = {query.id: query for query in queries}
    visited = set()

    # Start DFS from the first query
    dfs(queries[0], visited, all_queries)

    # Find queries that were not visited
    unreachable = [query for query in queries if query.id not in visited]

    return unreachable

def dfs_detect_loop(query, visited, rec_stack, all_queries):
    visited.add(query.id)
    rec_stack.add(query.id)

    for dep_id in query.dependencies:
        if dep_id in all_queries:
            if dep_id not in visited:
                if dfs_detect_loop(all_queries[dep_id], visited, rec_stack, all_queries):
                    return True
            elif dep_id in rec_stack:
                # Found a node in the recursion stack: cycle detected
                return True

    rec_stack.remove(query.id)
    return False

def detect_loops(queries):
    all_queries = {query.id: query for query in queries}
    visited = set()
    rec_stack = set()

    for query in queries:
        if query.id not in visited:
            if dfs_detect_loop(query, visited, rec_stack, all_queries):
                return True  # Loop detected

    return False  # No loop found

# Example usage
queries = [
    Query0("A", ["B", "C"]),
    Query0("B", ["C"]),
    Query0("C", ["A"]),  # This creates a loop
    Query0("D", ["A"]),
    Query0("E", [])
]

print("Loop detected:", detect_loops(queries))

# Example usage
queries = [
    Query0("A", ["B", "C"]),
    Query0("B", ["C"]),
    Query0("C", []),
    Query0("D", ["A"]),
    Query0("E", [])
]

unreachable = find_unreachable_queries(queries)
for query in unreachable:
    print(query.id)


In [ ]:
#query_graph = assessment_orient_express2['plan'].query_graph
#unreachable = find_unreachable_queries(query_graph)
#for query in unreachable:
#    print(query.id)

In [ ]:
class StatusReport(BaseModel):
    """
    Status report of the current plan under execution.
    Given the JSON of the current plan, perform these 3 steps:
    * `status` -- Assess how well the plan is progressing towards the root question.
    * `reasoning` -- Explain your reasoning behind your `status` response, and list suggested improvements after a heading '**Suggestions**:', assuming queued up queries will work fine when run. 
    * `suggestions` -- Convert any improvements in '**Suggestions**:' section of `reasoning` into a list of new `Query` nodes; or `None`, if no improvements necessary.
      - DO NOT include `Query` nodes in improvements that are already in the plan (executed or queued up).
    """
    status: str = Field(...,description="Assessment of adequacy of the current plan (executed and queued up queries) to answer the root question. Judge how well the plan is executing towards its goal. Consider whether the `response` fields include sufficient data & information in general to fulfill the purposes of the questions for whom they are one of the dependencies.")
    reasoning: str = Field(...,description="Detailed explanation of the step-by-step logical reasoning used to arrive at the `status`. If make suggestions, include '**Suggestions**:...' and list them.")
    suggestions:  Optional[List[Query]] = Field(...,description="If necessary, a list of new `Query` nodes (with appropriate `dependencies` and `search_query` fields) to implement `reasoning` field '**Suggestions**'. Else, `None`.")

    @field_validator('suggestions')
    @classmethod
    def generate_suggestions(cls, v, validation_info, **kwargs):
        # 'values' is a dictionary containing the field values of the model
        values = validation_info.data
        reasoning = values.get('reasoning')
        #print(f'\n------------\n{reasoning}\n------------\n')
        
        if (v is None):
            if reasoning:
                search_substring = "**Suggestions**"
                if search_substring.lower() in reasoning.lower():
                    raise ValueError(f"The `reasoning` field proposes '**Suggestions**', so the `suggestions` field cannot be `None`. It must include `Query` nodes to implement them:\n`reasoning`={reasoning}")
        return v

In [ ]:
class QueryPlan(BaseModel):
    """
    A query plan to answer a main 'root' question. Make sure every question is asked only once. Capture the plan as a textual outline in `query_plan_outline`.   
    First, define `search_query_for_context`, and use its automatically generated search results in `context` as you build the plan.
    The 'root' of the query tree and must appear as the first item in `query_graph` and in the 'query_plan_outline`.
    For all nodes requiring current data and the latest information, use `search_query` fields as lists of necessary internet search queries. Tabulate entities by metrics for comparisons. 
    
    It is better to have few `Query` nodes, each with long `search_query` lists and a "Tabulate..." statement for `question`, than to have many `Query` nodes, each with a 1-element `search_query` list.

    """

    search_query_for_context: str = Field(...,description="An internet search query to provide context for the plan.")
    context:  Optional[str] = Field(default=None, validate_default=True,description="Use this context as you build the plan. First, set search_query_for_context. Then this is set automatically as the result of searching the internet with query in search_query_for_context. ")
    query_plan_outline: str = Field(...,description="An indented outline of the query plan matching the query tree, where each entry is the query to be posed.")
    query_graph: List[Query] = Field(..., description="A list of `Query` nodes: First, the original question we are asking, its dependencies, and so on, as a query graph. Refer to `context` for background context in generating the query graph.")

    @field_validator('context')
    @classmethod
    def generate_context(cls,v,validation_info,**kwargs):
        if v is None:
            context = v
        else:
            try:
                eval(v)
                context = v # Must be a valid python expression (actually, a raw search result from search_GoogleSerperAPI()
            except Exception:
                context = None

        values = validation_info.data
        #if context is None:
        search_query_for_context = values.get('search_query_for_context')
        if context is None:
            if search_query_for_context:
                context = str(search_GoogleSerperAPI(topic = search_query_for_context,raw_results=True))
            else:
                raise ValueError("Field `context` cannot be set until `search_query_for_context` is specified. Then `context` is set automatically by searching the internet with search_query_for_context as the search query string.")
        #else:
            #raise ValueError("Field `context` must be set by specifying `search_query_for_context` first. Then `context` is set automatically by searching the internet with search_query_for_context as the search query string.") 

        return context
    
    @field_validator('query_graph')
    @classmethod
    def validate_query_graph(cls, v, validation_info, **kwargs):
        query_graph = v
        # 'values' is a dictionary containing the field values of the model
        values = validation_info.data
        if query_graph:
            no_path = find_unreachable_queries(query_graph)
            if len(no_path) > 0:
                raise ValueError(f"""All `Query` nodes in field `query_graph` must be on the `dependencies` path from the `root` question `Query` node, which should be the first node in the `query_graph`. Currently, these nodes are not: {no_path}.\n
    They do not have to be direct dependencies of the `root` question but, if not, they must be among the dependencies of the dependencies of the dependencies, etc.""")
            if detect_loops(query_graph):
                raise ValueError(f"""Field `query_graph` should NOT contain loops. A loop exists if at least one `Query` node has a path through its `dependencies` and their `dependencies`, and so on, that leads back to that node.""")
        return query_graph

    
    def _get_execution_order(self) -> List[int]:
        """
        Returns the order in which the tasks should be executed using topological sort.
        Copied from: https://github.com/jxnl/instructor/blob/main/examples/task_planner/task_planner_topological_sort.py
        Inspired by https://gitlab.com/ericvsmith/toposort/-/blob/master/src/toposort.py
        """
        tmp_dep_graph = {item.id: set(item.dependencies) for item in self.query_graph}

        def topological_sort(
            dep_graph: dict[int, set[int]],
        ) -> Generator[set[int], None, None]:
            while True:
                ordered = set(item for item, dep in dep_graph.items() if len(dep) == 0)
                if not ordered:
                    break
                yield ordered
                dep_graph = {
                    item: (dep - ordered)
                    for item, dep in dep_graph.items()
                    if item not in ordered
                }
            if len(dep_graph) != 0:
                raise ValueError(
                    f"Circular dependencies exist among these items: {{{', '.join(f'{key}:{value}' for key, value in dep_graph.items())}}}"
                )

        result = []
        for d in topological_sort(tmp_dep_graph):
            result.extend(sorted(d))
        return result


    
    def strip_context(cq):
        cq.response = re.sub(r"Context: ##`.+?`##","",cq.response,flags=re.DOTALL)
        return cq


    async def execute_dynamic(self,client, model = "gpt-3.5-turbo",status_updates=False,print_level=0) -> dict[int, ComputeQuery]:
        """
        Executes the queries in the query plan in the correct order using asyncio and chunks with answered dependencies.
        Copied from: https://github.com/jxnl/instructor/blob/main/examples/task_planner/task_planner_topological_sort.py
        Dynamically generate the query string for any node that has at least 1 dependency and
        if the question contains any unknowns enclosed by '[]'.
        Dynamically generate the search query for any node that has both a search_query string and at least 1 dependency and
        if the search query contains any unknowns enclosed by '[]'.
        """
        execution_order = self._get_execution_order()
        tasks = {q.id: q for q in self.query_graph}
        task_results = {}
        status = None
        while True:
            ready_to_execute = [
                tasks[task_id]
                for task_id in execution_order
                if task_id not in task_results
                and all(
                    subtask_id in task_results for subtask_id in tasks[task_id].dependencies
                )
            ]
            if print_level > 1:
                # prints chunks to visualize execution order
                print(f'*** QUEUE: {len(ready_to_execute)} item(s)')
                print(ready_to_execute)
                print('***')

            computed_answers = await asyncio.gather(
                *[
                    q.aexecute_dynamic(
                        sub_answers=MergedResponses(
                            responses=[
                                result #strip_context(result)
                                for result in task_results.values()
                                if result.task_id in q.dependencies
                            ]
                        ),
                        client=client,
                        model = model,
                        print_level = print_level
                    )
                    for q in ready_to_execute
                ]
            )
            for answer in computed_answers:
                task_results[answer.task_id] = answer

            if status_updates:
                if status:
                    prior_suggestions = f"\n\n::PRIOR SUGGESTED NODES::\n{status.suggestions}\n\n"
                else:
                    prior_suggestions = ""
                # Determine progress towards goal.
                plan_dict = {id: 
                             {'question': re.sub(r'Context: ##`.+?`##',"",task_results[id].query,flags=re.DOTALL) if id in task_results else tasks[id].question, 
                              'search_query': tasks[id].search_query, 
                              'response': ("**DONE**:\n" + task_results[id].response) if id in task_results else '**IN THE PROCESSING QUEUE**',
                              'dependencies': tasks[id].dependencies} 
                             for id in execution_order}
                plan_string = json.dumps(plan_dict,indent=4)
                query=f"""
::PLAN::
{plan_string}
{prior_suggestions}
"""
                status: StatusReport = client.chat.completions.create(
                    model=model, 
                    temperature=0, 
                    messages=[
                         {'role':'system',
                         'content':f"""You are an expert at logical and probabilistic reasoning. 
Use step-by-step logical reasoning to assess how well the given ::PLAN:: query graph is progressing towards its goal of resolving the 'root' question.
Consider how comprehensive, thorough, balanced, and intellectually honest the ::PLAN:: is.  
For example, does it comprehensively consider all alternatives?
Does it collect evidence for and against propositions?
Also, note how successful the completed and queued portions of ::PLAN:: are at meeting their goals. 
For example, are there many pieces of missing information in tables?
If there are any gaps or shortcomings, suggest potential new `Query` nodes that could fill in these gaps, and describe
them as a '**Suggestions**' list in the 'reasoning' field and generate them as a list of `Query` nodes in the `suggestions` field. 
Be sure to assign appropriate `dependencies` and `search_query` fields for any suggested `Query` nodes.
DO NOT suggest nodes that currently exist in the plan as completed or queued.
DO NOT suggest nodes that are tangential or irrelevant to the central issue: '{prop}'."""},
                        {'role':'user', 'content': query}
                    ], 
                    response_model=StatusReport,
                    max_retries=2,
                    max_tokens=3000
                )
                #print(status.suggestions)
                suggestion_string = "None" if status.suggestions is None else json.dumps({q.id: q.model_dump() for q in status.suggestions},indent=4)                
                status_report_string = '\n\n'+('-'*150)+'\n\n' + f"**Status**: {status.status}\n\n**Reasoning**: {status.reasoning}\n\n**Suggested Nodes**:\n{suggestion_string}" + '\n\n'+('-'*150)+'\n\n'
                print(status_report_string)
            
            if len(task_results) == len(execution_order):
                break
        return {k: task_results[k] for k in execution_order}

    def dependencies(self, idz: List[int]) -> List[Query]:
        """
        Returns the dependencies of the query with the given id.
        """
        return [q for q in self.query_graph if q.id in idz]



In [ ]:
#Query.model_rebuild()
#QueryPlan.model_rebuild()


def query_planner(question: str,client) -> QueryPlan:
    PLANNING_MODEL = "gpt-4-turbo-preview"

    messages = [
        {
            "role": "system",
            "content": """You are a world-class query planning algorithm capable of breaking apart a question into its dependency queries and search queries for current data such that the answers can be used
to inform the parent question. Do not answer the questions, simply provide a query plan as a correct compute graph with a Query node for the parent question at the top and all other Query nodes connecting to
it directly as its dependencies or indirectly as the dependencies of its dependencies, and so on.

When building the query plan, carefully consider whether a question requires an internet search for the latest information and current data. If so, then define a Query node with a search_query field
that has an appropriate list of internet search queries to obtain the information. If searching for a set of entities or concepts and statistics or metrics for each, tabulate the results.

Before you define Query nodes, think step-by-step using logical reasoning to get a better understanding of the problem.
Be sure to  define the `search_query` field for any steps in the query plan that require the latest information obtainable by internet searching before answering its question.
Review the `query_graph` and make sure that the `dependencies` of each Query node are truly just those nodes that directly inform that node's `question`, i.e. consider conditional independence properties.
Fix the dependencies throughout the graph to ensure that this is indeed the case.""",
        },
        {
            "role": "user",
            "content": f"Consider: {question}\n\nGenerate the correct query plan.",
        },
    ]

    root = client.chat.completions.create(
        model=PLANNING_MODEL,
        temperature=0,
        response_model=QueryPlan,
        messages=messages,
        max_retries = 2,
        max_tokens=4000,
    )
    return root

In [ ]:
def improve_upon_plan(plan) -> QueryPlan:
    PLANNING_MODEL = "gpt-4-turbo-preview"

    messages = [
        {
            "role": "system",
            "content": """You are a world-class query-planning algorithm. Your task is to take a previously generated query plan identified by the JSON provided to you as ::PLAN:: and to improve upon it.
The improved plan must have the exact same "root" question (i.e., top proposition) as does the original plan. However, other `Query` nodes in the plan may change if you think it is absolutely necessary for an improved plan.
If the query plan is satisfactory as presented in ::PLAN::, then merely copy it.

Use step-by-step logical reasoning to apply the IMPROVEMENT STRATEGY below to build up an improved query plan.

IMPROVEMENT STRATEGY:
* Improve and apply the search context.
  * Make sure that the `search_query_for_context` field is written so as to obtain the most relevant background information needed to best define the `question` and `search_query` fields
    of the `Query` nodes supplying the most definitional concepts and entities for the context of the entire reasoning behind the query plan.
* Consolidate `Query` nodes on parallel topics (e.g., collecting similar statistics for comparable entities) and improve search query lists.
  * Make sure that rather than having many separate `Query` nodes, each with a singleton list as its `search_query` field, that instead such comparable `Query` nodes as a set are consolidated
    into a single `Query` node with a list of all of those search queries as its `search_query` field, and that as its `question` field it has a "Tabulate..." statement (or "Enumerate..." statement) 
    to consolidate the information into a single response.
  * DO NOT consolidate `Query` nodes on disimilar, non-parallel topics.
* Improve dependency structure.
  * Make sure that the `Query` node `dependencies` fields are arranged such that the `Query` nodes that identify the most basic, definitional concepts, and entities
    appear in the `dependencies` field of `Query` nodes  that tabulate statistics or metrics on them and/or assess them.
  * Make sure that the `Query` node `dependencies` fields are arranged such that the `Query` nodes that identify the critical comparative statistics, metrics and indicators of the entities
    supplying evidence appear in the `dependencies` field of `Query` nodes that consolidate, tabulate and compare them.
  * Make sure that the `Query` nodes that are entity-comparing or statistics-tabulating appear in the `dependencies` fields of `Query` nodes that propose or critique propositions.
  * Apply the concept of conditional independence so that `Query` nodes need not include basic/primitive/evidential nodes in their `dependencies` list if such nodes appear as `dependencies` of the
    `dependencies`, and so on.  As long as ALL nodes are on a `dependencies` pathway from the top proposition (first) `Query` node in the `query_graph`.
* Rewrite the outline, if necessary.
  * Make sure that the `query_plan_outline` is rewritten to reflect the new improved dependency structure.
""",
        },
        {
            "role": "user",
            "content": f"""
--------------------------
Consider ::PLAN::
{plan.model_dump_json()}

--------------------------

Improve upon the query plan.""",
        },
    ]

    root = client.chat.completions.create(
        model=PLANNING_MODEL,
        temperature=0,
        response_model=QueryPlan,
        messages=messages,
        max_retries = 2,
        max_tokens=4000,
    )
    return root

## Report Generation Functions

In [ ]:
def generate_graph(input) -> KnowledgeGraph:
    """
    Use step-by-step logical reasoning to explain the input topic in detail.
    Deeply explore the central propositions & principles, the pre-suppositions & axiomatic principles that lead to them, and the consequences & conclusions that result from them.
    Iteratively recurse through the multiple chains or branches of these cause-effect representations of the causal mechanisms behind the topic.
    Repeat this logical reasoning at each level of relationships along the multiple branches/chains until the most thorough, complete, and accurate
    knowledge graph you can conceive of has been built.
    Respond, ultimately, with a detailed knowledge graph of nodes, representing the concepts, and edges, capturing the relationships between the concepts.
    """
    return client.chat.completions.create(
        model="gpt-4-turbo-preview", #"gpt-3.5-turbo",
        messages=[
            {
                "role": "user",
                "content": f"Help me understand the following by describing it as a detailed knowledge graph: {input}",
            }
        ],
        response_model=KnowledgeGraph,
    )  # type: ignore

In [ ]:
from graphviz import Digraph

def visualize_knowledge_graph(kg: KnowledgeGraph,filename_stem='knowledge_graph',aspect_ratio: str ='auto',flip=False):
    if flip:
        dot = Digraph(comment="Knowledge Graph", graph_attr={'rankdir': 'BT','ratio': aspect_ratio})
    else:
         dot = Digraph(comment="Knowledge Graph", graph_attr={'rankdir': 'BT','ratio': aspect_ratio})

    # Add nodes
    for node in kg.nodes:
        dot.node(str(node.id), node.label, color=node.color)

    # Add edges
    for edge in kg.edges:
        dot.edge(str(edge.source), str(edge.target), label=edge.label, color=edge.color)

    # Render the graph
    dot.render(f"{filename_stem}_gv", view=False)
    return dot

In [ ]:
import textwrap
def convert_to_graph(plan: QueryPlan,proposition: str,label_wrap=20,label_edges=False) -> KnowledgeGraph:
    """
    Convert a QueryPlan object into an equivalent KnowledgeGraph object.
    """
    
    kb_node_ids = {}
    def create_kb_node(kb_id,qp_id,query_node):
        """
        Create a new Node object for the knowledge graph from the QueryNode object from the query plan.
        kb_id is the new node id; qp_id is the given query node id within the query plan.
        """
        kb_node_ids[qp_id] = kb_id
        if len(query_node.search_query) == 0:
            color='black'
            description = 'proposition'
        else:
            color='red'
            description = f'search query: "{query_node.search_query}"'

        node_label = textwrap.fill(query_node.question,width=label_wrap).replace('\n','\\n')
        kb_node = Node(id=kb_id,label=node_label,description=description,color=color)
        return kb_node

    # Create a dictionary so that we can retrieve query nodes by their query plan id
    queries = {q.id: q for q in plan.query_graph}

    # Optionally, set edge labels to 'depends on'
    if label_edges:
        edge_label = 'depends on'
    else:
        edge_label = ''

    # Initialize tracking of knowledge graph nodes and edges
    kb_node_id = 0
    kb_edge_id = 0
    kb_nodes = []
    kb_edges = []

    # Convert each query node to a knowledge graph node.
    for q_id,q in queries.items():
        q_id = q.id
        q_kb_id = kb_node_ids.get(q_id,None)
        if q_kb_id is None: # if not seen this node yet, create a knowledge graph node
            kb_node_id += 1
            q_kb_id = kb_node_id
            q_kb_node = create_kb_node(q_kb_id, q_id, q)
            kb_nodes.append(q_kb_node)

        # Add an edge to the knowledge graph for each of the dependencies of the query node
        if q.dependencies is not None:
            for d_id in q.dependencies:
                d_kb_id = kb_node_ids.get(d_id,None)
                if d_kb_id is None: # if not seen this node yet, create a knowledge graph node
                    kb_node_id += 1
                    d_kb_id = kb_node_id
                    d_kb_node = create_kb_node(d_kb_id, d_id, queries[d_id])
                    kb_nodes.append(d_kb_node)
                # Create the new edge from the dependency to the query node
                d_edge = Edge(target=q_kb_id,source=d_kb_id,label=edge_label,description='')
                kb_edges.append(d_edge)
                
    # Create the knowledge graph from the nodes and edges
    kb = KnowledgeGraph(topic=proposition,nodes=kb_nodes,edges=kb_edges)
    return kb

In [ ]:
def make_report(computed_query,search_dict={}) -> str:
    """
    Create the text of an R Markdown file (i.e., markdown plus YAML header) representing a report
    of the results of the `computed_query` and incorporating the search results, if any, given by
    `search_dict`.
    """
    
    query_with_context = computed_query.query
    response = computed_query.response
    query = re.sub(r"(.+)\nContext: ##`.+?`##",r"\1",query_with_context,flags=re.DOTALL)
    query = re.sub(r"Is '(.+)' the truth\?",r"\1",query,flags=re.DOTALL)
    context = eval(re.sub(r".+Context: ##`(.+?)`##",r"\1",query_with_context,flags=re.DOTALL))['responses']

    section_text = "# Sub-Propositions\n\n"
    for r in context:
        section_text += f"## {r['query']}\n\n{r['response']}\n\n"
        
    markdown_text = f"""---
title: "Proposition: '{query}'"
author: "GPT-4, with Michael L. Thompson"
date: "`r Sys.Date()`"
output:
  pdf_document:
    toc: yes
    toc_depth: 4
linkcolor: red
urlcolor: blue
---

```{{r setup, include=FALSE}}
knitr::opts_chunk$set(echo = TRUE)
```
\\newpage


"""
    
    markdown_text += f"# Proposition: \"{query}\"\n\n{section_text}\n\n# Conclusion\n\n{response}"
    
    if len(search_dict)>0:
        search_text = "\n\n# Search Results\n\n"
        for k,search_result in search_dict.items():
            if len(search_result)>0:
                search_text += f"## \"{k}\"\n\n{search_result}\n\n"
        markdown_text += search_text
        
    return markdown_text

#print(make_report(computed_query = plan_college2a_result))

In [ ]:
from thefuzz import fuzz

def outline_processing(plan: QueryPlan, verbose: bool = False):
    return [q.id for q in plan.query_graph]

def outline_processing0(plan: QueryPlan, verbose: bool = False):
    """
    Sort the id's of the plan.query_graph Query objects according to their logical order as dictated by the plan.query_outline.
    Returns a list of the re-ordered id's of the associated Query object in the plan query_graph.
    """

    #plan = assessment_explain_wt_evidence['plan']
    # strip off the context info associated with each query.
    q_dict = {q.id: q.question for q in plan.query_graph}
    outline = plan.query_plan_outline.split('\n')
    
    # Split the plan query_plan_outline into separate lines and process each one.
    res = {}
    best_score = {}
    for id,qry in q_dict.items():
        #print((id,qry))
        # Use similarity scoring between the outline's line and the query nodes' queries, ignoring order of words.
        scores = [ fuzz.token_sort_ratio(qry, line) for line in outline ]
        #print(scores)
        # Find the highest scoring query
        max_score = max(scores)
        #if max_score > 60:
        i_best_match = scores.index(max_score)
        res[id] = i_best_match
        best_score[id] = max_score
        #else:
            #res[id] = None
            #best_score[id] = None
    
    if verbose:
        i_lines_matched = list(res.values())
        for i,line in enumerate(outline):
            if i in i_lines_matched:
                id = list(res.keys())[i_lines_matched.index(i)]
                print(f"\"{line}\" == \"{q_dict[id]}\" -- [{id}; {best_score[id]}]")
            else:
                print(f"\"{line}\" == ****")
                
    #print(res)
    nmax = len(plan.query_graph)
    node_dict = {q.id: q for q in plan.query_graph}
    res_frac = {id: (i + 0.1*(1 - len(node_dict[id].dependencies)/nmax)) for id,i in res.items()}
    #print(res_frac)
    
    num_list = list(res_frac.values())
    # Sorting the list and getting indices
    ordering = [index for index, value in sorted(enumerate(num_list), key=lambda x: x[1])]
    
    # [num_list[o] for o in ordering] will now give the sorted list
    sorted_list = [num_list[o] for o in ordering]
    #print(ordering)  # This will print the indices in sorted order
    #print(sorted_list)  # This will print the sorted values
    res_frac_sorted = {id: round(v,2) for id,v in zip([ list(res_frac.keys())[o] for o in ordering], sorted_list)}
    #print(res_frac_sorted)
    
    if verbose:
        for id in list(res_frac_sorted.keys()):
            print(node_dict[id].question)

    return list(res_frac_sorted.keys())

In [ ]:
#outline_processing(assessment_explain_wt_evidence['plan'],verbose=True)

In [ ]:
def list_deps(id,node_dict,lvl=0):
    node = node_dict[id]
    print((lvl*'\t') + f'{node.question} -- [{node.id}]')
    for did in node.dependencies:
        list_deps(did,node_dict,lvl+1)

#id_sorted = outline_processing(assessment_explain_wt_evidence['plan'],verbose=False)
#print(id_sorted)
#list_deps('main',{q.id: q for q in assessment_explain_wt_evidence['plan'].query_graph})
##print(response_outline(task_id='main',computed_queries={id: assessment_explain_wt_evidence['plan_result'][id] for id in id_sorted,level=0))

In [ ]:
#print(list(plan_civic2_result.keys()))

def response_outline(task_id,computed_queries: dict,dependencies, level=0, done_already=[],verbose=False):
    computed_query = computed_queries[task_id]
    query_with_context = computed_query.query
    response = computed_query.response
    query = re.sub(r"(.+)\nContext: ##`.+`##",r"\1",query_with_context,flags=re.DOTALL)
    query = re.sub(r"Is '(.+)' the truth\?",r"\1",query)
    if verbose:
        print("report: query = " + query )
    #print(query_with_context)
    
    if ('Context' in query_with_context):
        # As set by the Query.aexecute() treatment of context, which may be only merged_responses or may be both merged_responses & search_results.
        # We ignore search_results because they're handled by a separate 'Searches' section in the report.
        dict_str = re.sub(r".*Context: ##`(.+)`##",r"\1",query_with_context,flags=re.DOTALL) #.replace('\\"','"')
        try:
            full_context = eval(dict_str)
        except:
            dict_str2 = re.sub('`##".+','',dict_str,flags=re.DOTALL)
            dict_str2 = dict_str2.replace('\\\"','\"').replace('\\\n','\n')
            dict_str2 = f"""{dict_str2}"""
            full_context = eval(dict_str2)

        if isinstance(full_context,list):
            #print(full_context)
            search_results = None
            full_context = {}
        else:
            search_results = full_context.get('search_results',None) # check if merged_responses & search_results in context
            
        if search_results is None: # no search_results, implies only merged_responses
            merged_responses = full_context
        else: # must get merged_responses from full_context
            merged_responses = full_context.get('merged_responses',None)
            
        context = merged_responses.get('responses',None) # just to be sure, see if empty
    else:
        context=None

    hashes = "#"*(level+1)
    if context is None:
        markdown_text = f"{hashes} \"{query}\"\n\n{response}"
    else: # When context is given, this means that the node had sub-propositions that we need to report
        if level==0:
            section_text = hashes + " Sub-Propositions\n\n"
        else:
            section_text = '\n\n'

        if verbose:
            print('\t\tContext:')
            for r in context:
                print('\t'*3 + r['task_id'])
                
        context_dict = {r['task_id']: r for r in context if r['task_id'] not in done_already }
        context_dict = {id: context_dict[id] for id,cq in computed_queries.items() if id in context_dict}
        for r in list(context_dict.values()):
            #print(r)
            # Recurse: treat the sub-proposition, handling sub-sub-propositions, and so on...
            section_text += response_outline(r['task_id'],computed_queries,dependencies,level=level+1,done_already=done_already,verbose=verbose) + "\n\n" #f"## {r['query']}\n\n{r['response']}\n\n"
            done_already.append(r['task_id'])
            done_already.extend(dependencies[r['task_id']])
        if level==0:
            markdown_text = f"{hashes} Proposition: \"{query}\"\n\n{hashes} Summary\n\n{response}\n\n{section_text}"
        else:
            markdown_text = f"{hashes} {query}\n\n{section_text}\n\n{hashes} Conclusion\n\n{response}"
    
    return markdown_text

In [ ]:
import re
def generate_report(plan_outline,lastnode,computed_queries,dependencies,search_dict, filename_stem, added_instructions=None, return_Rmd=False, write_to_file=True):
    #lastnode = (list(plan_2020election_result2_gpt4.values())[-1])

    #report_markdown_gpt4a = response_outline(task_id=lastnode.task_id,computed_queries=plan_2020election_result2_gpt4,level=0)
    report_markdown = response_outline(task_id=lastnode.task_id,computed_queries=computed_queries,dependencies=dependencies,level=0,done_already=[])
    figure_txt = f'\n\n![Query plan graph.]({filename_stem}_gv.pdf)\n\n'
    #print(report_markdown)
    
    if added_instructions is None:
        added_instructions_section = ''
    else:
        added_instructions_section = f"""
        
## Added Instructions

{added_instructions}


"""

    plan_outline = re.sub(r'(([0-9]+|[A-Za-z]+)\.)+ ','* ',plan_outline,flags=re.DOTALL)

    report_markdown = re.sub(r'(# Proposition: "[^"]+"\n)',r'\1'+ figure_txt + added_instructions_section + '## Query Plan Outline\n\n'+plan_outline+'\n\n',report_markdown)
    print('='*150)
    #print(report_markdown)
    
    query = re.sub(r"(.+)\nContext: ##`.+?`##",r"\1",lastnode.query,flags=re.DOTALL)
    query = re.sub(r"Is '(.+)' the truth\?",r"\1",query,flags=re.DOTALL)

    
    mrkdwn = f"""---
title: "Proposition: '{query}'"
author: "GPT-4, with Michael L. Thompson"
date: "`r Sys.Date()`"
output:
  pdf_document:
    toc: yes
    toc_depth: 4
linkcolor: red
urlcolor: blue
---


^[Proposition: '{query}' [{filename_stem}.pdf] © 2024 by Michael L. Thompson is licensed under CC BY-SA 4.0. To view a copy of this license, visit http://creativecommons.org/licenses/by-sa/4.0/]

\\newpage

"""

    how_done_section = f"""

# How Was This Done? 

Generated by [Michael L. Thompson](https://www.linkedin.com/in/mlthomps) with OpenAI's GPT-4 (`gpt-4-turbo-preview`).

* **Python-based Jupyter Lab Notebook:** The contents of this document were generated through coded prompting of **OpenAI's GPT-4** using the Python package **[`instructor`](https://github.com/jxnl/instructor)** by Jason Liu.
  - Input: the text string "{query}".
  - Output: this report as an R Markdown text file.

* **GPT-4 Plan Generation:** GPT-4 autonomously generated the query plan (using code based off of an `instructor` example). 

* **GPT-4 Plan Execution:** GPT-4 then autonomously executed the plan, which involved the following:
  - writing search queries and initiating calls to **[Serper (Google Search API)](https://serper.dev/)** for searching, 
  - generating responses to the questions in the plan using the searches as context, and 
  - creating all text in the Outline, Summary, and Sub-Propositions sections. 
  
* **Python Report Generation:** The Python code then consolidated GPT-4 responses into an R Markdown text file. 

* **Manual Rendering:** Manually, this section was written, the Query Plan Outline was re-formatted, and, via **RStudio**, the PDF document was rendered.  A few miscellaneous manual formatting corrections were also done. No other content was manually generated or edited.


"""
    if len(search_dict)>0:
        search_text = "\n\n# Search Results\n\n"
        for k,search_result in search_dict.items():
            if len(search_result)>0:
                search_text += f"## \"{k}\"\n\n{search_result}\n\n"
        report_markdown += search_text

    report_markdown += how_done_section
    
    if write_to_file:
        mrkdwn_file_path = f'{filename_stem}.Rmd'
        with open(mrkdwn_file_path, 'w', encoding='utf-8') as file:
            file.write(mrkdwn + to_ascii(report_markdown))

    if return_Rmd:
        report_markdown = mrkdwn + to_ascii(report_markdown)

    return report_markdown

In [ ]:
import pickle
import json
from graphviz import Digraph

def pickle_it(assessment_results: dict, save = False) -> dict:
    """
    Save to disk or load from disk a full proposition assessment analysis, as generated by function `assess_proposition()`,
    based upon the `short_name` saved in the `assessment_results` dictionary, which must contain a key name `short_name`.
    If `save` is True, then saves `assessment_results` to disk with name `f'results_{short_name}.pkl'` -- overwriting it if it already exists -- and returns `assessment_results` as passed in.
    Else, ignores what was passed in as `assessment_results`, and 
    if file of that name exists in current directory, loads and returns the dictionary from that file; else returns None.
    """
      
    short_name = assessment_results['short_name']
    results_filename = f'results_{short_name}.pkl'
    file_path = './' + results_filename
    
    if save:
        results = {}
        results.update(assessment_results)
        dump_dict = {}
        dump_dict.update(assessment_results)
        
        plan_result = assessment_results.get('plan_result',{}) # a dictionary of task_id strings as keys and ComputeQueryGPT4 as values
        if plan_result:
            serialized_plan_result = {task_id: value.model_dump() for task_id,value in plan_result.items() if isinstance(value,BaseModel)}
        else:
            serialized_plan_result = None
        dump_dict['plan_result'] = serialized_plan_result

        dump_dict['kb_dot'] = None # Drop the Digraph object
        
        # get a JSON representation
        # Convert Pydantic models to dictionaries and serialize the entire dictionary
        serialized_dict = {
            key: value.model_dump() if isinstance(value, BaseModel) else value
            for key, value in dump_dict.items()
        }
        
        # Dump the serialized dictionary to a JSON string
        json_string = json.dumps(serialized_dict)
        
        # dump to file
        with open(file_path, 'wb') as state_file:
            pickle.dump(json_string, state_file)
    else:
        exist_file = os.path.isfile(file_path)
        if exist_file:
            # get from file
            with open(file_path, 'rb') as state_file:
                json_string = pickle.load(state_file)
            # Convert the JSON string back into a dictionary
            parsed_dict = json.loads(json_string)
            
            results = parsed_dict
            # should have form: {'short_name': short_name, 'plan': plan, 'plan_result': plan_result, 'search_dict': search_dict,'kb_graph': graph, 'kb_dot': kb_dot, 'report': report_rmarkdown, 'model': model }
            # Reconstruct the Pydantic models
            results['plan'] = QueryPlan(**parsed_dict['plan'])
            plan0 = parsed_dict.get('plan0')
            if plan0:
                results['plan0'] = plan0
            plan_result = parsed_dict.get('plan_result')
            if plan_result:
                results['plan_result'] = { task_id: ComputeQuery(**compute_query_dict) for task_id,compute_query_dict in plan_result.items() }

            kb_graph = KnowledgeGraph(**parsed_dict['kb_graph'])
            results['kb_dot'] = visualize_knowledge_graph(kb_graph,filename_stem=short_name,flip=True, aspect_ratio= 'auto')
        else:
            results = None
    return results

## Proposition Assessment Functions

"Snout to tail...." -- this section takes the entirety of the proposition assessment code and wraps it into a single function `assess_proposition()`.

### Definition of function `assess_proposition2()`

In [ ]:
import re
import warnings
import time

async def assess_proposition2(proposition: str, short_name: str, search_cache: dict,client, use_plan: QueryPlan = None,plan_only: bool = False, added_instructions: str = '', 
                              model: str ="gpt-3.5-turbo",improvement_round=False, report_to_file: bool = False,kb_label_wrap: int = 20, force_run: bool = False, save_pickle: bool = True,status_updates=False,print_level: int = 0):
    """
    Performs an assessment of the proposition using logical reasoning and evidence evaluation.
    Is asynchronous, so must execute with the `await` keyword.
    If `use_plan` is given, then it `proposition` must agree with whatever proposition is specified within the given query plan as `use_plan` for the results to make sense.
    
    Usage example:
    assessment_tesla_y =  await assess_proposition(
        proposition = "Tesla Model Y is the best electric vehicle currently available.", 
        short_name = 'tesla_y', 
        search_cache = search_cache, 
        use_plan = None,
        added_instructions = '', 
        model ="gpt-3.5-turbo", # 'gpt-4-turbo-preview'
        report_to_file = False,
        kb_label_wrap = 20,
        force_run = False,
        save_pickle = True
        
    )
    # The results are dictionary with all of the work generated.
    print(list(assessment_tesla_y.keys()))
    # A report is generated in R Markdown format and returned -- it is also saved to file if `write_to_file` is True.
    print(assessment_tesla_y['report'])
    """

    # Record the start time
    start_time = time.time()
    plan0 = None
    if print_level > 0:
        print("\n\n================================= *** BEGIN *** =================================\n")
    
    if force_run:
        assessment_results = None
        search_dict = None
        report_rmarkdown = None
        plan_result = None
    else:
        assessment_results = pickle_it(assessment_results = {'short_name': short_name}, save = False)
        if assessment_results is None:
            if print_level > 1:
                print(f"*** NO RESULTS FOUND ('results_{short_name}.pkl' not found) -- WILL GENERATE RESULTS ***\n")
            assessment_results = {'short_name': short_name}
        else:
            if print_level > 1:
                print(f"*** RETRIEVED RESULTS FROM 'results_{short_name}.pkl' ***\n")
            
        use_plan = assessment_results.get('plan',use_plan)
        plan_result = assessment_results.get('plan_result',None)
        if plan_result is not None: # make sure model is same as from saved results rather than input argument
            model = assessment_results.get('model',None)
        search_dict = assessment_results.get('search_dict',None)
        report_rmarkdown = assessment_results.get('report_rmarkdown',None)
        
    if use_plan is None: # No plan given and none retrieved from results so erase all results
        plan_result = None
        search_dict = None
        report_markdown = None            
    
        key_question = f"""# Mandate:
Given proposition '{proposition}'. 
1. Enumerate all sub-propositions, and their sub-propositions and so on, that need to be true to support it.
2. Enumerate all factual axiomatic statements as evidence that support or refute each and every one of the propositions. 
3. Record these propositions and evidence as a textual outline (in 'query_plan_outline') describing the query plan.
3. Generate the query plan as a tree of query nodes (in 'query_graph').
   - Assign a separate query node for each specific sub-proposition.
   - Assign a separate query node for each specific piece of evidence needed to confirm and refute each sub-proposition. 
   - Be sure to specify search_flag=True for a query node when needed to gather the latest factual information as evidence.
   - Demand that actual specific data and statistics are gathered as evidence and that the answers to the propositions are stated in terms of these data.
"""
        mandate = f"""
# The Proposition for which we require a Query Plan:
'{proposition}'"""
    
        # Generate the query plan (always uses gpt-4-turbo-preview)
        if len(added_instructions)>0:
            added_text = f'\n\nAdditional Instructions: {added_instructions}\n\n'
        else:
            added_text = ''
            
        if print_level > 0:
            print("*** GENERATING PLAN ***")
        plan = query_planner( key_question + f'{added_text}' + mandate, client )
        if improvement_round:
            if print_level > 0:
                print("*** IMPROVING  PLAN ***")
            plan0 = plan
            plan = improve_upon_plan(plan)
        else:
            plan0 = None
    else:
        plan = use_plan
        dids = []
        for q in plan.query_graph:
            dids.extend(q.dependencies)
        top_q = [q for q in plan.query_graph if q.id not in dids]
        provided_prop = re.sub(r"Is '(.+)' the truth",r"\1",top_q[0].question)
        if print_level > 0:
            print(f"Given `proposition` = '{proposition}'.\nProvided proposition from `use_plan` = '{provided_prop}'.\n")
    
    if print_level > 2:
        pprint(plan.model_dump())

    # Visualize the query graph -- always saves it to disc as a PDF
    graph: KnowledgeGraph = convert_to_graph(plan,proposition,label_wrap=kb_label_wrap)
    kb_dot = visualize_knowledge_graph(graph,filename_stem=short_name,flip=True, aspect_ratio= 'auto')

    # added this because without it sometimes errors later on and fails without saving any results!
    assessment_results = {'short_name': short_name, 'plan': plan,'plan0': plan0, 'search_dict': search_dict,'kb_graph': graph, 'kb_dot': kb_dot, 'report': report_rmarkdown, 'model': model }
    if save_pickle:
        _ = pickle_it(assessment_results = assessment_results, save = True)
        if print_level > 1:
            print(f"... saving results (plan & kb graph) to 'results_{short_name}.pkl' ...")

    if plan_only:
        # Just skip generating results or returning retrieved results, if plan_only is True.
        plan_result = None
        search_dict = None
        report_markdown = None    
    else:
        if plan_result is None:
            # Execute the plan if the results aren't already gotten from file.
            if search_dict is not None:
                searche_cache.update(search_dict)
                
            if print_level > 0:
                print("*** EXECUTING PLAN ***")
            # Execute the plan using the LLM given by `model`
            plan_result = await plan.execute_dynamic(client=client,model=model,status_updates=status_updates,print_level=print_level)
    
        
        last_node_result = list(plan_result.values())[-1]
        # added this because without it sometimes errors occur during report generation and fails without saving any results!
        assessment_results = {'short_name': short_name, 'plan': plan,'plan0': plan0, 'plan_result': plan_result, 'kb_graph': graph, 'kb_dot': kb_dot,  'model': model }
        if save_pickle:
            _ = pickle_it(assessment_results = assessment_results, save = True)
            if print_level > 1:
                print(f"... saving results (without search dictionary and report) to 'results_{short_name}.pkl' ...")
            
        if print_level > 0:
            # Display the top-level query response
            print("\n\n================================= *** RESULTS *** =================================\n")
            print('QUERY: ' + re.sub(r'\\"response(s)*\\"',r'\n"response\1"',last_node_result.query.replace("\\n","\n")).replace('"query"','\n"query"').replace(',"response"',',\n\n* "response"') + '\n\n')
            print('RESPONSE: ' + last_node_result.response)
    
        if search_dict is None:
            # Gather up the results of all searches performed
            #search_keys = [q.search_query for q in plan.query_graph if len(q.search_query)>0]
            if plan.search_query_for_context:
                try:
                    search_dict = { plan.search_query_for_context : summarize_raw_search(eval(plan.context)) }
                except:
                    search_dict = {}
                    warnings.warn("Could not get `plan.context`. Will not include it in search results.", UserWarning)
            else:
                search_dict = {}
            search_keys = [search_query for q in plan.query_graph for search_query in q.search_query]
            if print_level > 1:
                print(search_keys)
            search_dict.update( {key: summarize_raw_search(eval(search_cache[key])) for key in search_keys if search_cache.get(key,None) is not None} )

        # added this because without it sometimes errors occur during report generation and fails without saving any results!
        assessment_results = {'short_name': short_name, 'plan': plan,'plan0': plan0, 'plan_result': plan_result, 'search_dict': search_dict,'kb_graph': graph, 'kb_dot': kb_dot, 'report': report_rmarkdown, 'model': model }
        if save_pickle:
            _ = pickle_it(assessment_results = assessment_results, save = True)
            if print_level > 1:
                print(f"... saving results (without report) to 'results_{short_name}.pkl' ...")
            
        if report_rmarkdown is None:
            if print_level > 0:
                print("\n*** GENERATING REPORT ***")
            # Generate the report as an R Markdown (.Rmd) document
            dependencies = {q.id: q.dependencies for q in plan.query_graph}
            try:
                # Put the computed queries of the plan_result in the same order that their questions appear in the plan's query outline text.
                node_ids =  outline_processing(plan)
                computed_queries = {id: plan_result[id] for id in node_ids}

                report_rmarkdown = generate_report(
                    plan_outline = plan.query_plan_outline,
                    lastnode=last_node_result,
                    computed_queries=computed_queries,                    
                    dependencies = dependencies,
                    search_dict=search_dict, 
                    filename_stem=short_name, 
                    write_to_file=report_to_file,
                    added_instructions = added_instructions if len(added_instructions)>0 else None
                )
            except:
                warnings.warn("Could not generate report. Results will have report_markdown = None.", UserWarning)


    # Record the end time
    end_time = time.time()
    
    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    
    assessment_results = {'short_name': short_name, 'plan': plan,'plan0': plan0, 'plan_result': plan_result, 'search_dict': search_dict,'kb_graph': graph, 'kb_dot': kb_dot, 'report': report_rmarkdown, 'model': model, 
                          'start_time': start_time, 'elapsed_time': elapsed_time }

    # Ahh, relief. Got a valid report. Save all results with report.
    if save_pickle:
        _ = pickle_it(assessment_results = assessment_results, save = True)
        if print_level > 1:
            print(f"*** RESULTS SAVED TO 'results_{short_name}.pkl' ***")
        
    # Return all the results
    if print_level > 0:
        print("\n\n================================= *** DONE *** =================================\n")

    return assessment_results

In [ ]:
def print_summary(assmnt):
    summary_response = list(assmnt['plan_result'].values())[-1].response
    print(f"""**PROPOSITION {assmnt['short_name']} **\n**"{assmnt['plan'].query_graph[0].question}"**\n**Elapsed time: {assmnt['elapsed_time']} sec**""")
    print(f"\n**SEARCH FOR CONTEXT:**\n{assmnt['plan'].search_query_for_context}")
    print(summarize_raw_search(eval(assmnt['plan'].context)))
    print(f'\n**RESPONSE:**\n{summary_response}')
    return(summary_response)

In [ ]:
def get_report(result_dictionary,search_cache=search_cache):    
    report_rmarkdown = generate_report(
        plan_outline = result_dictionary['plan'].query_plan_outline,
        lastnode=list(result_dictionary['plan_result'].values())[-1],
        computed_queries={id: result_dictionary['plan_result'][id] for id in outline_processing(result_dictionary['plan'])},                    
        dependencies = {q.id: q.dependencies for q in result_dictionary['plan'].query_graph},
        search_dict= {key: summarize_raw_search(eval(search_cache[key])) for key in  [search_query for q in result_dictionary['plan'].query_graph for search_query in q.search_query] if search_cache.get(key,None) is not None}, 
        filename_stem=result_dictionary['short_name'], 
        write_to_file=True,
        added_instructions = None
    )
    return report_rmarkdown

#rpt = get_report(assessment_brady_qb_new)

#print(rpt)

# Proposition Analyses

## Initialization

In [ ]:
search_cache_exists = ('search_cache' in globals()) | ('search_cache' in locals())
INITIALIZE = False | (not search_cache_exists) # force it by replacing False with True 
if INITIALIZE: # Set INITIALIZE to True if want to clear search_cache
    search_cache = {}

# Begin again with a new `client` object
client = instructor.patch(OpenAI())

## Arbitrary Queries

#### “Not once in the twentieth century, ... has a single politician, actor, athlete, or surgeon emerged as a first-rate novelist, despite the dismayingly huge breadth of experience each profession affords.”

In [ ]:
assessment_novelists = await assess_proposition2(
    proposition = "Not once in the twentieth century,... has a single politician, actor, athlete, or surgeon emerged as a first-rate novelist, despite the dismayingly huge breadth of experience each profession affords.", 
    short_name = 'novelists',
    added_instructions="",
    search_cache = search_cache,
    improvement_round=True,
    client = client, 
    model="gpt-4-turbo-preview", 
    report_to_file = True, 
    kb_label_wrap = 24, 
    force_run = True, 
    save_pickle = True,
    print_level = 2
)
display(visualize_knowledge_graph(convert_to_graph(assessment_novelists['plan0'],proposition='copilotkt_gamechanger')))
display(assessment_novelists['kb_dot'])
if assessment_novelists.get('plan_result'):
    _ = print_summary(assessment_novelists)

#### CopilotKit is a game-changer

In [ ]:
assessment_copilotkit_gamechanger = await assess_proposition2(
    proposition = "CopilotKit is a unique game-changer for AI application development.", 
    #plan_only = True,
    short_name = 'copilotkit_gamechanger',
    added_instructions="""
Accept ::GIVEN_FACTS:: as factual information that does not need to be verified by search but rather supplies clarity and resources to help in posing search queries.

::GIVEN_FACTS::

* CopilotKit is 'The Open-Source Copilot Platform'.
* CopilotKit is available for download at GitHub repository https://github.com/CopilotKit/CopilotKit .
* Detailed documentation of CopilotKit features and usage is available at https://docs.copilotkit.ai/getting-started/quickstart-textarea  .

""",
    search_cache = search_cache,
    improvement_round=True,
    client = client, 
    model="gpt-4-turbo-preview", 
    report_to_file = True, 
    kb_label_wrap = 24, 
    force_run = True, 
    save_pickle = True,
    print_level = 2
)
display(visualize_knowledge_graph(convert_to_graph(assessment_copilotkit_gamechanger['plan0'],proposition='copilotkt_gamechanger')))
display(assessment_copilotkit_gamechanger['kb_dot'])
if assessment_copilotkit_gamechanger.get('plan_result'):
    _ = print_summary(assessment_copilotkit_gamechanger)

#### Altman was prophetic ("Moore's Law for Everything")

In [ ]:
assessment_altman_prophetic = await assess_proposition2(
    proposition = "Sam Altman's essay [*Moore's Law for Everything* (March 16, 2021)](https://moores.samaltman.com/) was prophetic.", 
    #plan_only = True,
    short_name = 'altman_prophetic',
    added_instructions="",
    search_cache = search_cache,
    improvement_round=True,
    client = client, 
    model="gpt-4-turbo-preview", 
    report_to_file = True, 
    kb_label_wrap = 24, 
    force_run = False, 
    save_pickle = True,
    print_level = 2
)
#display(visualize_knowledge_graph(convert_to_graph(assessment_altman_prophetic['plan0'],proposition='Prophetic')))
display(assessment_altman_prophetic['kb_dot'])
if assessment_altman_prophetic.get('plan_result'):
    _ = print_summary(assessment_altman_prophetic)

In [ ]:
display(visualize_knowledge_graph(convert_to_graph(assessment_altman_prophetic['plan0'],proposition='Prophetic')))

#### RAG vs Fine-tuning

In [ ]:
assessment_rag_vs_finetuning = await assess_proposition2(
    proposition = "Which gives a Generative AI-based application the greatest capability and differentiation over competitors: RAG or Fine-Tuning.", 
    short_name = 'rag_vs_finetuning',
    added_instructions="""
* Identify best use cases for both approaches.
* Give a reasoned argument in favor of the better one for a maximally profitable AI-based commercial product intended for professional product formulators in
consumer packaged goods (CPG) R&D.
""",
    search_cache = search_cache,
    improvement_round=True,
    client = client, 
    model="gpt-4-turbo-preview", 
    report_to_file = True, 
    kb_label_wrap = 24, 
    force_run = True, 
    save_pickle = True,
    print_level = 2
)
display(assessment_rag_vs_finetuning['kb_dot'])
if assessment_rag_vs_finetuning.get('plan_result'):
    _ = print_summary(assessment_rag_vs_finetuning)

#### Cox Arboretum

In [ ]:
assessment_cox_arboretum = await assess_proposition2(
    proposition = "Cox Arboretum Metropark is the best place to walk for pleasurable exercise in the entire Cincinnati-Dayton metroplex.", 
    short_name = 'cox_arboretum',
    search_cache = search_cache,
    improvement_round=True,
    client = client, 
    model="gpt-4-turbo-preview", 
    report_to_file = True, 
    kb_label_wrap = 24, 
    force_run = False, 
    save_pickle = True,
    print_level = 2
)
display(assessment_cox_arboretum['kb_dot'])
if assessment_cox_arboretum.get('plan_result'):
    _ = print_summary(assessment_cox_arboretum)

#### U.S.: a Chrisitian nation?

In [ ]:
assessment_us_christian = await assess_proposition2(
    proposition = "America was conceived and founded with the intent of being a Christian nation, according to the writings of the Nation's forefathers and the Constitution.", 
    short_name = 'us_christian',
    search_cache = search_cache,
    improvement_round=True,
    client = client, 
    model="gpt-4-turbo-preview", 
    report_to_file = True, 
    kb_label_wrap = 24, 
    force_run = False, 
    save_pickle = True,
    print_level = 2
)
display(assessment_us_christian['kb_dot'])
if assessment_us_christian.get('plan_result'):
    _ = print_summary(assessment_us_christian)

In [ ]:
assessment_us_christian2 = await assess_proposition2(
    proposition = "America was conceived and founded with the intent of being a Christian nation, according to the writings of the Nation's forefathers and the Constitution.", 
    short_name = 'us_christian2',
    added_instructions="""
Follow these principles in generating the query plan:

* DO NOT make up sources, references, or any information.
* Seek out and prioritize the most authoritative and reliable sources when collecting all information as evidence.
* Emphasize a contrast of propositions and their alternative propositions (i.e., negations), thoroughly accumulating factual evidence for and against each proposition.
* Systematically, lay out the arguments for and against each proposition.
* DO NOT blindly accept any proposition as true unless reliable factual evidence in support of it outweighs similarly reliable factual evidence in refutation of it.
""",
    search_cache = search_cache,
    improvement_round=True,
    client = client, 
    model="gpt-4-turbo-preview", 
    report_to_file = True, 
    kb_label_wrap = 24, 
    force_run = False, 
    save_pickle = True,
    print_level = 2
)
display(assessment_us_christian2['kb_dot'])
if assessment_us_christian2.get('plan_result'):
    _ = print_summary(assessment_us_christian2)

In [ ]:
assessment_results = pickle_it(assessment_results = {'short_name': 'graph_rep_dishwashing2'}, save = False)

In [ ]:
visualize_knowledge_graph(kg= convert_to_graph(plan= QueryPlan(**assessment_results['plan0']),proposition='Dishwashing Liquid'))

In [ ]:
 
assessment_graph_rep_dishwashing2a = await assess_proposition2(
    proposition = "Conceive of a graph network representation of commercial dishwashing liquid formulation science.", 
    added_instructions ="""
It is critical that in the execution of the plan, you absolutely must explicitly reference the concepts presented below, especially in the search queries.

Consider these concepts:

* Each formulation space is represented by 5 types of entities:
  1. ::INGREDIENTS::,
  2. functional ::ROLES::, 
  3. causal ::MECHANISMS::, 
  4. delivered ::BENEFITS::, 
  5. observed ::MANIFESTATIONS::.
* Each ::INGREDIENT:: has 1 or more functional ::ROLES:: in the formulation.
* Each functional ::ROLE:: in the formulation participates in 1 or more causal ::MECHANSIMS:: resulting in the delivery of an intended ::BENEFIT:: of the finished product.
* Each delivered ::BENEFIT:: is the result of 1 or more causal ::MECHANISMS::.
* Each ::BENEFIT:: is realized in terms of observed ::MANIFESTATIONS:: of 3 types:
    1. laboratory measurements, ::LAB_MEASUREMENT:: (composition, physical properties, performance efficacy relevant to delivery of benefits)
    2. sensory panel assessments, ::SENSORY_METRIC:: (organoleptic qualities relevant to benefits)
    3. consumer survey responses. ::SURVEY:: (perception of benefits, satisfaction, overall liking)
* Each functional ::ROLE:: is a parent node of the ::INGREDIENT:: that have that ::ROLE::.
* Each functional ::ROLE:: is also a parent node of every causal ::MECHANISM:: that realizes that functional ::ROLE::.
* Each causal ::MECHANISM:: is a parent node of the delivered ::BENEFITS:: that are the direct or indirect result of that ::MECHANISM::.
* Each delivered ::BENEFIT:: is a parent node of the directly observed ::MANIFESTATIONS:: (::LAB_MEASUREMENT::, ::SENSORY_METRIC::, and/or ::SURVEY_RESPONSE::) of that ::BENEFIT::.

Finally, as an example of realizing this proposition:

* Generate a detailed, fine-granularity, comprehensive graph for the formulation space.
* Make sure that all of the specific entities you can think of are included. 
* Present the graph using the Graphviz-compatible DOT syntax.
""",
    short_name = 'graph_rep_dishwashing2a', 
    use_plan = QueryPlan(**assessment_results['plan0']),
    search_cache = search_cache,
    improvement_round=True,
    client = client, 
    model="gpt-4-turbo-preview", 
    report_to_file = True, 
    kb_label_wrap = 24, 
    force_run = False, 
    save_pickle = True,
    #status_updates = True,
    print_level = 2
)
display(assessment_graph_rep_dishwashing2a['kb_dot'])
if assessment_graph_rep_dishwashing2a.get('plan_result'):
    _ = print_summary(assessment_graph_rep_dishwashing2a)

In [ ]:
assessment_graph_rep_dishwashing2 = await assess_proposition2(
    proposition = "Conceive of a graph network representation of commercial dishwashing liquid formulation science.", 
    added_instructions ="""
It is critical that in the execution of the plan, you absolutely must explicitly reference the concepts presented below, especially in the search queries.

Consider these concepts:

* Each formulation space is represented by 5 types of entities:
  1. ::INGREDIENTS::,
  2. functional ::ROLES::, 
  3. causal ::MECHANISMS::, 
  4. delivered ::BENEFITS::, 
  5. observed ::MANIFESTATIONS::.
* Each ::INGREDIENT:: has 1 or more functional ::ROLES:: in the formulation.
* Each functional ::ROLE:: in the formulation participates in 1 or more causal ::MECHANSIMS:: resulting in the delivery of an intended ::BENEFIT:: of the finished product.
* Each delivered ::BENEFIT:: is the result of 1 or more causal ::MECHANISMS::.
* Each ::BENEFIT:: is realized in terms of observed ::MANIFESTATIONS:: of 3 types:
    1. laboratory measurements, ::LAB_MEASUREMENT:: (composition, physical properties, performance efficacy relevant to delivery of benefits)
    2. sensory panel assessments, ::SENSORY_METRIC:: (organoleptic qualities relevant to benefits)
    3. consumer survey responses. ::SURVEY:: (perception of benefits, satisfaction, overall liking)
* Each functional ::ROLE:: is a parent node of the ::INGREDIENT:: that have that ::ROLE::.
* Each functional ::ROLE:: is also a parent node of every causal ::MECHANISM:: that realizes that functional ::ROLE::.
* Each causal ::MECHANISM:: is a parent node of the delivered ::BENEFITS:: that are the direct or indirect result of that ::MECHANISM::.
* Each delivered ::BENEFIT:: is a parent node of the directly observed ::MANIFESTATIONS:: (::LAB_MEASUREMENT::, ::SENSORY_METRIC::, and/or ::SURVEY_RESPONSE::) of that ::BENEFIT::.

Finally, as an example of realizing this proposition:

* Generate a detailed, fine-granularity, comprehensive graph for the formulation space.
* Make sure that all of the specific entities you can think of are included. 
* Present the graph using the Graphviz-compatible DOT syntax.
""",
    short_name = 'graph_rep_dishwashing2', 
    plan_only = False,
    search_cache = search_cache,
    improvement_round=True,
    client = client, 
    model="gpt-4-turbo-preview", 
    report_to_file = True, 
    kb_label_wrap = 24, 
    force_run = True, 
    save_pickle = True,
    #status_updates = True,
    print_level = 2
)
display(assessment_graph_rep_dishwashing2['kb_dot'])
if assessment_graph_rep_dishwashing2.get('plan_result'):
    _ = print_summary(assessment_graph_rep_dishwashing2)

In [ ]:
assessment_graph_rep_dishwashing2['plan']

In [ ]:
assessment_graph_rep_formulations = await assess_proposition2(
    proposition = "Conceive of a graph network representation of chocolate chip recipes.", 
    added_instructions ="""
Consider these concepts:

* Each formulation space is represented by 5 types of entities:
  1. ::INGREDIENTS::,
  2. functional ::ROLES::, 
  3. causal ::MECHANISMS::, 
  4. delivered ::BENEFITS::, 
  5. observed ::MANIFESTATIONS::.
* Each ::INGREDIENT:: has 1 or more functional ::ROLES:: in the formulation.
* Each functional ::ROLE:: in the formulation participates in 1 or more causal ::MECHANSIMS:: resulting in the delivery of an intended ::BENEFIT:: of the finished product.
* Each delivered ::BENEFIT:: is the result of 1 or more causal ::MECHANISMS::, and is realized in terms of observed ::MANIFESTATIONS::.
* ::MANIFESTATIONS:: are observations of 3 types: 
  1. laboratory measurements, ::LAB_MEASURE::
  2. sensory panel assessments, ::SENSORY::
  3. consumer survey responses. ::SURVEY::
* Each functional ::ROLE:: is a parent node of the ::INGREDIENT:: that have that ::ROLE::.
* Each functional ::ROLE:: is also a parent node of every causal ::MECHANISM:: that realizes that functional ::ROLE::.
* Each causal ::MECHANISM:: is a parent node of the delivered ::BENEFITS:: that are the direct or indirect result of that ::MECHANISM::.
* Each delivered ::BENEFIT:: is a parent node of the directly observed ::MANIFESTATIONS:: of that ::BENEFIT::.

""",
    short_name = 'graph_rep_formulations', 
    plan_only = False,
    search_cache = search_cache,
    improvement_round=True,
    client = client, 
    model="gpt-4-turbo-preview", 
    report_to_file = True, 
    kb_label_wrap = 24, 
    force_run = False, 
    save_pickle = True,
    #status_updates = True,
    print_level = 2
)
display(assessment_graph_rep_formulations['kb_dot'])
if assessment_graph_rep_formulations.get('plan_result'):
    _ = print_summary(assessment_graph_rep_formulations)

#### Perception of time as you age

In [ ]:
assessment_perception_of_time = await assess_proposition2(
    proposition = "Time seems to pass faster as you age.", 
    added_instructions ="""
Consider these musings:

* Each year you live is a smaller and smaller proportion of your life as you age: '1 yr as % of age' = 1/Age.
* Therefore, the proportion of your life you've lived before that latest year becomes larger: '% of Life lived before latest year' = 1 - 1/Age
* As you go through early adulthood, you become accustomed to life experiences/events at a certain pace, which is proportional to 1/Age.
* For many people, an effective 'Reference Age' at which you benchmark this pace is 'Reference Age' = 30 years old.
* Then, the pace of life at any age, is basically 'Pace of Life' = (1/'Reference Age')/(1/Age).
* And, you eventually become acclimated to this perception as 'Perceived Speed of Life' = 1/(1 + exp(-6*('Pace of Life'-1))) = 1/(1 + exp(-6*(Age - 'Reference Age')/'Reference Age')) .
* In other words, as you age, years ago seem to be as if they were just yesterday because of this 'Perceived Speed of Life'.

""",
    short_name = 'perception_of_time', 
    search_cache = search_cache,
    improvement_round=True,
    client = client, 
    model="gpt-4-turbo-preview", 
    report_to_file = True, 
    kb_label_wrap = 24, 
    force_run = False, 
    save_pickle = True,
    #status_updates = True,
    print_level = 2
)
display(assessment_perception_of_time['kb_dot'])
if assessment_perception_of_time.get('plan_result'):
    _ = print_summary(assessment_perception_of_time)

#### Gary, Indiana

In [ ]:
assessment_gary_indiana_test = await assess_proposition2(
    proposition = "Gary, Indiana is a failed city.", 
    short_name = 'gary_indiana_test', 
    search_cache = search_cache,
    improvement_round=True,
    client = client, 
    model="gpt-4-turbo-preview", 
    report_to_file = True, 
    kb_label_wrap = 24, 
    force_run = True, 
    save_pickle = True,
    status_updates = True,
    print_level = 2
)
display(assessment_gary_indiana_test['kb_dot'])
if assessment_gary_indiana_test.get('plan_result'):
    _ = print_summary(assessment_gary_indiana_test)

#### Brady query

In [ ]:
question = "Tom Brady is the NFL's greatest quarterback of all time."
assessment_brady_qb_reflect = await assess_proposition2(
    #plan_only = True,
    proposition = question, 
    short_name = 'brady_qb_reflect', 
    search_cache = search_cache,
    improvement_round=True,
    client = client, 
    model="gpt-4-turbo-preview", 
    report_to_file = True, 
    kb_label_wrap = 24, 
    force_run = True,  # Always set True to force the generation of a new plan for this test run.
    save_pickle = True,
    status_updates = True,
    print_level = 2
)
display(assessment_brady_qb_reflect['kb_dot'])
if assessment_brady_qb_reflect.get('plan_result'):
    _ = print_summary(assessment_brady_qb_reflect)

In [ ]:
question = "Tom Brady is the NFL's greatest quarterback of all time."
assessment_brady_qb_test_improve = await assess_proposition2(
    #plan_only = True,
    proposition = question, 
    short_name = 'brady_qb_test_improve', 
    search_cache = search_cache,
    improvement_round=True,
    client = client, 
    model="gpt-4-turbo-preview", 
    report_to_file = True, 
    kb_label_wrap = 24, 
    force_run = True,  # Always set True to force the generation of a new plan for this test run.
    save_pickle = True,
    print_level = 2
)
display(assessment_brady_qb_test_improve['kb_dot'])
if assessment_brady_qb_test_improve.get('plan_result'):
    _ = print_summary(assessment_brady_qb_test_improve)

#### Mahomes, NFL GOAT

In [ ]:
assessment_mahomes_goat = await assess_proposition2(
    #plan_only = True,
    proposition = "Patrick Mahomes is the NFL's greatest quarterback of all time.", 
    short_name = 'mahomes_goat', 
    search_cache = search_cache,
    improvement_round=True,
    client = client, 
    model="gpt-4-turbo-preview", 
    report_to_file = True, 
    kb_label_wrap = 24, 
    force_run = False, 
    save_pickle = True,
    print_level = 2
)
display(assessment_mahomes_goat['kb_dot'])
if assessment_mahomes_goat.get('plan_result'):
    _ = print_summary(assessment_mahomes_goat)

#### Economies of Africa

In [ ]:
question = "What is the best country, economically, in Africa?"
assessment_econ_africa = await assess_proposition2(
    proposition = question, 
    short_name = 'econ_africa', 
    search_cache = search_cache,
    client = client, 
    model="gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20, 
    force_run = False, 
    save_pickle = True,
    print_level = 2
)
display(assessment_econ_africa['kb_dot'])
if assessment_econ_africa.get('plan_result'):
    _ = print_summary(assessment_econ_africa)

In [ ]:
question = "What is the best country, economically, in Africa?"
assessment_econ_africa_new = await assess_proposition2(
    #plan_only = True,
    proposition = question, 
    short_name = 'econ_africa_new', 
    added_instructions = """
* Focus on only the largest 5 African economies. 
* Perform searches to provide the identity of these countries as context for defining subsequent `Query` nodes by putting the node with the country names in the `dependencies` fields of other nodes.
* If necessary, consider using dynamic search queries for [Country 1], [Country 2], etc., 
  being sure to supply the `Query` nodes with ample context to identify these countries at time of search.
""",
    search_cache = search_cache,
    client = client, 
    model="gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20, 
    force_run = True, 
    save_pickle = True,
    print_level = 2
)
display(assessment_econ_africa_new['kb_dot'])
if assessment_econ_africa_new.get('plan_result'):
    _ = print_summary(assessment_econ_africa_new)

In [ ]:
question = "What is the best country, economically, in Africa?"
assessment_econ_africa_new_improve = await assess_proposition2(
    #plan_only = True,
    proposition = question, 
    short_name = 'econ_africa_new_improve', 
    added_instructions = """
* Focus on only the largest 5 African economies. 
* Perform searches to provide the identity of these countries as context for defining subsequent `Query` nodes by putting the node with the country names in the `dependencies` fields of other nodes.
* If necessary, consider using dynamic search queries for [Country 1], [Country 2], etc., 
  being sure to supply the `Query` nodes with ample context to identify these countries at time of search.
""",
    search_cache = search_cache,
    client = client, 
    improvement_round=True,
    model="gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20, 
    force_run = True, 
    save_pickle = True,
    status_updates = True,
    print_level = 2
)
display(assessment_econ_africa_new_improve['kb_dot'])
if assessment_econ_africa_new_improve.get('plan_result'):
    _ = print_summary(assessment_econ_africa_new_improve)

In [ ]:
[search_query for q in assessment_econ_africa_new['plan'].query_graph for search_query in q.search_query] 

In [ ]:
rpt = get_report(assessment_econ_africa_new)

print(rpt)

#### Difference in Population

In [ ]:
question = "What is the difference in populations between the country where Toronto is and Barack Obama's father's home country?"
assessment_pop_diff_reflect = await assess_proposition2(
    proposition = question, 
    short_name = 'pop_diff_reflect', 
    search_cache = search_cache,
    client = client, 
    improvement_round=True,
    added_instructions = '', 
    model="gpt-3.5-turbo", 
    report_to_file = False,
    kb_label_wrap = 20, 
    force_run = False, 
    save_pickle = True,
    print_level = 2
)
display(assessment_pop_diff_reflect['kb_dot'])
if assessment_pop_diff_reflect.get('plan_result'):
    _ = print_summary(assessment_pop_diff_reflect)

In [ ]:
question = "What is the difference in populations between the country where Toronto is and Barack Obama's father's home country?"
assessment_pop_diff = await assess_proposition2(
    proposition = question, 
    short_name = 'pop_diff', 
    search_cache = search_cache,
    client = client, 
    use_plan = None,
    plan_only = False, 
    added_instructions = '', 
    model="gpt-3.5-turbo", 
    report_to_file = False,
    kb_label_wrap = 20, 
    force_run = False, 
    save_pickle = True,
    print_level = 2
)
assessment_pop_diff['kb_dot']

In [ ]:
question = "What is the difference in populations between the country where Toronto is and Barack Obama's father's home country?"
assessment_pop_diff_improve = await assess_proposition2(
    proposition = question, 
    short_name = 'pop_diff_improve', 
    search_cache = search_cache,
    improvement_round=True,
    client = client, 
    use_plan = None,
    plan_only = False, 
    added_instructions = '', 
    model="gpt-3.5-turbo", 
    report_to_file = False,
    kb_label_wrap = 20, 
    force_run = False, 
    save_pickle = True,
    print_level = 2
)
assessment_pop_diff_improve['kb_dot']

In [ ]:
# using the same proposition and plan, execute the assessment using GPT-4 instead of GPT-3.5-Turbo
assessment_pop_diff_gpt4 = await assess_proposition2(
    proposition = question, 
    short_name = 'pop_diff_gpt4', 
    search_cache = search_cache,
    client = client, 
    use_plan = assessment_pop_diff['plan'],
    plan_only = False, 
    added_instructions = '', 
    model="gpt-4-turbo-preview", 
    report_to_file = False,
    kb_label_wrap = 20, 
    force_run = False, 
    save_pickle = True,
    print_level = 0
)
_=print_summary(assessment_pop_diff_gpt4)

In [ ]:
assessment_pop_diff_gpt4.keys()

#### Long, complex proposition

In [ ]:
long_proposition = "The eastern neighbor of the country where the father of the 44th U.S. president was born is similar in size to the total population of the U.S. state that has the largest population at risk of earthquakes."
assessment_long_proposition_reflect =  await assess_proposition2(
    proposition = long_proposition, 
    short_name = 'long_proposition_reflect', 
    improvement_round=True,
    search_cache = search_cache, 
    added_instructions = "", 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20,
    force_run = True,
    save_pickle = True,
    status_updates = True,
    print_level = 2
)
display(assessment_long_proposition_reflect['kb_dot'])
if assessment_long_proposition_reflect.get('plan_result'):
    _=print_summary(assessment_long_proposition_reflect)

In [ ]:
long_proposition = "The eastern neighbor of the country where the father of the 44th U.S. president was born is similar in size to the total population of the U.S. state that has the largest population at risk of earthquakes."
assessment_long_proposition =  await assess_proposition2(
    proposition = long_proposition, 
    short_name = 'long_proposition', 
    improvement_round=True,
    search_cache = search_cache, 
    added_instructions = "", 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20,
    force_run = True,
    save_pickle = True,
    print_level = 0
)
display(assessment_long_proposition['kb_dot'])
_=print_summary(assessment_long_proposition)

In [ ]:
long_proposition = "The eastern neighbor of the country where the father of the 44th U.S. president was born is similar in size to the total population of the U.S. state that has the largest population at risk of earthquakes."
assessment_long_proposition0 =  await assess_proposition2(
    proposition = long_proposition, 
    short_name = 'long_proposition0', 
    search_cache = search_cache, 
    added_instructions = """
    Use this key:
    * <<A>> = the 44th U.S. president
    * <<B>> = the father of <<A>>
    * <<C>> = the country where <<B>> was born
    * <<D>> = the eastern neighbor of <<C>>
    * <<E>> = the total population of <<D>>
    * <<F>> = the U.S. state that has the largest population at risk of earthquakes
    * <<G>> = the total population of <<F>>
    """, 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 35,
    force_run = False,
    save_pickle = True,
    print_level=0
)
display(assessment_long_proposition0['kb_dot'])
_=print_summary(assessment_long_proposition0)

In [ ]:
long_proposition0 = "The eastern neighbor of the country where the father of the 44th U.S. president was born is similar in size to the total population of the U.S. state that has the largest population at risk of earthquakes."
assessment_long_proposition0_dynamic =  await assess_proposition2(
    proposition = long_proposition0, 
    short_name = 'long_proposition0_dynamic', 
    use_plan = assessment_long_proposition0['plan'],
    search_cache = search_cache, 
    added_instructions = """
    Use this key:
    * <<A>> = the 44th U.S. president
    * <<B>> = the father of <<A>>
    * <<C>> = the country where <<B>> was born
    * <<D>> = the eastern neighbor of <<C>>
    * <<E>> = the total population of <<D>>
    * <<F>> = the U.S. state that has the largest population at risk of earthquakes
    * <<G>> = the total population of <<F>>
    """, 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 35,
    force_run = True,
    save_pickle = True,
    print_level = 1
)
#display(assessment_long_proposition0_dynamic['kb_dot'])
_=print_summary(assessment_long_proposition0_dynamic)

In [ ]:
long_proposition0 = "The eastern neighbor of the country where the father of the 44th U.S. president was born is similar in size to the total population of the U.S. state that has the largest population at risk of earthquakes."
assessment_long_proposition0_dynamic_nokey =  await assess_proposition2(
    proposition = long_proposition0, 
    short_name = 'long_proposition0_dynamic_nokey', 
    use_plan = None,
    search_cache = search_cache, 
    added_instructions = "", 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20,
    force_run = False,
    save_pickle = True
)
display(assessment_long_proposition0_dynamic_nokey['kb_dot'])
_=print_summary(assessment_long_proposition0_dynamic_nokey)

In [ ]:
long_proposition = "<<D>> is similar in size to <<G>>."
assessment_long_proposition =  await assess_proposition2(
    proposition = long_proposition, 
    short_name = 'long_proposition', 
    search_cache = search_cache, 
    added_instructions = """
    Use this key:
    * <<A>> = the 44th U.S. president
    * <<B>> = the father of <<A>>
    * <<C>> = the country where <<B>> was born
    * <<D>> = the eastern neighbor of <<C>>
    * <<E>> = the total population of <<D>>
    * <<F>> = the U.S. state that has the largest population at risk of earthquakes
    * <<G>> = the total population of <<F>>
    """, 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20,
    force_run = True,
    save_pickle = True
)

In [ ]:
assessment_long_proposition_dynamic =  await assess_proposition2(
    proposition = long_proposition, 
    short_name = 'long_proposition_dynamic', 
    use_plan = assessment_long_proposition['plan'],
    search_cache = search_cache, 
    added_instructions = """
    Use this key:
    * <<A>> = the 44th U.S. president
    * <<B>> = the father of <<A>>
    * <<C>> = the country where <<B>> was born
    * <<D>> = the eastern neighbor of <<C>>
    * <<E>> = the total population of <<D>>
    * <<F>> = the U.S. state that has the largest population at risk of earthquakes
    * <<G>> = the total population of <<F>>
    """, 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20,
    force_run = True,
    save_pickle = True
)

In [ ]:
plan = assessment_long_proposition_dynamic['plan']
plan_result = assessment_long_proposition_dynamic['plan_result']
last_node_result = list(plan_result.values())[-1]
search_dict = assessment_long_proposition_dynamic['search_dict']
short_name = assessment_long_proposition_dynamic['short_name']
report_rmarkdown = generate_report(
    plan_outline = plan.query_plan_outline,
    lastnode=last_node_result,
    computed_queries=plan_result,
    search_dict=search_dict, 
    filename_stem=short_name, 
    write_to_file=True
)

#### Arbitrary

In [ ]:
long_prop2 = "What is the tallest building in the 3rd most populous country in Asia?"
assessment_long_prop2 =  await assess_proposition2(
    proposition = long_prop2, 
    short_name = 'long_prop2', 
    use_plan = None,
    search_cache = search_cache, 
    added_instructions = "", 
    dynamic = True,
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20,
    force_run = False,
    save_pickle = True,
    print_level = 0
)
display(assessment_long_prop2['kb_dot'])
_=print_summary(assessment_long_prop2)

In [ ]:
long_prop2a = "The Autograph Tower of Indonesia is taller than the Pertamina Energy Tower."
assessment_long_prop2a =  await assess_proposition2(
    proposition = long_prop2a, 
    short_name = 'long_prop2a', 
    use_plan = None,
    search_cache = search_cache, 
    added_instructions = "", 
    dynamic = True,
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20,
    force_run = True,
    save_pickle = True
)
display(assessment_long_prop2a['kb_dot'])
_=print_summary(assessment_long_prop2a)

In [ ]:
long_proposition_result_dynamic = await assessment_long_proposition['plan'].execute_dynamic(client,model='gpt-4-turbo-preview')

In [ ]:
last_long_proposition_result_dynamic = list(long_proposition_result_dynamic.values())[-1]
print('QUERY: ' + re.sub(r'\\"response(s)*\\"',r'\n"response\1"',last_long_proposition_result_dynamic.query.replace("\\n","\n")) + '\n\n')
print('RESPONSE: ' + last_long_proposition_result_dynamic.response)

#### Baboons vs. Hippos

In [ ]:
assessment_baboon_hippo2 =  await assess_proposition2(
    proposition = "A baboon would beat a hippo in a 40-meter race.", 
    short_name = 'baboon_hippo2', 
    search_cache = search_cache, 
    improvement_round=True,
    added_instructions = '', 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True, # Don't bother saving a report to file
    kb_label_wrap = 20,
    force_run = False,
    save_pickle = True,
    print_level = 2
)
display(assessment_baboon_hippo2['kb_dot'])
_=print_summary(assessment_baboon_hippo2)

#### Grease Fire

In [ ]:
assessment_grease_fire =  await assess_proposition2(
    proposition =  "Covering it with a pan lid is the best way to handle a grease fire in the kitchen.", 
    short_name = 'grease_fire', 
    improvement_round=True,
    search_cache = search_cache, 
    added_instructions = "", 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20,
    print_level = 2
)
display(assessment_grease_fire['kb_dot'])
if assessment_grease_fire.get('plan_result'):
    print_summary(assessment_grease_fire)

#### Smoke detectors

In [ ]:
assessment_smoke_detectors_reflect =  await assess_proposition2(
    proposition =  "The best smoke detectors and locations in the home.", 
    short_name = 'smoke_detectors_reflect', 
    improvement_round=True,
    search_cache = search_cache, 
    added_instructions = "", 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 25,
    status_updates=True,
    print_level = 2
)
display(assessment_smoke_detectors_reflect['kb_dot'])
if assessment_smoke_detectors_reflect.get('plan_result'):
    print_summary(assessment_smoke_detectors_reflect)

In [ ]:
assessment_smoke_detectors =  await assess_proposition2(
    proposition =  "The best smoke detectors and locations in the home.", 
    short_name = 'smoke_detectors', 
    improvement_round=True,
    search_cache = search_cache, 
    added_instructions = "", 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 25,
    print_level = 2
)
display(assessment_smoke_detectors['kb_dot'])
if assessment_smoke_detectors.get('plan_result'):
    print_summary(assessment_smoke_detectors)

In [ ]:
assessment_smoke_detectors2 =  await assess_proposition2(
    proposition =  "The best smoke detectors, locations in the home, and highly recommended models (with URLs).", 
    short_name = 'smoke_detectors2', 
    improvement_round=True,
    search_cache = search_cache, 
    added_instructions = "", 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 25,
    force_run = True,
    print_level = 2
)
display(assessment_smoke_detectors2['kb_dot'])
if assessment_smoke_detectors2.get('plan_result'):
    print_summary(assessment_smoke_detectors2)

## Sports

#### Tom Brady, NFL GOAT

In [ ]:
client = instructor.patch(OpenAI())
proposition = "Tom Brady is the GOAT of the NFL"
#key_question = f"""Given proposition '{proposition}'. First identify all sub-propositions and their sub-propositions, and so on, that need to be true to support it.
#Then identify all factual axiomatic statements as evidence that support or refute each and every one of the propositions. 
#Finally, answer this question: Is '{proposition}' the truth?"""

key_question = f"""# Mandate:
Given proposition '{proposition}'. 
1. Enumerate all sub-propositions, and their sub-propositions and so on, that need to be true to support it.
2. Enumerate all factual axiomatic statements as evidence that support or refute each and every one of the propositions. 
3. Record these propositions and evidence as a textual outline (in 'query_plan_outline') describing the query plan.
3. Generate the query plan as a tree of query nodes (in 'query_graph').
   - Assign a separate query node for each specific sub-proposition.
   - Assign a separate query node for each specific piece of evidence needed to confirm and refute each sub-proposition. 
   - Be sure to specify search_flag=True for a query node when needed to gather the latest factual information as evidence.
   - Demand that actual specific data and statistics are gathered as evidence and that the answers to the propositions are stated in terms of these data.
"""

mandate = f"""
# The Question for which we require a Query Plan:
Is '{proposition}' the truth?"""

plan_brady = query_planner( key_question + mandate, client )
pprint(plan_brady.model_dump())

In [ ]:
graph: KnowledgeGraph = convert_to_graph(plan_brady,proposition,label_wrap=20)
kb_dot = visualize_knowledge_graph(graph,flip=True, aspect_ratio= 'auto')
kb_dot

In [ ]:
plan_brady2_result = await plan_brady.execute(client,model='gpt-4-turbo-preview')

In [ ]:
last_node_brady2_result_gpt4 = list(plan_brady2_result.values())[-1]
print('QUERY: ' + re.sub(r'\\"response(s)*\\"',r'\n"response\1"',last_node_brady2_result_gpt4.query.replace("\\n","\n")).replace('"query"','\n"query"').replace(',"response"',',\n\n* "response"') + '\n\n')
print('RESPONSE: ' + last_node_brady2_result_gpt4.response)

In [ ]:
assessment_brady_goat =  await assess_proposition2(
    proposition = "Tom Brady is the GOAT of the NFL", 
    short_name = 'brady_goat', 
    search_cache = search_cache, 
    added_instructions = '', 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20,
    print_level = 1
)
display(assessment_brady_goat['kb_dot'])
_=print_summary(assessment_brady_goat)

In [ ]:
question ="Tom Brady is the GOAT of the NFL"
assessment_brady_new = await assess_proposition2(
    proposition = question, 
    short_name = 'brady_new', 
    #use_plan = assessment_nfl_playoffs_new['plan'],
    #plan_only = True,
    search_cache = search_cache,
    client = client, 
    model="gpt-4-turbo-preview", 
    report_to_file = False,
    kb_label_wrap = 20, 
    force_run = True, 
    save_pickle = True,
    print_level = 1
)
display(assessment_brady_new['kb_dot'])
if assessment_brady_new.get('plan_result'):
    _ = print_summary(assessment_brady_new)

#### Michael Jordan, NBA GOAT

In [ ]:
client = instructor.patch(OpenAI())
assessment_jordan_goat2 =  await assess_proposition2(
    proposition = "Michael Jordan is the GOAT of the NBA", 
    short_name = 'jordan_goat2', 
    search_cache = search_cache, 
    added_instructions = '', 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20,
    print_level = 1
)
display(assessment_jordan_goat2['kb_dot'])
_=print_summary(assessment_jordan_goat2)

#### Lebron James, NBA GOAT

In [ ]:
assessment_lebron_goat =  await assess_proposition2(
    proposition = "Lebron James is the GOAT of the NBA", 
    short_name = 'lebron_goat', 
    search_cache = search_cache, 
    added_instructions = '', 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_lebron_goat['kb_dot']

#### Lewis Hamilton, Formula 1 GOAT

In [ ]:
assessment_hamilton_goat_reflect =  await assess_proposition2(
    proposition = "Lewis Hamilton is the GOAT of Formula 1 racing.", 
    short_name = 'hamilton_goat_reflect', 
    plan_only=False,
    search_cache = search_cache, 
    added_instructions = 'Consider the top 5 Formula 1 drivers of all-time.', 
    client = client,
    model = "gpt-4-turbo-preview",
    force_run = True,
    report_to_file = True,
    kb_label_wrap = 20,
    status_updates=True,
    print_level=2
)
display(assessment_hamilton_goat_reflect['kb_dot'])
if assessment_hamilton_goat_reflect.get('plan_result'):
    _=print_summary(assessment_hamilton_goat_reflect)

In [ ]:
assessment_hamilton_goat_new =  await assess_proposition2(
    proposition = "Lewis Hamilton is the GOAT of Formula 1 racing.", 
    short_name = 'hamilton_goat_new', 
    plan_only=True,
    search_cache = search_cache, 
    added_instructions = 'Consider the top 5 Formula 1 drivers of all-time.', 
    client = client,
    model = "gpt-4-turbo-preview",
    force_run = True,
    report_to_file = True,
    kb_label_wrap = 20,
    print_level=2
)
display(assessment_hamilton_goat_new['kb_dot'])
if assessment_hamilton_goat_new.get('plan_result'):
    _=print_summary(assessment_hamilton_goat_new)

In [ ]:
assessment_hamilton_goat_improved =  await assess_proposition2(
    proposition = "Lewis Hamilton is the GOAT of Formula 1 racing.", 
    short_name = 'hamilton_goat_improved', 
    improvement_round=True,
    #plan_only=True,
    search_cache = search_cache, 
    added_instructions = 'Consider the top 5 Formula 1 drivers of all-time.', 
    client = client,
    model = "gpt-4-turbo-preview",
    force_run = False,
    report_to_file = True,
    kb_label_wrap = 20,
    print_level=2
)
display(assessment_hamilton_goat_improved['kb_dot'])
if assessment_hamilton_goat_improved.get('plan_result'):
    _=print_summary(assessment_hamilton_goat_improved)

In [ ]:
type(assessment_hamilton_goat_new['plan'].query_graph[0])

#### Novak Djokovic, Tennis GOAT

In [ ]:
question ="Novak Djokovic is the GOAT of professional tennis."
assessment_djokovic_goat = await assess_proposition2(
    proposition = question, 
    short_name = 'djokovic_goat', 
    search_cache = search_cache,
    client = client, 
    model="gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20, 
    force_run = False, 
    save_pickle = True,
    print_level = 2
)
display(assessment_djokovic_goat['kb_dot'])
if assessment_djokovic_goat.get('plan_result'):
    _ = print_summary(assessment_djokovic_goat)

In [ ]:
question ="Novak Djokovic is the GOAT of professional tennis."
assessment_djokovic_goat_new = await assess_proposition2(
    proposition = question, 
    short_name = 'djokovic_goat_new', 
    search_cache = search_cache,
    client = client, 
    model="gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 24, 
    force_run = False, 
    save_pickle = True,
    print_level = 2
)
display(assessment_djokovic_goat_new['kb_dot'])
if assessment_djokovic_goat_new.get('plan_result'):
    _ = print_summary(assessment_djokovic_goat_new)

#### Sports Prediction: Kansas City defeats San Francisco in Super Bowl LVIII, February 11, 2024

In [ ]:
assessment_super_bowl_lviii_new =  await assess_proposition2(
    proposition =  "Predict if San Francisco 49ers defeat Kansas City Chiefs in Super Bowl LVIII on February 11,2024", 
    short_name = 'super_bowl_lviii_new', 
    improvement_round=True,
    use_plan = None,
    search_cache = search_cache, 
    added_instructions = """
Accept the \<\<GIVEN FACTS\>\> as truths that needn't be questioned. DO NOT create Query nodes for these given facts.

=========

\<\<GIVEN FACTS\>\>:
* Super Bowl LVIII will be played on February 11,2024.
* San Francisco 49ers will play the Kansas City Chiefs in Super Bowl LVIII.

==========
""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    force_run = False,
    save_pickle = True,
    report_to_file = True,
    kb_label_wrap = 24,
    print_level = 2
)
display(assessment_super_bowl_lviii_new['kb_dot'])
if assessment_super_bowl_lviii_new.get('plan_result'):
    _ = print_summary(assessment_super_bowl_lviii_new)

In [ ]:
assessment_super_bowl_lviii_score =  await assess_proposition2(
    proposition =  "Prediction of the final score of the San Francisco 49ers versus the Kansas City Chiefs in Super Bowl LVIII on February 11,2024.", 
    short_name = 'super_bowl_lviii_score', 
    improvement_round=True,
    use_plan = None,
    search_cache = search_cache, 
    added_instructions = """
Devise a query plan to predict the final score of Super Bowl LVIII.  Just find a plan that works.

Guarantee that when the plan is executed it adheres to the following principles:

* A prediction of the final score will be produced for each team. 
* Focus on deriving the narrative that leads to the score rather than external experts and media predictions of final scores or point spreads.
* The plan explains how the factors logically accumulate over the course of the game to result in the final score for each team.
* DO NOT make any statements of caveats about the unpredictability and uncertainty of predicting scores. 
* Each team's strengths and weaknesses are carefully, systematically, and meticulously analyzed to arrive at a reasoned rationale for the predictions.
  * Each team's coaching staffs going head-to-head against their respective coaching adversary: SF offensive coordinator vs. KC defensive coordinator; KC offensive coordinator vs. SF defensive coordinator.
  * Each team's offensive talent paired against each team's defensive talent: SF offense vs. KC defense; KC offense vs. SF defense.
  * Each team's special teams going both ways: SF kicking vs KC receiving/defending; KC kicking vs. SF receiving/defending.
* Provided a reasoned explanation of factors such as fast or slow start by the respective teams and the impact on their odds of scoring quarter-by-quarter.
    
Accept the \<\<GIVEN FACTS\>\> as truths that needn't be questioned. DO NOT create Query nodes for these given facts.

=========

\<\<GIVEN FACTS\>\>:
* Super Bowl LVIII will be played on February 11,2024.
* San Francisco 49ers and Kansas City Chiefs will play against each other in Super Bowl LVIII.

==========
""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    force_run = True,
    save_pickle = True,
    report_to_file = True,
    kb_label_wrap = 24,
    print_level = 2
)
display(assessment_super_bowl_lviii_score['kb_dot'])
if assessment_super_bowl_lviii_score.get('plan_result'):
    _ = print_summary(assessment_super_bowl_lviii_score)

In [ ]:
assessment_super_bowl_lviii =  await assess_proposition2(
    proposition =  "Predict if Baltimore Ravens will (1) meet and (2) defeat Dallas Cowboys in Super Bowl LVIII on February 11,2024", 
    short_name = 'super_bowl_lviii', 
    use_plan = None,
    search_cache = search_cache, 
    added_instructions = """
Ensure that all Query nodes either lead directly or indirectly back to the main proposition. 
So, the query graph has a single "sink" node capturing the main proposition that generates the ultimate conclusions.

Investigate alternative explanations for each subproposition.
Investigate evidence for and evidence against each subproposition.

Accept the <<GIVEN FACTS>> as truths that needn't be questioned. DO NOT create Query nodes for these given facts.

=========

<<GIVEN FACTS>>:
* Super Bowl LVIII will be played on February 11,2024.

==========
""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    dynamic = True,
    force_run = True,
    save_pickle = True,
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_super_bowl_lviii['kb_dot']

In [ ]:
assessment_super_bowl_lviii2 =  await assess_proposition2(
    proposition =  "Predict if Baltimore Ravens will (1) meet and (2) defeat Dallas Cowboys in Super Bowl LVIII on February 11,2024", 
    short_name = 'super_bowl_lviii2', 
    use_plan = None,
    search_cache = search_cache, 
    added_instructions = """
Ensure that all Query nodes either lead directly or indirectly back to the main proposition. 
So, the query graph has a single "sink" node capturing the main proposition that generates the ultimate conclusions.

Investigate alternative explanations for each subproposition.
Investigate evidence for and evidence against each subproposition.

Accept the <<GIVEN FACTS>> as truths that are not to be questioned. DO NOT create Query nodes for any of the <<GIVEN FACTS>>.

=========

<<GIVEN FACTS>>:
* Super Bowl LVIII will be played on February 11,2024.
* Both Baltimore and Dallas have qualified for the playoffs this year (2024).
* The Baltimore Ravens are the #1 seed in the AFC and will have a bye in the Wildcard Round.
* The Dallas Cowboys are the #2 seed in the NFC and will host the #7 seed Green Bay Packers in the Wildcard Round on January 15, 2024. 

==========
""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    dynamic = True,
    force_run = True,
    save_pickle = True,
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_super_bowl_lviii2['kb_dot']

#### Buffalo Bills at Miami Dolphins

In [ ]:
assessment_bills_dolphins =  await assess_proposition2(
    proposition =  "Who will win Buffalo Bills vs Miami Dolphins today (January 7, 2024)?", 
    short_name = 'bills_dolphins', 
    search_cache = search_cache, 
    added_instructions = """
Ensure that all Query nodes either lead directly or indirectly back to the main proposition. 
So, the query graph has a single "sink" node capturing the main proposition that generates the ultimate conclusions.

Investigate alternative explanations for each subproposition.
Investigate evidence for and evidence against each subproposition.

Accept the <<GIVEN FACTS>> as truths that needn't be questioned. DO NOT create Query nodes for these given facts.

=========

<<GIVEN FACTS>>:
* The game is being played today, January 7, 2024.

==========
""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    dynamic = True,
    force_run = True,
    save_pickle = True,
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_bills_dolphins['kb_dot']

In [ ]:
def replace_teams(query_plan : QueryPlan,home: str, away: str, date_string: str = None, verbose: bool = False):
    home_city = re.sub(r'(.+) [^ ]+$',r'\1',home).lower()
    away_city = re.sub(r'(.+) [^ ]+$',r'\1',away).lower()
    nfl_game_txt = assessment_bills_dolphins['plan'].model_dump_json()
    nfl_game_txt = nfl_game_txt.replace("Miami Dolphins",home).replace("Buffalo Bills",away)
    nfl_game_txt = nfl_game_txt.replace("miami",home_city).replace("buffalo",away_city)
    if date_string is not None:
        nfl_game_txt = nfl_game_txt.replace(" today","").replace("January 7, 2024",date_string)
    if verbose:
        print(nfl_game_txt)
    new_plan = QueryPlan.model_validate_json(nfl_game_txt)
    return new_plan

plan_browns_bengals = replace_teams(query_plan=assessment_bills_dolphins['plan'],home ='Cincinnati Bengals', away = 'Cleveland Browns', date_string = None, verbose = False)
kb_browns_bengals = convert_to_graph(plan = plan_browns_bengals, proposition=plan_browns_bengals.query_graph[0].question)
visualize_knowledge_graph(kb_browns_bengals)

#### Cleveland Browns at Cincinnati Bengals

In [ ]:
assessment_browns_bengals_dynamic =  await assess_proposition2(
    proposition =  "Who will win Cleveland Browns vs Cincinnati Bengals today (January 7, 2024)?", 
    short_name = 'browns_bengals_dynamic', 
    use_plan = plan_browns_bengals,
    search_cache = search_cache, 
    added_instructions = """
Ensure that all Query nodes either lead directly or indirectly back to the main proposition. 
So, the query graph has a single "sink" node capturing the main proposition that generates the ultimate conclusions.

Investigate alternative explanations for each subproposition.
Investigate evidence for and evidence against each subproposition.

Accept the <<GIVEN FACTS>> as truths that needn't be questioned. DO NOT create Query nodes for these given facts.

=========

<<GIVEN FACTS>>:
* The game is being played today, January 7, 2024.

==========
""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    dynamic = True,
    force_run = True,
    save_pickle = True,
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
report_browns_bengals_dynamic =  generate_report(
    plan_outline = assessment_browns_bengals_dynamic['plan'].query_plan_outline,
    lastnode=list(assessment_browns_bengals_dynamic['plan_result'].values())[-1],
    computed_queries=assessment_browns_bengals_dynamic['plan_result'],
    search_dict=assessment_browns_bengals_dynamic['search_dict'], 
    filename_stem=assessment_browns_bengals_dynamic['short_name'], 
    write_to_file=False
)

#### Roles of Ingredients in Salad Dressing

In [ ]:
client = instructor.patch(OpenAI())
plan_dressing = query_planner( "What are the functional roles of the ingredients in a commercially manufactured salad dressing? Be meticulous, thorough, accurate, and complete in enumerating all of the types of ingredients.",client)
pprint(plan_dressing.model_dump())

In [ ]:
graph: KnowledgeGraph = convert_to_graph(plan_dressing,question,label_wrap=20)
kb_dot = visualize_knowledge_graph(graph,filename_stem='plan_dressing_gpt4',flip=True, aspect_ratio= 'auto')
kb_dot

In [ ]:
plan_dressing_result = await plan_dressing.execute(client,model="gpt-4-turbo-preview")

In [ ]:
#import re
last_node_dressing_result = list(plan_dressing_result.values())[-1]
print('QUERY: ' + re.sub(r'\\"response(s)*\\"',r'\n"response\1"',last_node_dressing_result.query.replace("\\n","\n")).replace('"query"','\n"query"').replace(',"response"',',\n\n* "response"') + '\n\n')
print('RESPONSE: ' + last_node_dressing_result.response)

In [ ]:
#list(search_cache.keys())

In [ ]:
dressing_search_keys = [q.search_query for q in plan_dressing.query_graph if len(q.search_query)>0]
dressing_search = {key: summarize_raw_search(eval(search_cache[key])) for key in dressing_search_keys if search_cache.get(key,None) is not None}

report_markdown_dressing_gpt4 = generate_report(
    plan_outline = plan_dressing.query_plan_outline,
    lastnode=last_node_dressing_result,
    computed_queries=plan_dressing_result,
    search_dict=dressing_search, 
    filename_stem='plan_dressing_gpt4_again', 
    write_to_file=True
)

In [ ]:
assessment_dressing =  await assess_proposition2(
    proposition =  "What are the functional roles of the ingredients in a commercially manufactured salad dressing?", 
    short_name = 'dressing', 
    improvement_round=True,
    search_cache = search_cache, 
    added_instructions = "Be meticulous, thorough, accurate, and complete in enumerating all of the types of ingredients.", 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 25,
    print_level = 2
)
display(assessment_dressing['kb_dot'])
if assessment_dressing.get('plan_result'):
    print_summary(assessment_dressing)

In [ ]:
assessment_dressing_scm =  await assess_proposition2(
    proposition =  "The fundamental basis of formulating and assessing commercially manufactured salad dressing can be represented in a detailed structural causal model (SCM).", 
    short_name = 'dressing_scm', 
    improvement_round=True,
    search_cache = search_cache, 
    added_instructions = "", 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 25,
    print_level = 2
)
display(assessment_dressing_scm['kb_dot'])
if assessment_dressing_scm.get('plan_result'):
    print_summary(assessment_dressing_scm)

In [ ]:
assessment_dressing_scm2 =  await assess_proposition2(
    proposition =  "The fundamental basis of formulating and assessing commercially manufactured salad dressing can be represented in a detailed structural causal model (SCM).", 
    short_name = 'dressing_scm2', 
    improvement_round=True,
    search_cache = search_cache, 
    added_instructions = """
* Generate a SCM for formulation and assessment of commercially manufactured salad dressing to provide proof of the proposition in the form of a fully executed example.
* Capture the full causal mechanisms from ingredients through all of the subsequent stages of latent variables:
  * composition
  * morphology
  * physical & chemical phenomena
  * product behavior
  * product performance
* Manifest the latent variable at each stage with the appropriate observable outcomes:
  * analytical laboratory measurements
  * human sensory panel assessments
  * consumer survey perception of product benefits
  * consumer survey assessment of product benefits and, ultimately, overall liking
  
""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 25,
    print_level = 2
)
display(assessment_dressing_scm2['kb_dot'])
if assessment_dressing_scm2.get('plan_result'):
    print_summary(assessment_dressing_scm2)

In [ ]:
print(list(assessment_dressing_scm2['plan_result'].values())[-1].response)

In [ ]:
assessment_dressing_scm3 =  await assess_proposition2(
    proposition =  "The fundamental basis of formulating and assessing commercially manufactured salad dressing can be represented in a detailed structural causal model (SCM).", 
    short_name = 'dressing_scm3', 
    improvement_round=True,
    search_cache = search_cache, 
    added_instructions = """
* Generate a SCM for formulation and assessment of commercially manufactured salad dressing to provide proof of the proposition in the form of a fully executed example.
  * Capture the full causal mechanisms from ingredients through all of the subsequent stages of latent variables:
    * composition
    * morphology
    * physical & chemical phenomena
    * product behavior
    * product performance
  * Manifest the latent variable at each stage with the appropriate observable outcomes:
    * analytical laboratory measurements
    * human sensory panel assessments
    * consumer survey perception of product benefits
    * consumer survey assessment of product benefits and, ultimately, overall liking
  * Generate the resulting directed acyclic graph (DAG)
    * create distinct nodes for all entities
      * functional ingredient roles as inputs, latent variables within each stage, and manifest outcomes of all types (lab, sensory, survey)
      * capture as a high-resolution, fine-grain representation of the important entities for reliable causal inference
    * represent all of the direct causal relationships between the entities as edges
    * represent the DAG in DOT syntax suitable for display in Graphviz
  
""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 25,
    print_level = 2
)
display(assessment_dressing_scm3['kb_dot'])
if assessment_dressing_scm3.get('plan_result'):
    print_summary(assessment_dressing_scm3)

In [ ]:
assessment_dressing_scm4 =  await assess_proposition2(
    proposition =  "The fundamental basis of formulating and assessing commercially manufactured salad dressing can be represented in a detailed Structural Causal Model.", 
    short_name = 'dressing_scm4', 
    improvement_round=True,
    search_cache = search_cache, 
    added_instructions = """
Ensure that the query plan includes `Query` nodes to accomplish the following task:

* Generate a Structural Causal Model for the formulation and assessment of commercially manufactured salad dressing to provide proof of the proposition in the form of a fully executed example.
  * Capture the full causal mechanisms from ingredients through all of the subsequent stages of latent variables:
    * composition
    * morphology
    * physical & chemical phenomena
    * product behavior
    * product performance
  * Manifest the latent variable at each stage with the appropriate observable outcomes:
    * analytical laboratory measurements
    * human sensory panel assessments
    * consumer survey perception of product benefits
    * consumer survey assessment of product benefits and, ultimately, overall liking
  * Generate the resulting directed acyclic graph (DAG)
    * create distinct nodes for all entities at the following granularity:
      * functional ingredient roles as inputs, latent variables within each stage of causal mechanisms, and manifest outcomes of all types (lab, sensory, survey)
      * capture as a high-resolution, fine-grain representation of the important entities for reliable causal inference
    * represent all of the direct causal relationships between the entities as edges
    * represent the DAG in DOT syntax suitable for display in Graphviz
  
""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 25,
    print_level = 2
)
display(assessment_dressing_scm4['kb_dot'])
if assessment_dressing_scm4.get('plan_result'):
    print_summary(assessment_dressing_scm4)

In [ ]:
assessment_dressing_scm4_reflect =  await assess_proposition2(
    proposition =  "The fundamental basis of formulating and assessing commercially manufactured salad dressing can be represented in a detailed Structural Causal Model.", 
    short_name = 'dressing_scm4_reflect', 
    improvement_round=True,
    search_cache = search_cache, 
    added_instructions = """
Ensure that the query plan includes `Query` nodes to accomplish the following task:

* Generate a Structural Causal Model for the formulation and assessment of commercially manufactured salad dressing to provide proof of the proposition in the form of a fully executed example.
  * Capture the full causal mechanisms from ingredients through all of the subsequent stages of latent variables:
    * composition
    * morphology
    * physical & chemical phenomena
    * product behavior
    * product performance
  * Manifest the latent variable at each stage with the appropriate observable outcomes:
    * analytical laboratory measurements
    * human sensory panel assessments
    * consumer survey perception of product benefits
    * consumer survey assessment of product benefits and, ultimately, overall liking
  * Generate the resulting directed acyclic graph (DAG)
    * create distinct nodes for all entities at the following granularity:
      * functional ingredient roles as inputs, latent variables within each stage of causal mechanisms, and manifest outcomes of all types (lab, sensory, survey)
      * capture as a high-resolution, fine-grain representation of the important entities for reliable causal inference
    * represent all of the direct causal relationships between the entities as edges
    * represent the DAG in DOT syntax suitable for display in Graphviz
  
""", 
    client = client,
    force_run=True,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 25,
    status_updates=True,
    print_level = 2
)
display(assessment_dressing_scm4_reflect['kb_dot'])
if assessment_dressing_scm4_reflect.get('plan_result'):
    print_summary(assessment_dressing_scm4_reflect)

#### Roles of Ingredients in Sunscreen Lotion

In [ ]:
plan_sunscreen = query_planner( "What are the functional roles of the ingredients in a commercially manufactured sunscreen lotion?",client)
pprint(plan_sunscreen.model_dump())

In [ ]:
plan_sunscreen_result = await plan_sunscreen.execute(client)

In [ ]:
#for k,v in plan_dressing_result.items():
#    print({k: v})

In [ ]:
#import re
last_node_result = list(plan_sunscreen_result.values())[-1]
#last_node_result = plan_dressing_result['1']
print('QUERY: ' + re.sub(r'\\"response(s)*\\"',r'\n"response\1"',last_node_result.query.replace("\\n","\n")).replace('"query"','\n"query"').replace(',"response"',',\n\n* "response"') + '\n\n')
print('RESPONSE: ' + last_node_result.response)

In [ ]:
#list(search_cache.keys())

In [ ]:
sunscreen_search_keys = [q.search_query for q in plan_dressing.query_graph if len(q.search_query)>0]
sunscreen_search = {key: summarize_raw_search(eval(search_cache[key])) for key in sunscreen_search_keys if search_cache.get(key,None) is not None}

#report_markdown = make_report(computed_query = plan_sunscreen_result,search_dict=sunscreen_search)
report_markdown = make_report(computed_query = last_node_result,search_dict=sunscreen_search)
mrkdwn_file_path = 'plan_sunscreen.Rmd'
with open(mrkdwn_file_path, 'w') as file:
    file.write(to_ascii(report_markdown))

#### Salad Dressing, again

In [ ]:
print('What are the functional roles of the ingredients in a commercially manufactured salad dressing?\nContext: {"responses":[{"query":"What is the functional role of emulsifiers in salad dressing?","response":"The functional role of emulsifiers in salad dressing is to stabilize and maintain the emulsion between oil and water-based ingredients. Salad dressings typically consist of oil, vinegar or other acidic ingredients, and water-based ingredients like herbs, spices, and flavorings. Since oil and water do not naturally mix, emulsifiers are added to prevent separation and create a smooth and uniform texture.\\n\\nEmulsifiers work by reducing the surface tension between oil and water, allowing them to mix together more easily. They form a protective layer around the oil droplets, preventing them from coalescing and separating from the water-based ingredients. This helps to create a stable and homogeneous mixture, ensuring that the dressing remains well-blended and does not separate over time.\\n\\nAdditionally, emulsifiers can enhance the mouthfeel and texture of salad dressings by providing a creamy and smooth consistency. They can also improve the overall sensory experience by enhancing flavor release and preventing the formation of undesirable textures, such as oiliness or grittiness.\\n\\nCommon emulsifiers used in salad dressings include lecithin, which is derived from soybeans or egg yolks, and xanthan gum, a natural thickening agent. These emulsifiers help to create a visually appealing and palatable salad dressing that is easy to pour, spread, and mix with salad ingredients."},{"query":"What is the functional role of preservatives in salad dressing?","response":"The functional role of preservatives in salad dressing is to inhibit the growth of microorganisms, such as bacteria, yeast, and mold, thereby extending the shelf life of the product. Preservatives help maintain the freshness, quality, and safety of the salad dressing by preventing spoilage and the growth of harmful bacteria that could cause foodborne illnesses. They also help preserve the flavor, texture, and appearance of the dressing over time."},{"query":"What is the functional role of flavor enhancers in salad dressing?","response":"The functional role of flavor enhancers in salad dressing is to enhance and intensify the taste and aroma of the dressing. Flavor enhancers are typically added to salad dressings to improve the overall flavor profile, making them more appealing and enjoyable to consume. They work by stimulating the taste buds and enhancing the perception of flavors, making the dressing more flavorful and satisfying. Flavor enhancers can include ingredients such as monosodium glutamate (MSG), yeast extracts, natural or artificial flavorings, herbs, spices, or other flavor-enhancing compounds."},{"query":"What is the functional role of thickeners in salad dressing?","response":"Thickeners play a crucial functional role in salad dressings by providing the desired texture and consistency. They help to create a smooth and creamy texture, preventing the dressing from separating or becoming watery. Thickeners also enhance the mouthfeel of the dressing, giving it a more luxurious and satisfying feel when consumed. Additionally, they help to suspend and distribute the various ingredients evenly throughout the dressing, ensuring a consistent flavor and appearance."},{"query":"What is the functional role of acids in salad dressing?","response":"Acids play several functional roles in salad dressing:\\n\\n1. Flavor enhancement: Acids, such as vinegar or citrus juices, add a tangy and refreshing taste to salad dressings. They help balance the flavors by providing a contrast to the richness of oils and other ingredients.\\n\\n2. pH adjustment: Acids help in adjusting the pH of the dressing. Most salad dressings are slightly acidic, which helps to inhibit the growth of bacteria and increase the shelf life of the dressing.\\n\\n3. Emulsification: Acids aid in the emulsification process, which is the process of combining oil and water-based ingredients to create a stable and homogeneous mixture. The acid helps to break down the oil into smaller droplets, allowing it to mix more easily with other ingredients.\\n\\n4. Tenderizing: Some acids, like lemon juice or vinegar, can help tenderize certain ingredients in the salad, such as raw vegetables or proteins. The acid breaks down the fibers and proteins, making them more tender and easier to eat.\\n\\n5. Preservation: Acids act as natural preservatives by inhibiting the growth of bacteria and other microorganisms. They create an environment that is less favorable for spoilage, thus extending the shelf life of the salad dressing.\\n\\nOverall, acids in salad dressing contribute to the taste, texture, preservation, and overall quality of the dressing."},{"query":"What is the functional role of colorants in salad dressing?","response":"The functional role of colorants in salad dressing is to enhance the visual appeal of the product. Colorants are added to give the dressing an attractive and appetizing appearance, making it more visually appealing to consumers. This can help in attracting attention and influencing purchasing decisions. Colorants are used to achieve a desired color or shade that is visually appealing and consistent with the flavor or type of dressing. Additionally, colorants can also be used to differentiate between different varieties or flavors of salad dressings."}]}')
print('-'*100)
print('The functional roles of the ingredients in a commercially manufactured salad dressing are as follows:\n\n1. Emulsifiers: Emulsifiers stabilize and maintain the emulsion between oil and water-based ingredients. They prevent separation and create a smooth and uniform texture in the dressing.\n\n2. Preservatives: Preservatives inhibit the growth of microorganisms, extending the shelf life of the dressing. They help maintain freshness, quality, and safety by preventing spoilage and the growth of harmful bacteria.\n\n3. Flavor enhancers: Flavor enhancers intensify the taste and aroma of the dressing, making it more appealing and enjoyable to consume. They stimulate the taste buds and enhance the perception of flavors.\n\n4. Thickeners: Thickeners provide the desired texture and consistency to the dressing. They create a smooth and creamy texture, prevent separation, and enhance the mouthfeel of the dressing.\n\n5. Acids: Acids add flavor, adjust the pH, aid in emulsification, tenderize ingredients, and act as natural preservatives. They enhance the taste, texture, preservation, and overall quality of the dressing.\n\n6. Colorants: Colorants enhance the visual appeal of the dressing. They give the dressing an attractive and appetizing appearance, making it visually appealing to consumers and helping to differentiate between different varieties or flavors of salad dressings.')

## Book Reviews

#### Assess Annie Duke's book "Thinking in Bets"

In [ ]:
assessment_duke_bets =  await assess_proposition2(
    proposition =  "Explain the premise of and assess the importance of the book 'Thinking in Bets' by Annie Duke.", 
    short_name = 'duke_bets', 
    search_cache = search_cache, 
    added_instructions = "", 
    client = client,
    model = "gpt-4-turbo-preview", 
    dynamic = True,
    report_to_file = True,
    kb_label_wrap = 25,
    print_level = 2
)
display(assessment_duke_bets['kb_dot'])
if assessment_duke_bets.get('plan_result'):
    print_summary(assessment_duke_bets)

#### Assess Annie Duke's book "Quit"

In [ ]:
plan_duke_quit_dict = eval(re.sub("Thinking in Bets","Quit",assessment_duke_bets['plan'].model_dump_json()))
plan_duke_quit = QueryPlan(**plan_duke_quit_dict)
visualize_knowledge_graph(convert_to_graph(plan_duke_quit,"Explain the premise of and assess the importance of the book 'Quit' by Annie Duke."))

In [ ]:
assessment_duke_quit =  await assess_proposition2(
    proposition =  proposition, 
    short_name = 'duke_quit', 
    use_plan = plan_duke_quit,
    search_cache = search_cache, 
    added_instructions = "", 
    client = client,
    model = "gpt-4-turbo-preview", 
    force_run = False,
    report_to_file = True,
    kb_label_wrap = 25,
    print_level = 2
)
#display(assessment_duke_quit['kb_dot'])
if assessment_duke_quit.get('plan_result'):
    print_summary(assessment_duke_quit)

#### Assess Andy Grove's book "Only the Paranoid Survive"

In [ ]:
proposition_duke_quit = "Explain the premise of and assess the importance of the book 'Quit' by Annie Duke."
proposition = re.sub("Annie Duke","Andrew S. Grove",re.sub("Quit","Only the Paranoid Survive",proposition_duke_quit))
plan_json = re.sub("Annie Duke","Andrew S. Grove",re.sub("Quit","Only the Paranoid Survive",assessment_duke_quit['plan'].model_dump_json()))
plan_dict = eval(plan_json)
plan = QueryPlan(**plan_dict)
visualize_knowledge_graph(convert_to_graph(plan,proposition))

In [ ]:
assessment_grove_paranoid =  await assess_proposition2(
    proposition =  proposition, 
    short_name = 'grove_paranoid', 
    use_plan = plan,
    search_cache = search_cache, 
    added_instructions = "", 
    client = client,
    model = "gpt-4-turbo-preview", 
    force_run = False,
    report_to_file = True,
    kb_label_wrap = 25,
    print_level = 2
)
#display(assessment_grove_paranoid['kb_dot'])
if assessment_grove_paranoid.get('plan_result'):
    print_summary(assessment_grove_paranoid)

#### Murder on the Orient Express

In [ ]:
question ="Agatha Christie's 'Murder on the Orient Express' has a great plot."
assessment_orient_express = await assess_proposition2(
    proposition = question,
    short_name = 'orient_express', 
    search_cache = search_cache,
    client = client, 
    model="gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 25, 
    force_run = True, 
    save_pickle = True,
    print_level = 2
)
display(assessment_orient_express['kb_dot'])
if assessment_orient_express.get('plan_result'):
    _ = print_summary(assessment_orient_express)

In [ ]:
for q in assessment_orient_express['plan'].query_graph:
    print(f"{q.id}: {q.question}\n-- {q.dependencies}")

In [ ]:
question ="Agatha Christie's book 'Murder on the Orient Express' has a great plot."
assessment_orient_express2 = await assess_proposition2(
    proposition = question,
    short_name = 'orient_express2', 
    search_cache = search_cache,
    client = client, 
    model="gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 25, 
    force_run = False, 
    save_pickle = True,
    print_level = 2
)
display(assessment_orient_express2['kb_dot'])
if assessment_orient_express2.get('plan_result'):
    _ = print_summary(assessment_orient_express2)

In [ ]:
for q in assessment_orient_express2['plan'].query_graph:
    print(f"{q.id}: {q.question}\n-- {q.dependencies}")

## Culture

#### Culture wars

In [ ]:
assessment_culture_wars = await assess_proposition2(
    proposition = "ESPN is the epicenter of the American 'culture wars'.", 
    short_name = 'culture_wars', 
    added_instructions = """
Especially, consider tensions between the hosts, commentators, guests, and audiences of the following ESPN shows:

* First Take
* The Pat McAfee Show
* Get Up
* SportsCenter

And the tensions in the following sports and their relationships with stars in pop, rap, and country music and the movie industry:

* NFL
* NBA
* WNBA
* NCAA Mens Football
* NCAA Mens Basketball
* NCAA Womens Basketball
* NHL
* MSL
""",
    improvement_round=True,
    search_cache = search_cache,
    client = client, 
    model="gpt-4-turbo-preview", 
    report_to_file = True, 
    kb_label_wrap = 24, 
    force_run = False, 
    save_pickle = True,
    print_level = 2
)
display(assessment_culture_wars['kb_dot'])
if assessment_culture_wars.get('plan_result'):
    _ = print_summary(assessment_culture_wars)

#### "Chill soundtrack for a generation"

According to this [article at Fatherly](https://www.fatherly.com/entertainment/air-talkie-walkie-retrospective).

In [ ]:
question ="What album, 20 years ago, became the 'chill soundtrack of a generation'."
assessment_album_chill = await assess_proposition2(
    proposition = question,
    short_name = 'album_chill', 
    search_cache = search_cache,
    client = client, 
    model="gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 28, 
    force_run = False, 
    save_pickle = True,
    print_level = 2
)
display(assessment_album_chill['kb_dot'])
if assessment_album_chill.get('plan_result'):
    _ = print_summary(assessment_album_chill)

In [ ]:
question ="What album of 2004 became the 'chill soundtrack for a generation'."
assessment_album_chill2 = await assess_proposition2(
    proposition = question,
    short_name = 'album_chill2', 
    added_instructions = """
    
* Identify candidate albums of 2004.
* Enumerate criteria to be 'chill soundtrack of a generation'.
* Evaluate candidates by the criteria.
* Answer the top-level question.

""",
    search_cache = search_cache,
    client = client, 
    model="gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 28, 
    force_run = False, 
    save_pickle = True,
    print_level = 2
)
display(assessment_album_chill2['kb_dot'])
if assessment_album_chill2.get('plan_result'):
    _ = print_summary(assessment_album_chill2)

In [ ]:
question ="The 2004 album 'Talkie Walkie' by Air has become the 'chill soundtrack for a generation'."
assessment_album_chill2_improved = await assess_proposition2(
    proposition = question,
    short_name = 'album_chill2_improved', 
    added_instructions = """
Consider the following information.

The essence of this proposition is posed as a question in The Fatherly Turntable article
["20 Years Ago, One Album Became The Chill Soundtrack For A Generation: Why Air’s *Talkie Walkie* is more than that one *Lost in Translation* song."](https://www.fatherly.com/entertainment/air-talkie-walkie-retrospective).

After noting the popularity of the tracks *Run*, used in the TV show *Veronica Mars*, and *Alone in Kyoto*, used in the movie *Lost in Translation*,
the article poses the following quintessential question:

* "Are there Air songs left on *Talkie Walkie* that can become part of the movie of your life, today, two decades later?"

And answered within that article with a resounding "Yes".

""",
    search_cache = search_cache,
    client = client, 
    model="gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 28, 
    force_run = False, 
    save_pickle = True,
    print_level = 2
)
display(assessment_album_chill2_improved['kb_dot'])
if assessment_album_chill2_improved.get('plan_result'):
    _ = print_summary(assessment_album_chill2_improved)

In [ ]:
question ="The 2004 album 'Talkie Walkie' by Air has become the 'chill soundtrack for a generation'."
assessment_album_chill2_improved_a = await assess_proposition2(
    proposition = question,
    short_name = 'album_chill2_improved_a', 
    added_instructions = """
Make sure that all search queries unambiguously reference "the Air album 'Talkie Walkie'" to avoid confusion with the walkie-talkie communication device.
DO NOT discuss the walkie-talkie communication device.

Consider the following information.

The essence of this proposition is posed as a question in The Fatherly Turntable article
["20 Years Ago, One Album Became The Chill Soundtrack For A Generation: Why Air’s *Talkie Walkie* is more than that one *Lost in Translation* song."](https://www.fatherly.com/entertainment/air-talkie-walkie-retrospective).

After noting the popularity of the tracks *Run*, used in the TV show *Veronica Mars*, and *Alone in Kyoto*, used in the movie *Lost in Translation*,
the article poses the following quintessential question:

* "Are there Air songs left on *Talkie Walkie* that can become part of the movie of your life, today, two decades later?"

And the article answers that question with a resounding "Yes".

""",
    search_cache = search_cache,
    client = client, 
    model="gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 28, 
    force_run = False, 
    save_pickle = True,
    print_level = 2
)
display(assessment_album_chill2_improved_a['kb_dot'])
if assessment_album_chill2_improved_a.get('plan_result'):
    _ = print_summary(assessment_album_chill2_improved_a)

In [ ]:
question ="The 2004 album 'Talkie Walkie' by Air has become the 'chill soundtrack for a generation'."
assessment_album_chill2_improved_b = await assess_proposition2(
    proposition = question,
    short_name = 'album_chill2_improved_b', 
    added_instructions = """
Make sure that all search queries unambiguously reference "the Air album 'Talkie Walkie'" to avoid confusion with the walkie-talkie communication device.
DO NOT discuss the walkie-talkie communication device.

Consider the following information.

The essence of this proposition is posed as a question in The Fatherly Turntable article
["20 Years Ago, One Album Became The Chill Soundtrack For A Generation: Why Air’s *Talkie Walkie* is more than that one *Lost in Translation* song."](https://www.fatherly.com/entertainment/air-talkie-walkie-retrospective).

After noting the popularity of the tracks *Run*, used in the TV show *Veronica Mars*, and *Alone in Kyoto*, used in the movie *Lost in Translation*,
the article poses the following question:

* "Are there Air songs left on *Talkie Walkie* that can become part of the movie of your life, today, two decades later?"

And the article answers that question with a resounding "Yes".

Discuss that central concept: "songs... that can become part of the movie of your life, today, two decades later" -- a soundtrack of the lives of a generation.

""",
    search_cache = search_cache,
    client = client, 
    model="gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 28, 
    force_run = False, 
    save_pickle = True,
    print_level = 2
)
display(assessment_album_chill2_improved_b['kb_dot'])
if assessment_album_chill2_improved_b.get('plan_result'):
    _ = print_summary(assessment_album_chill2_improved_b)

In [ ]:
question ='For those who were young adults in 2004, the Air album *Talkie Walkie* has songs "... that can become part of the movie of your life, today, two decades later" -- a soundtrack of the lives of a generation.'
assessment_album_chill2_improved_c = await assess_proposition2(
    proposition = question,
    short_name = 'album_chill2_improved_c', 
    added_instructions = """
Make sure that all search queries unambiguously reference "the Air album 'Talkie Walkie'" to avoid confusion with the walkie-talkie communication device.
DO NOT discuss the walkie-talkie communication device.

Consider the following information:

* The essence of this proposition is posed as a question in The Fatherly Turntable article
["20 Years Ago, One Album Became The Chill Soundtrack For A Generation: Why Air’s *Talkie Walkie* is more than that one *Lost in Translation* song."](https://www.fatherly.com/entertainment/air-talkie-walkie-retrospective).
After noting the popularity of the tracks *Run*, used in the TV show *Veronica Mars*, and *Alone in Kyoto*, used in the movie *Lost in Translation*,
the article poses the following question:
   * "Are there Air songs left on *Talkie Walkie* that can become part of the movie of your life, today, two decades later?"
   * And the article answers that question with a resounding "Yes".

""",
    search_cache = search_cache,
    client = client, 
    model="gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 28, 
    force_run = True, 
    save_pickle = True,
    print_level = 2
)
display(assessment_album_chill2_improved_c['kb_dot'])
if assessment_album_chill2_improved_c.get('plan_result'):
    _ = print_summary(assessment_album_chill2_improved_c)

#### TwoSetViolin

In [ ]:
assessment_twosetviolin =  await assess_proposition2(
    proposition =  "TwoSetViolin is revolutionizing the classical music fan experience.", 
    short_name = 'twosetviolin', 
    search_cache = search_cache, 
    added_instructions = """
Ensure that all Query nodes lead directly or indirectly back to the main proposition. 
So, the query graph has a single "sink" node capturing the main proposition that generates the ultimate conclusions.
Be sure to leverage internet searches while investigating the entities, concepts, ideas, and propositions.
""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    dynamic = True,
    force_run = False,
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_twosetviolin_improved =  await assess_proposition2(
    proposition =  "TwoSetViolin is revolutionizing the classical music fan experience.", 
    short_name = 'twosetviolin_improved', 
    search_cache = search_cache, 
    improvement_round=True,
    client = client,
    model = "gpt-4-turbo-preview", 
    force_run = False,
    report_to_file = True,
    kb_label_wrap = 20,
    print_level=2
)
display(assessment_twosetviolin_improved['kb_dot'])
if assessment_twosetviolin_improved.get('plan_result'):
    _ = print_summary(assessment_twosetviolin_improved)

#### Taylor Swift

In [ ]:
assessment_taylor_swift =  await assess_proposition2(
    proposition =  "Taylor Swift is revolutionizing the pop music fan experience.", 
    short_name = 'taylor_swift', 
    search_cache = search_cache, 
    added_instructions = """
Ensure that all Query nodes lead directly or indirectly back to the main proposition. 
So, the query graph has a single "sink" node capturing the main proposition that generates the ultimate conclusions.
Be sure to leverage internet searches while investigating the entities, concepts, ideas, and propositions.
""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    dynamic = True,
    force_run = False,
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
outline_processing(assessment_taylor_swift['plan'],True)

In [ ]:
assessment_taylor_swift['kb_dot']

#### TwoSetViolin can learn from Taylor Swift

In [ ]:
assessment_twoset_swift =  await assess_proposition2(
    proposition =  "TwoSetViolin can learn from Taylor Swift about career maturation as they, their careers, and their fans mature.", 
    short_name = 'twoset_swift', 
    search_cache = search_cache, 
    added_instructions = """
Ensure that all Query nodes lead directly or indirectly back to the main proposition. 
So, the query graph has a single "sink" node capturing the main proposition that generates the ultimate conclusions.
Be sure to leverage internet searches while investigating the entities, concepts, ideas, and propositions.
""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    dynamic = True,
    force_run = False,
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
outline_processing(assessment_twoset_swift['plan'],True)

In [ ]:
assessment_twoset_swift['kb_dot']

#### Netflix 'The Diplomat'

In [ ]:
assessment_netflix_diplomat =  await assess_proposition(
    proposition = "In the Netflix drama 'The Diplomat', the portrayal of diversity in the highest ranks of government, esp. the diplomatic corp, does not accurately reflect reality.", 
    short_name = 'netflix_diplomat', 
    search_cache = search_cache, 
    added_instructions =  """
Accept the <<GIVEN FACTS>> as truths that needn't be questioned. DO NOT create Query nodes for these facts.
=========

<<GIVEN FACTS>>:
* Netflix has a drama called 'The Diplomat'.
* Season 1 of 'The Diplomat' has a racially & ethnically diverse cast.
* 'The Diplomat' portrays many people of color in the highest ranks of the U.S. government, esp. the diplomatic corp -- e.g. ambassadors, diplomats, foreign ministers, etc.
* 'The Diplomat' portrays many people of color in the highest ranks of the U.K. government, esp. the diplomatic corp -- e.g. ambassadors, diplomats, foreign ministers, etc.
* 'The Diplomat' portrays many women in the highest ranks of the U.S. government, esp. the diplomatic corp -- e.g. ambassadors, diplomats, foreign ministers, etc.
* 'The Diplomat' portrays many women in the highest ranks of the U.K. government, esp. the diplomatic corp -- e.g. ambassadors, diplomats, foreign ministers, etc.

==========

""", 
    client = client,
    model = "gpt-4-turbo-preview",
    dynamic = True,
    report_to_file = True,
    kb_label_wrap = 20,
    force_run = False
)

In [ ]:
print(qg[-1])

In [ ]:
qp = QueryPlan(**(assessment_netflix_diplomat['plan'].model_dump()))
qg = qp.query_graph

qnodes = {q.id : q for q in qg}

qnodes['main_proposition'].dependencies.extend(['racial_ethnic_diversity_cast','gender_diversity_cast'])
qnodes['racial_ethnic_diversity_us'].dependencies.extend(['racial_ethnic_diversity_cast'])
qnodes['racial_ethnic_diversity_uk'].dependencies.extend(['racial_ethnic_diversity_cast'])
qnodes['gender_diversity_us'].dependencies.extend(['gender_diversity_cast'])
qnodes['gender_diversity_uk'].dependencies.extend(['gender_diversity_cast'])

qg = list(qnodes.values())

qg_add = [
    Query(
        id='racial_ethnic_diversity_cast',
        question='What is the racial and ethnic composition of the cast of the Netflix drama "The Diplomat", esp. roles portraying the highest ranks of the U.K. government, esp. the diplomatic corp?',
        dependencies =[],
        search_query='racial ethnic composition of the cast of Netflix "The Diplomat" (season 1) with roles'
    ),
    Query(id='gender_diversity_cast',
        question='What is the gender composition of the cast of the Netflix drama "The Diplomat", esp. roles portraying the highest ranks of the U.K. government, esp. the diplomatic corp?',
        dependencies =[],
        search_query='gender composition of the cast of Netflix "The Diplomat" (season 1) with roles'
    )
]
qg.extend(qg_add)

qp.query_graph = qg

In [ ]:
assessment_netflix_diplomat2 =  await assess_proposition(
    proposition = "In the Netflix drama 'The Diplomat', the portrayal of diversity in the highest ranks of government, esp. the diplomatic corp, does not accurately reflect reality.", 
    short_name = 'netflix_diplomat2', 
    use_plan = qp,
    search_cache = search_cache, 
    added_instructions =  """
Accept the <<GIVEN FACTS>> as truths that needn't be questioned. DO NOT create Query nodes for these facts.
=========

<<GIVEN FACTS>>:
* Netflix has a drama called 'The Diplomat'.
* Season 1 of 'The Diplomat' has a racially & ethnically diverse cast.
* 'The Diplomat' portrays many people of color in the highest ranks of the U.S. government, esp. the diplomatic corp -- e.g. ambassadors, diplomats, foreign ministers, etc.
* 'The Diplomat' portrays many people of color in the highest ranks of the U.K. government, esp. the diplomatic corp -- e.g. ambassadors, diplomats, foreign ministers, etc.
* 'The Diplomat' portrays many women in the highest ranks of the U.S. government, esp. the diplomatic corp -- e.g. ambassadors, diplomats, foreign ministers, etc.
* 'The Diplomat' portrays many women in the highest ranks of the U.K. government, esp. the diplomatic corp -- e.g. ambassadors, diplomats, foreign ministers, etc.

==========

""", 
    client = client,
    model = "gpt-4-turbo-preview",
    dynamic = True,
    report_to_file = True,
    kb_label_wrap = 20,
    force_run = True
)

#### Rainer Rilke -- 'dragons'

In [ ]:
assessment_rilke_dragons =  await assess_proposition2(
    proposition =  "Rainer Rilke references dragons in a perfect metaphor for life's challenges.", 
    short_name = 'rilke_dragons', 
    search_cache = search_cache, 
    added_instructions = """
Ensure that all Query nodes lead directly or indirectly back to the main proposition. 
So, the query graph has a single "sink" node capturing the main proposition that generates the ultimate conclusions.

Accept the <<GIVEN FACTS>> as truths that needn't be questioned. DO NOT create Query nodes for these facts.
=========

<<GIVEN FACTS>>:
* Quote by Rilke: "Perhaps all the dragons in our lives are princesses who are only waiting to see us act, just once, with beauty and courage. Perhaps everything that frightens us is, in its deepest essence, something helpless that wants our love."
-- Rainer Maria Rilke, Letters to a Young Poet
==========

""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    dynamic = False,
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_rilke_dragons['kb_dot']

#### Rainer Rilke -- Philosopher

In [ ]:
assessment_rilke_philosopher2 =  await assess_proposition2(
    proposition =  "Rainer Rilke was a philosopher with great insights into life.", 
    short_name = 'rilke_philosopher2', 
    search_cache = search_cache, 
    improvement_round=True,
    added_instructions = "", 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20,
    print_level=2
)
display(assessment_rilke_philosopher2['kb_dot'])
if assessment_rilke_philosopher2.get('plan_result'):
    _=print_summary(assessment_rilke_philosopher2)

#### UK Lottery

In [ ]:
assessment_uk_lottery =  await assess_proposition(
    proposition =  "Mathematicians found a guaranteed way to win the UK lottery in 2023.", 
    short_name = 'uk_lottery', 
    search_cache = search_cache, 
    added_instructions = '', 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_uk_lottery['kb_dot']

#### 'The Expanse' greatest sci-fi of all time

In [ ]:
assessment_expanse_goat_reflect =  await assess_proposition2(
    proposition =  "'The Expanse' sci-fi novel series surpasses all great sci-fi novels other than 'The Expanse'.", 
    short_name = 'expanse_goat_reflect',
    improvement_round=True,
    search_cache = search_cache, 
    added_instructions = """
Ensure that all Query nodes lead directly or indirectly back to the main proposition. 
So, the query graph has a single "sink" node capturing the main proposition that generates the ultimate conclusions.

Accept the <<GIVEN FACTS>> as truths that needn't be questioned. DO NOT create Query nodes for these facts.
=========

<<GIVEN FACTS>>:
* The Expanse is a series of science fiction novels by James S. A. Corey, the joint pen name of authors Daniel Abraham and Ty Franck.

==========

""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    force_run=True,
    report_to_file = True,
    status_updates=True,
    print_level = 2
)
display(assessment_expanse_goat_reflect['kb_dot'])
_=print_summary(assmnt=assessment_expanse_goat_reflect)

In [ ]:
assessment_expanse_goat =  await assess_proposition2(
    proposition =  "'The Expanse' sci-fi novel series surpasses all great sci-fi novels other than 'The Expanse'.", 
    short_name = 'expanse_goat', 
    search_cache = search_cache, 
    added_instructions = """
Ensure that all Query nodes lead directly or indirectly back to the main proposition. 
So, the query graph has a single "sink" node capturing the main proposition that generates the ultimate conclusions.

Accept the <<GIVEN FACTS>> as truths that needn't be questioned. DO NOT create Query nodes for these facts.
=========

<<GIVEN FACTS>>:
* The Expanse is a series of science fiction novels by James S. A. Corey, the joint pen name of authors Daniel Abraham and Ty Franck.

==========

""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    print_level = 0
)
display(assessment_expanse_goat['kb_dot'])
_=print_summary(assmnt=assessment_expanse_goat)

## Science & Technology

#### Low Calorie Density Foods

In [ ]:
assessment_low_calorie_density =  await assess_proposition2(
    proposition =  "Low calorie density foods are excellent for maintaining a healthy weight.", 
    short_name = 'low_calorie_density', 
    improvement_round=True,
    search_cache = search_cache, 
    added_instructions = """
Things to consider:

* Provide plenty of examples of low calorie density foods, with explanations for each.
* Provide examples of good meal plans with low calorie density foods.
""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 35,
    print_level = 2,
    force_run = True
)
display(assessment_low_calorie_density['kb_dot'])
if assessment_low_calorie_density.get('plan_result'):
    print_summary(assessment_low_calorie_density)

#### Fat calorie burn

In [ ]:
assessment_fat_calories =  await assess_proposition2(
    proposition =  "The '500 cal/day Myth' for a 1-pound-a-week weight loss is misleading; so it is better to simply habitually eat low calorie-density foods.", 
    short_name = 'fat_calories', 
    improvement_round=True,
    search_cache = search_cache, 
    added_instructions = """
Things to consider:

* The 500 cal/day Myth: A pound of fat is equivalent to about 3500 calories, so to lose a pound in a week, you must achieve a caloric deficit of 3500/7 = 500 cal/day relative to the calories calculated for your starting basal metabolic rate.
* It is a myth because as you lose weight, your basal metabolic rate (BMR) decreases, so if you set your caloric deficit relative to starting BMR, your rate of weight loss will decrease.
* Moreover, counting calories every day is not a sustainable habit for most people; so it is better to establish healthy eating habits rather than counting calories.
* What are the best sustainable habits for healthy weight maintenance for a 65-year-old man who is 30-lbs overweight?
""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 30,
    print_level = 2,
    force_run = True
)
display(assessment_fat_calories['kb_dot'])
if assessment_fat_calories.get('plan_result'):
    print_summary(assessment_fat_calories)

#### Deflation due to Generative AI

In [ ]:
assessment_ai_deflation =  await assess_proposition2(
    proposition =  "Generative AI will lead to significant and widespread deflation in the compensation of white-collar workers.", 
    short_name = 'ai_deflation', 
    improvement_round=True,
    search_cache = search_cache, 
    added_instructions = "", 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 30,
    print_level = 2
)
display(assessment_ai_deflation['kb_dot'])
if assessment_ai_deflation.get('plan_result'):
    print_summary(assessment_ai_deflation)

In [ ]:
assessment_cheapening_creation =  await assess_proposition2(
    proposition =  "In a capitalist economy, devaluation of the value of creation depresses wages for jobs of all types whether creation-driven (design, engineering, etc.) or production-driven (manufacturing, assembly, service, etc.).", 
    short_name = 'cheapening_creation', 
    improvement_round=True,
    search_cache = search_cache, 
    added_instructions = """
Consider the following propositions:

* Creative talents set a ceiling and buoy wages for jobs of all types.
* Until now, technology across all of history has been a net creator of creation-driven jobs even as it has displaced production-driven jobs.
* The new trait of Generative AI as opposed to past technological disruptions is that it is the first technology that can substantially reduce the number of humans needed in creation-driven jobs even moreso than production-driven jobs.
* A cheapening of creative assets leads to depression of wages across all jobs.
* Downward pressure on wages leads to deflation and, if too rapidly occurring, to economic depression.
""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 30,
    print_level = 2
)
display(assessment_cheapening_creation['kb_dot'])
if assessment_cheapening_creation.get('plan_result'):
    print_summary(assessment_cheapening_creation)

#### Income inequality exacerbated by Generative AI

In [ ]:
assessment_ai_inequality =  await assess_proposition2(
    proposition =  "Generative AI will exacerbate income inequality.", 
    short_name = 'ai_inequality', 
    improvement_round=True,
    search_cache = search_cache, 
    added_instructions = "", 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 30,
    print_level = 2
)
display(assessment_ai_inequality['kb_dot'])
if assessment_ai_inequality.get('plan_result'):
    print_summary(assessment_ai_inequality)

In [ ]:
assessment_ai_inequality2 =  await assess_proposition2(
    proposition =  "Generative AI will exacerbate income inequality.", 
    short_name = 'ai_inequality2', 
    improvement_round=True,
    search_cache = search_cache, 
    added_instructions = """
* Consider changes in wage structure -- not only changes in job distribution.
* Attempt to assess degree (quantitative extent) -- not only type (qualitative nature) -- of economic changes.
* Consider the rapidity of economic changes relative to the slowness at which governmental policy changes.
""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 30,
    print_level = 2
)
display(assessment_ai_inequality2['kb_dot'])
if assessment_ai_inequality2.get('plan_result'):
    print_summary(assessment_ai_inequality2)

#### Einstein's theory of relativity

In [ ]:
assessment_explain_relativity2 =  await assess_proposition2(
    proposition =  "Explain Einstein's theories of relativity.", 
    short_name = 'explain_relativity2', 
    search_cache = search_cache, 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20,
    print_level=2
)
display(assessment_explain_relativity2['kb_dot'])
if assessment_explain_relativity2.get('plan_result'):
    print_summary(assessment_explain_relativity2)

#### Richard Feynman -- Physicist

In [ ]:
assessment_feynman_physicist =  await assess_proposition2(
    proposition =  "Richard Feynman was the greatest physicist of his generation.", 
    short_name = 'feynman_physicist', 
    search_cache = search_cache, 
    added_instructions = """
Ensure that all Query nodes lead directly or indirectly back to the main proposition. 
So, the query graph has a single "sink" node capturing the main proposition that generates the ultimate conclusions.

Accept the <<GIVEN FACTS>> as truths that needn't be questioned. DO NOT create Query nodes for these facts.
=========

<<GIVEN FACTS>>:
* Richard Feynman was a famous physicist.

==========""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    dynamic = True,
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_feynman_physicist['kb_dot']

#### Einstein and Hilbert

In [ ]:
assessment_einstein_hilbert =  await assess_proposition2(
    proposition =  "Einstein needed Hilbert to help him understand the mathematics that led to the Theory of General Relativity.", 
    short_name = 'einstein_hilbert', 
    search_cache = search_cache, 
    added_instructions = """
Ensure that all Query nodes lead directly or indirectly back to the main proposition. 
So, the query graph has a single "sink" node capturing the main proposition that generates the ultimate conclusions.

Investigate alternative explanations for each subproposition.
Investigate evidence for and evidence against each subproposition.

""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    dynamic = True,
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_einstein_hilbert['kb_dot']

#### Explain Branden Fitelsen Theory

In [ ]:
assessment_explain_fitelsen =  await assess_proposition2(
    proposition =  "Explain Bayesian Confirmation Theory, per Branden Fitelsen.", 
    short_name = 'explain_fitelsen', 
    search_cache = search_cache, 
    added_instructions = """Especially, thoroughly investigate these topics:
    
* Bayesian Confirmation Theory in the context of scientific inquiry and the logic of science. 
* Confirmation as relative to induction and deduction.
* Confirmation perspectives with respect to Carnap, Hume, and Howson & Urbach.  
* Enumeration and contrast of Bayesian confirmation measures.
* Practical application in reasoning and decision-making.
""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    dynamic = True,
    report_to_file = True,
    kb_label_wrap = 25,
    print_level = 2
)
display(assessment_explain_fitelsen['kb_dot'])
if assessment_explain_fitelsen.get('plan_result'):
    print_summary(assessment_explain_fitelsen)

#### Explain Homotopy Theory

In [ ]:
assessment_explain_homotopy =  await assess_proposition2(
    proposition =  "Explain homotopy theory.", 
    short_name = 'explain_homotopy', 
    search_cache = search_cache, 
    added_instructions = "", 
    client = client,
    model = "gpt-4-turbo-preview", 
    dynamic = True,
    report_to_file = True,
    kb_label_wrap = 20
)
display(assessment_explain_homotopy['kb_dot'])
if assessment_explain_homotopy.get('plan_result'):
    print_summary(assessment_explain_homotopy)

#### Explain Bayesian belief networks

In [ ]:
assessment_explain_bbn =  await assess_proposition2(
    proposition =  "Explain Bayesian belief networks.", 
    short_name = 'explain_bbn', 
    search_cache = search_cache, 
    added_instructions = """
Do so in a manner that a 7th-grader can understand.
Provide many examples in a clear and concise manner.

Ensure that all Query nodes lead directly or indirectly back to the main proposition. 
So, the query graph has a single "sink" node capturing the main proposition that generates the ultimate conclusions.

Investigate alternative explanations for each subproposition.
Investigate evidence for and evidence against each subproposition.

""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    dynamic = True,
    report_to_file = True,
    kb_label_wrap = 20
)
display(assessment_explain_bbn['kb_dot'])
_=print_summary(assessment_explain_bbn)

#### Investigate Yuan et al, "Most Relevant Explanation in Bayesian Networks"

In [ ]:
assessment_mre_bbn =  await assess_proposition2(
    proposition = 'Investigate "Most Relevant Explanation in Bayesian Networks" by Yuan et al.', 
    short_name = 'mre_bbn', 
    search_cache = search_cache, 
    added_instructions = """
Do so in a manner that a 7th-grader can understand.
Provide many examples in a clear and concise manner.

All Query nodes must lead directly or indirectly back to the main proposition (the 'sink' node, which is the only node upon which no other node depends). 
The query graph must have one and only one "sink" node capturing the main proposition that generates the ultimate conclusions.

Consider the work in the context of 
(a) Explanation - given evidence, find hypotheses that explain it - and 
(b) Argumentation - given hypothesis, find evidence that supports and refute it.

Investigate alternative explanations, i.e., hypotheses, for each subproposition.
Investigate evidence for and evidence against each subproposition.

Rely upon the <<GIVEN FACTS>> as established truths and resources for further information:
==================================
<<GIVEN FACTS>>
* Yuan et al (Changhe Yuan, Heejin Lim, and Tsai-Ching Lu) published "Most Relevant Explanation in Bayesian Networks" at arxiv: https://arxiv.org/abs/1401.3893
  - Abstract: 
    A major inference task in Bayesian networks is explaining why some variables are observed in their particular states using a set of target variables. 
    Existing methods for solving this problem often generate explanations that are either too simple (underspecified) or too complex (overspecified). In this paper, 
    we introduce a method called Most Relevant Explanation (MRE) which finds a partial instantiation of the target variables that maximizes the generalized Bayes factor (GBF) 
    as the best explanation for the given evidence. Our study shows that GBF has several theoretical properties that enable MRE to automatically identify the most relevant 
    target variables in forming its explanation. In particular, conditional Bayes factor (CBF), defined as the GBF of a new explanation conditioned on an existing explanation, 
    provides a soft measure on the degree of relevance of the variables in the new explanation in explaining the evidence given the existing explanation. As a result, 
    MRE is able to automatically prune less relevant variables from its explanation. We also show that CBF is able to capture well the explaining-away phenomenon that is often 
    represented in Bayesian networks. Moreover, we define two dominance relations between the candidate solutions and use the relations to generalize MRE to find a set of 
    top explanations that is both diverse and representative. Case studies on several benchmark diagnostic Bayesian networks show that MRE is often able to find explanatory hypotheses 
    that are not only precise but also concise.
==================================

""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    dynamic = True,
    report_to_file = True,
    kb_label_wrap = 20
)
display(assessment_mre_bbn['kb_dot'])
_=print_summary(assessment_mre_bbn)

#### Bayesian Taxonomy

In [ ]:
assessment_bayesian_taxonomy =  await assess_proposition2(
    proposition = 'A taxonomy of abstracted concepts ties together all of the [CONCEPTS] into a cohesive science of Bayesian reasoning under uncertainty.', 
    short_name = 'bayesian_taxonomy', 
    search_cache = search_cache, 
    added_instructions = """
Derive a query plan that results in such a Conceptual Taxonomy for the [CONCEPTS] given below.
All Query nodes must lead directly or indirectly back to the main proposition (the 'sink' node, which is the only node upon which no other node depends). 
The query graph must have one and only one "sink" node capturing the main proposition that generates the ultimate conclusions.

Consider the work in the context of 
(a) Explanation - given evidence, find hypotheses that explain it - and 
(b) Argumentation - given hypothesis, find evidence that supports and refute it.

Investigate alternative explanations, i.e., hypotheses, for each subproposition.
Investigate evidence for and evidence against each subproposition.

Explore the following [CONCEPTS], being sure to cite the literature references that explain them:
==================================
[CONCEPTS]

* Bayesian blocks, e.g., Scargle, with dynamic programming
* Bayesian optimal stopping rules with dynamic programming
* Bayesian updating in Kalman filters with dynamic programming
* Bayesian updating in robotics SLAM (Simultaneous Localization and Mapping) with particle filters
* Bayesian optimization for optimal experimental design & search under uncertainty
* Bayesian basis of Good-Turing estimation
* Bayesian Weight of Evidence in hypothesis confirmation, e.g., I.J. Good, and connection to mutual information by expectation (averaging)
* Bayesian hypothesis confirmation theory, e.g., Branden Fitelsen
* Bayesian reasoning under uncertainty
* Bayesian decision-making under uncertainty

==================================

""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 30,
    print_level = 2
)
display(assessment_bayesian_taxonomy['kb_dot'])
_=print_summary(assessment_bayesian_taxonomy)

In [ ]:
context_dict = ({"responses":[{"task_id":"bayesian_blocks","query":"Explore Bayesian blocks as an example of dynamic programming and cite literature references.\n","response":"Bayesian blocks is a method for creating histograms with variable-width bins, which is particularly useful when dealing with data that has varying event rates. It is a non-parametric technique that can adaptively determine the optimal binning of data by maximizing a fitness function that is based on a Bayesian prior and likelihood. The method is often used in the analysis of astronomical data, but it can be applied to any type of data that is amenable to histogram representation.\n\nThe connection to dynamic programming comes from the algorithm's approach to optimizing the binning. Dynamic programming is a method for solving complex problems by breaking them down into simpler subproblems, solving each of these subproblems just once, and storing their solutions - ideally, using a recursive relation. The key feature of dynamic programming is the use of these stored solutions to solve larger subproblems, which avoids the need to recompute the answers to the subproblems multiple times.\n\nIn the context of Bayesian blocks, dynamic programming is used to efficiently find the optimal segmentation of the data into blocks (bins). The algorithm iteratively evaluates the fitness of different possible partitions of the data, storing the best solution found for each segment. By building on these solutions, the algorithm can efficiently find the best overall partitioning of the data into blocks.\n\nFor literature references, here are some sources that discuss Bayesian blocks and dynamic programming:\n\n1. Scargle, J. D., Norris, J. P., Jackson, B., & Chiang, J. (2013). Studies in Astronomical Time Series Analysis. VI. Bayesian Block Representations. The Astrophysical Journal, 764(2), 167. [IOPscience](https://iopscience.iop.org/article/10.1088/0004-637X/764/2/167)\n   - This paper presents a detailed description of the Bayesian blocks algorithm and its application to astronomical time series analysis.\n\n2. VanderPlas, J. T. (2012). Dynamic Programming in Python: Bayesian Blocks. [Jake VanderPlas's blog](https://jakevdp.github.io/blog/2012/09/12/dynamic-programming-in-python/)\n   - A blog post that explains how dynamic programming is used in the implementation of the Bayesian blocks algorithm in Python.\n\n3. AstroML documentation. Bayesian Blocks for Histograms. [astroML 1.0 documentation](https://www.astroml.org/examples/algorithms/plot_bayesian_blocks.html)\n   - The documentation for the astroML library, which includes an example of using Bayesian blocks for histogramming.\n\n4. Astropy documentation. bayesian_blocks — Astropy v6.0.0. [Astropy stats documentation](https://docs.astropy.org/en/stable/api/astropy.stats.bayesian_blocks.html)\n   - The documentation for the Astropy library, which provides an implementation of the Bayesian blocks algorithm.\n\n5. Singh, S. P. N., & Moore, A. W. (2005). Bayesian Real-Time Dynamic Programming. In Proceedings of the 19th International Joint Conference on Artificial Intelligence (IJCAI'05), 3-9. [IJCAI Proceedings](https://www.ijcai.org/Proceedings/09/Papers/297.pdf)\n   - This paper discusses real-time dynamic programming in the context of Markov decision processes, which is related to the concept of dynamic programming used in Bayesian blocks.\n\nThese references provide a starting point for understanding the Bayesian blocks algorithm and its relationship to dynamic programming. The algorithm's efficiency and adaptability make it a powerful tool for data analysis in various fields, particularly when the underlying data structure is complex or unevenly distributed."},{"task_id":"bayesian_hypothesis_confirmation","query":"Explore Bayesian hypothesis confirmation theory, e.g., Branden Fitelsen and cite literature references.\n","response":"Bayesian confirmation theory is a probabilistic approach to the philosophy of science that deals with how evidence confirms or supports scientific hypotheses. According to this theory, evidence E confirms a hypothesis H if the probability of H given E is higher than the probability of H by itself. This is often expressed using Bayes' theorem, which relates the conditional and marginal probabilities of events.\n\nBranden Fitelson is a prominent philosopher who has contributed significantly to the study of Bayesian confirmation theory. His work includes a detailed analysis of how Bayesian principles can be applied to the confirmation of hypotheses and the resolution of paradoxes in the philosophy of science.\n\nHere are some key references to Fitelson's work on Bayesian confirmation theory:\n\n1. Fitelson, B. (2001). \"Studies in Bayesian Confirmation Theory.\" PhD thesis, University of Wisconsin--Madison. This thesis is a comprehensive study of Bayesian confirmation theory, discussing various measures of confirmation and addressing issues such as the paradox of the ravens and the problem of old evidence. [Link](https://fitelson.org/thesis.pdf)\n\n2. Fitelson, B. (2006). \"How Bayesian Confirmation Theory Handles the Paradox of the Ravens.\" Philosophy Compass, 1(1), 95–113. In this paper, Fitelson discusses the famous paradox of the ravens and how Bayesian confirmation theory can provide a resolution. [Link](https://philpapers.org/rec/HAWHBC)\n\n3. Fitelson, B., & Waterman, A. (2005). \"Bayesian Confirmation and Auxiliary Hypotheses Revisited.\" Journal of Philosophy, 102(7), 298-315. This paper revisits the problem of auxiliary hypotheses in the context of Bayesian confirmation theory. [Link](https://www.jstor.org/stable/3541703)\n\n4. Fitelson, B. (1999). \"The Plurality of Bayesian Measures of Confirmation and the Problem of Measure Sensitivity.\" Philosophy of Science, 66(S1), S362-S378. This paper discusses the issue of having multiple non-equivalent Bayesian measures of confirmation and the problem this poses for the theory. [Link](https://www.jstor.org/stable/188784)\n\n5. Fitelson, B. (2001). \"Studies in Bayesian Confirmation Theory.\" This book provides an in-depth look at Bayesian confirmation theory, including discussions on various measures of confirmation and their implications. [Link](https://books.google.com/books/about/Studies_in_Bayesian_Confirmation_Theory.html?id=M64IAQAAMAAJ)\n\nThese references provide a solid foundation for understanding Fitelson's contributions to Bayesian confirmation theory. His work has been influential in clarifying how evidence can incrementally confirm hypotheses and in addressing various philosophical problems through a Bayesian lens."},{"task_id":"bayesian_optimization","query":"Explore Bayesian Optimization of expensive, uncertain objective functions as applied to experimental design and cite literature references.\n","response":"Bayesian Optimization (BO) is a powerful strategy for the optimization of expensive and uncertain objective functions, which is particularly useful in experimental design where each experiment can be time-consuming and costly. The goal of BO is to find the maximum (or minimum) of an objective function efficiently, which means with as few evaluations as possible. This is achieved by building a probabilistic model of the objective function and using it to make decisions about where to sample next.\n\nHere's a step-by-step logical reasoning on how Bayesian Optimization can be applied to experimental design, along with literature references:\n\n1. **Define the Objective Function**: In experimental design, the objective function typically represents the outcome of an experiment that depends on certain input parameters. The function is often unknown or partially known, and direct evaluations are expensive (e.g., time, resources).\n\n2. **Choose a Prior**: Bayesian Optimization starts with a prior over the objective function, which captures beliefs about the behavior of the function before any data is collected. Gaussian Processes (GPs) are commonly used as priors due to their flexibility and tractability (Rasmussen & Williams, 2006, \"Gaussian Processes for Machine Learning\").\n\n3. **Acquire Data**: Initial data is collected, often through a small number of random experiments, to update the prior and create a posterior distribution.\n\n4. **Update the Model**: The GP model is updated with the new data, which involves computing the posterior distribution of the function's values at new points given the observed data.\n\n5. **Select Next Sample Point**: A new sample point is chosen by optimizing an acquisition function, which balances exploration (sampling where the model is uncertain) and exploitation (sampling where the model predicts high values). Common acquisition functions include Expected Improvement, Probability of Improvement, and Upper Confidence Bound.\n\n6. **Iterate**: Steps 3-5 are repeated, each time updating the model with the new data, until a stopping criterion is met (e.g., a maximum number of iterations, convergence of the objective function value, or exhaustion of resources).\n\n7. **Make Decisions**: The results of Bayesian Optimization can inform decisions about the experimental design, such as the optimal settings of the experimental parameters.\n\nLiterature references that discuss Bayesian Optimization in the context of experimental design include:\n\n- Shahriari, B., Swersky, K., Wang, Z., Adams, R. P., & de Freitas, N. (2016). \"Taking the human out of the loop: A review of Bayesian optimization.\" Proceedings of the IEEE, 104(1), 148-175. This paper provides a comprehensive review of Bayesian optimization, including its application to experimental design.\n\n- Frazier, P. I. (2018). \"A tutorial on Bayesian optimization.\" arXiv preprint arXiv:1807.02811. This tutorial covers the basics of Bayesian optimization and includes discussion on experimental design.\n\n- Snoek, J., Larochelle, H., & Adams, R. P. (2012). \"Practical Bayesian optimization of machine learning algorithms.\" Advances in neural information processing systems, 25. While focused on machine learning, this paper provides insights into the practical application of Bayesian optimization that can be applied to experimental design.\n\n- Garnett, R., Osborne, M. A., & Roberts, S. J. (2010). \"Bayesian optimization for sensor set selection.\" Proceedings of the 9th ACM/IEEE International Conference on Information Processing in Sensor Networks. This paper discusses the use of Bayesian optimization for selecting sensor sets, which is a form of experimental design.\n\nThe literature references provided in the context above also offer valuable insights into the application of Bayesian Optimization to experimental design:\n\n- \"Bayesian Optimization for Adaptive Experimental Design: A Review\" (https://ieeexplore.ieee.org/document/8957442) provides a review of Bayesian optimization as a statistical method for modeling and optimizing expensive black-box functions, which is relevant to experimental design.\n\n- The arXiv preprint \"Bayesian Optimisation for Sequential Experimental Design\" (https://arxiv.org/pdf/2107.12809) discusses the approach to globally optimizing black-box objective functions that are expensive to evaluate, which is a key challenge in experimental design.\n\n- \"Bayesian Experimental Design: A Review\" (http://homepage.divms.uiowa.edu/~gwoodwor/AdvancedDesign/Chaloner%20Verdinelli.pdf) reviews the literature on Bayesian experimental design, providing a unified view based on a decision-theoretic approach.\n\n- The Wikipedia page on \"Bayesian experimental design\" (https://en.wikipedia.org/wiki/Bayesian_experimental_design) provides a general overview of the framework and its theoretical basis.\n\nThese references collectively provide a solid foundation for understanding the application of Bayesian Optimization to experimental design, highlighting its advantages in efficiently navigating the trade-off between exploration and exploitation to optimize experimental outcomes."},{"task_id":"good_turing_estimation","query":"Explore Good-Turing estimation with respect to derivation from Bayesian principles and cite literature references.\n","response":"The Good-Turing estimation is a statistical technique originally developed by I.J. Good and Alan Turing during World War II for cryptanalysis purposes. It has since been widely used in various fields, such as ecology, linguistics, and information theory, for estimating the probability of encountering an unseen event given a set of observed data. The method is particularly useful when dealing with sparse data.\n\nThe Good-Turing estimator adjusts the observed frequencies of events to better estimate the true frequencies, including those events that have not been observed (i.e., have zero frequency in the sample). The key idea is to reassign some of the probability mass from observed to unobserved events.\n\nFrom a Bayesian perspective, the Good-Turing estimation can be derived by considering a nonparametric Bayesian model. In such a model, one assumes a prior distribution over the possible frequencies of different events, and then updates this distribution in light of the observed data. The Good-Turing estimate can be seen as an approximation to the Bayesian posterior predictive distribution of the frequency of an unobserved event.\n\nThe literature references provided in the context suggest that there has been work on rediscovering Good-Turing estimators through Bayesian nonparametrics. For example, the paper titled \"Rediscovery of Good–Turing estimators via Bayesian Nonparametrics\" investigates the relationship between the Good-Turing approach and Bayesian nonparametric methods. This paper can be found on JSTOR (https://www.jstor.org/stable/24738771), PubMed (https://pubmed.ncbi.nlm.nih.gov/26224325/), arXiv (https://arxiv.org/abs/1401.0303), and the Oxford Academic website (https://academic.oup.com/biometrics/article/72/1/136/7511697).\n\nThe Bayesian nonparametric approach typically involves using Dirichlet Process or Pitman-Yor Process priors, which allow for an infinite number of possible events (species, words, etc.). The Good-Turing estimator can be derived as a special case or approximation within these Bayesian nonparametric frameworks.\n\nTo summarize, the Good-Turing estimation can be connected to Bayesian principles by considering it as an approximation to the Bayesian posterior predictive distribution for unseen events, derived within a nonparametric Bayesian framework. The literature references provided offer a deeper exploration of this connection and the mathematical derivations involved."},{"task_id":"kalman_filters","query":"Explore Kalman filters and smoothing with respect to derivation from Bayesian principles and as an example of dynamic programming and cite literature references.\n","response":"Kalman filters are a class of filters that are used for estimating the state of a dynamic system over time. They are based on Bayesian principles and can be seen as an example of dynamic programming. Here's a step-by-step logical reasoning that connects Kalman filters to Bayesian principles and dynamic programming, along with literature references:\n\n1. **Bayesian Principles:**\n   - Kalman filters operate on the Bayesian principle of recursive estimation. At each time step, the filter takes a prediction of the system's state and updates it with new measurement data to produce a posterior estimate.\n   - The prediction step is based on a prior model of the system's evolution, while the update step incorporates new data using Bayes' theorem to revise the belief about the system's state.\n   - The Kalman filter assumes that both the system dynamics and the measurements are subject to Gaussian noise, which allows for closed-form expressions for the prediction and update steps.\n\n   **Literature Reference:**\n   - \"Kalman filter demystified: from intuition to probabilistic graphical models\" (https://arxiv.org/pdf/1811.11618) provides an intuitive explanation of the Kalman filter from a Bayesian perspective.\n\n2. **Dynamic Programming:**\n   - Dynamic programming is a method for solving complex problems by breaking them down into simpler subproblems. It is applicable to problems exhibiting the properties of overlapping subproblems and optimal substructure.\n   - The Kalman filter can be seen as an example of dynamic programming because it solves the filtering problem in a recursive manner. Each step of the filter computes an optimal estimate of the current state based on the previous estimate and the latest measurement, which is analogous to the principle of optimality in dynamic programming.\n\n   **Literature Reference:**\n   - \"The Kalman Filter and Related Algorithms: A Literature Review\" (https://www.researchgate.net/publication/236897001_The_Kalman_Filter_and_Related_Algorithms_A_Literature_Review) discusses the Kalman filter in the context of various algorithms, which can be related to dynamic programming.\n\n3. **Kalman Smoothing:**\n   - Kalman smoothing refers to the process of improving the state estimates obtained by a Kalman filter by incorporating future measurements. This is typically done in a batch process, where the entire sequence of measurements is processed to refine the state estimates.\n   - Smoothing can also be derived from Bayesian principles, as it involves computing the posterior distribution of the state over the entire time series of measurements.\n\n   **Literature Reference:**\n   - \"BAYESIAN FILTERING AND SMOOTHING\" (https://users.aalto.fi/~ssarkka/pub/cup_book_online_20131111.pdf) provides an introduction to non-linear Kalman filtering and smoothing from a Bayesian perspective.\n\n4. **Further Reading:**\n   - For a comprehensive understanding of the Kalman filter's derivation from Bayesian principles, \"Advanced Kalman Filtering Approaches to Bayesian State Estimation\" (https://www.diva-portal.org/smash/get/diva2:1077486/FULLTEXT01.pdf) can be a valuable resource.\n   - The Wikipedia page on Kalman filters (https://en.wikipedia.org/wiki/Kalman_filter) provides a broad overview and links to various extensions of the Kalman filter, which can be useful for understanding its relationship to Bayesian principles and dynamic programming.\n\nIn summary, Kalman filters are deeply rooted in Bayesian principles, using prior knowledge and observed data to make inferences about the state of a system. The recursive nature of the Kalman filter aligns with the principles of dynamic programming, where each step optimizes the state estimate based on the previous step and new information. The literature provided offers a detailed exploration of these concepts."},{"task_id":"optimal_stopping_rules","query":"Explore optimal stopping rules with respect to derivation from Bayesian principles and as an example of dynamic programming and cite literature references.\n","response":"Optimal stopping rules are a fundamental concept in decision theory and statistics, particularly in the context of sequential analysis and dynamic programming. The goal of an optimal stopping rule is to determine the best time to take a particular action to maximize an expected payoff or to minimize an expected cost, based on sequentially observed random variables.\n\nFrom a Bayesian perspective, optimal stopping involves updating beliefs about an uncertain process as new information becomes available and making a decision about when to stop based on these updated beliefs. Bayesian principles provide a framework for incorporating prior knowledge and evidence into the decision-making process.\n\nDynamic programming, on the other hand, is a method for solving complex problems by breaking them down into simpler subproblems. It is particularly useful in the context of optimal stopping because it allows for the problem to be solved recursively, considering the value of stopping at each point in time versus the expected value of continuing and making a decision later.\n\nLet's explore some literature references that discuss optimal stopping rules, Bayesian principles, and dynamic programming:\n\n1. \"Optimal Stopping and Applications\" by Thomas S. Ferguson (https://www.math.ucla.edu/~tom/Stopping/sr1.pdf) provides a comprehensive overview of the theory of optimal stopping, including its applications in various fields. The document discusses the connection between stopping rule problems and sequential statistical analysis as developed by Wald.\n\n2. \"Sequential Analysis and Optimal Stopping\" available on JSTOR (https://www.jstor.org/stable/40749860) discusses the development of dynamic programming in conjunction with the theory of sequential analysis, which is closely related to optimal stopping problems.\n\n3. The Wikipedia page on \"Optimal Stopping\" (https://en.wikipedia.org/wiki/Optimal_stopping) provides a general overview of the theory, including its mathematical foundations and applications.\n\n4. \"Optimal Stopping Rules\" on ResearchGate (https://www.researchgate.net/publication/304179768_Optimal_Stopping_Rules) discusses the problem of choosing a time to take a given action based on sequentially observed random variables in the context of optimal control literature.\n\n5. \"Optimal Stopping Methods for Investment Decisions: A Literature Review\" published on MDPI (https://www.mdpi.com/2227-7072/10/4/96) reviews the use of optimal stopping methods in investment decisions, describing the stochastic models used to represent stock market trends.\n\n6. \"Dynamic Programming Principles for Optimal Stopping Problems\" on arXiv (https://arxiv.org/abs/1708.02192) analyzes an optimal stopping problem with a constraint on the expected cost, highlighting the use of dynamic programming principles.\n\n7. \"Optimal Stopping with Multiple Priors\" on JSTOR (https://www.jstor.org/stable/40263845) discusses the classical theory of optimal stopping and its reliance on martingale theory, which is a probabilistic model of a fair game against nature.\n\n8. \"On optimal stopping rules\" in the journal \"Probability Theory and Related Fields\" (https://link.springer.com/article/10.1007/BF00535296) discusses the Bayesian and minimax solutions of sequential decision problems, which are relevant to optimal stopping.\n\n9. \"Approximate Dynamic Programming With Correlated Bayesian Beliefs\" (https://optimallearning.princeton.edu/Papers/RyzhovPowell-kgdp_AllertonconferenceJune142010.pdf) discusses the representation of uncertainty about the value function using a Bayesian model in the context of approximate dynamic programming.\n\n10. \"Optimal stopping, incomplete information, and stochastic games\" (https://uu.diva-portal.org/smash/get/diva2:1777701/FULLTEXT01.pdf) extends the classical Bayesian sequential decision framework to include optimal stopping and stochastic games.\n\nThese references provide a solid foundation for understanding the principles and applications of optimal stopping rules, as well as their derivation from Bayesian principles and their implementation using dynamic programming techniques."},{"task_id":"robotics_slam","query":"Explore Robotics SLAM with particle filters with respect to derivation from Bayesian principles and cite literature references.\n","response":"Simultaneous Localization and Mapping (SLAM) is a critical problem in robotics, where a robot aims to build a map of an unknown environment while simultaneously determining its location within that map. Particle filters, also known as Sequential Monte Carlo methods, are a popular choice for solving the SLAM problem due to their ability to handle non-linear models and non-Gaussian noise.\n\nThe particle filter approach to SLAM is derived from Bayesian principles, which provide a probabilistic framework for state estimation. In Bayesian terms, the goal is to compute the posterior distribution of the robot's path and the map given all the available data (sensor measurements and control inputs). The particle filter approximates this posterior distribution using a set of particles, where each particle represents a hypothesis of the robot's trajectory and the map.\n\nHere's a step-by-step logical reasoning of how particle filters are derived from Bayesian principles in the context of SLAM:\n\n1. **Bayesian Filtering**: The foundation of particle filters lies in Bayesian filtering, which updates the belief about the state of a system as new data becomes available. The belief is represented as a probability distribution over possible states.\n\n2. **Recursive Estimation**: Bayesian filtering is recursive and consists of two main steps: prediction and update. The prediction step uses the system model to propagate the belief over time, while the update step incorporates new measurements to refine the belief.\n\n3. **Non-parametric Representation**: Particle filters use a non-parametric representation of the probability distribution, which means they do not assume any specific form (like Gaussian) for the distribution. Instead, they represent the distribution as a set of samples (particles).\n\n4. **Importance Sampling**: Particle filters employ importance sampling to approximate the posterior distribution. Each particle is drawn from a proposal distribution and is assigned a weight based on how well it explains the observed data.\n\n5. **Resampling**: To avoid the problem of degeneracy, where all but a few particles have negligible weight, particle filters periodically perform resampling. This process generates a new set of particles by sampling with replacement from the current set, biased towards particles with higher weights.\n\n6. **Rao-Blackwellization**: In the context of SLAM, particle filters can be combined with Rao-Blackwellization to reduce variance and computational cost. This technique analytically computes the conditional distribution of the map given the trajectory, which is represented by the particles.\n\nThe literature provides extensive coverage of these principles and their application to SLAM:\n\n- Thrun, S. (2002). Particle Filters in Robotics. In Proceedings of the Eighteenth Conference on Uncertainty in Artificial Intelligence (UAI2002). Retrieved from http://robots.stanford.edu/papers/thrun.pf-in-robotics-uai02.pdf\n- Doucet, A., de Freitas, N., & Gordon, N. (2001). An Introduction to Sequential Monte Carlo Methods. In Sequential Monte Carlo Methods in Practice. Retrieved from https://www.stats.ox.ac.uk/~doucet/doucet_defreitas_gordon_SMCbookintro.pdf\n- Montemerlo, M., Thrun, S., Koller, D., & Wegbreit, B. (2002). FastSLAM: A Factored Solution to the Simultaneous Localization and Mapping Problem. In AAAI/IAAI. Retrieved from http://robots.stanford.edu/papers/Montemerlo02a.pdf\n- Arulampalam, M. S., Maskell, S., Gordon, N., & Clapp, T. (2002). A tutorial on particle filters for online nonlinear/non-Gaussian Bayesian tracking. IEEE Transactions on Signal Processing, 50(2), 174-188. Retrieved from http://liu.diva-portal.org/smash/get/diva2:360880/FULLTEXT01\n\nThese references provide a comprehensive understanding of how particle filters are derived from Bayesian principles and their application in the SLAM problem within robotics."},{"task_id":"weight_of_evidence","query":"Explore Weight of Evidence in hypothesis confirmation, e.g., I.J. Good, and connection to mutual information by expectation (averaging) and cite literature references.\n","response":"The concept of \"Weight of Evidence\" (WoE) in hypothesis confirmation is a measure of how much a piece of evidence supports or contradicts a hypothesis. It was extensively discussed by the British statistician I.J. Good, who contributed significantly to Bayesian statistics and the philosophy of probability.\n\nIn Good's framework, the weight of evidence is often quantified as the logarithm of the likelihood ratio. If we have two hypotheses, H1 and H2, and evidence E, the weight of evidence provided by E in favor of H1 over H2 is given by:\n\nWoE = log(P(E|H1)/P(E|H2))\n\nwhere P(E|H1) is the probability of observing the evidence E given that hypothesis H1 is true, and P(E|H2) is the probability of observing E given that H2 is true.\n\nThis concept is closely related to mutual information, which in information theory is a measure of the amount of information that one random variable contains about another random variable. Mutual information can be seen as an expectation (average) of the weight of evidence that an observation provides about the state of the world.\n\nThe connection between weight of evidence and mutual information can be understood through the lens of Bayesian inference. Mutual information quantifies the reduction in uncertainty (or increase in certainty) about one random variable due to the knowledge of another. In the context of hypothesis testing, mutual information can be seen as the expected weight of evidence across all possible observations.\n\nFor literature references, you can look at the following sources:\n\n1. Good, I. J. (1960). \"Weight of evidence, corroboration, explanatory power, information and the utility of experiments\". Journal of the Royal Statistical Society. Series B (Methodological), 22(3), 319-331. This paper discusses the concept of weight of evidence in the context of hypothesis confirmation and its relation to information theory.\n\n2. Good, I. J. (1950). \"Probability and the Weighing of Evidence\". Charles Griffin, London. This book by Good provides a comprehensive look at the use of probability in evaluating evidence and making decisions.\n\n3. Cover, T. M., & Thomas, J. A. (2006). \"Elements of Information Theory\" (2nd ed.). Wiley-Interscience. This textbook includes a discussion of mutual information and its properties, which are relevant to understanding the connection to weight of evidence.\n\n4. Kullback, S., & Leibler, R. A. (1951). \"On Information and Sufficiency\". The Annals of Mathematical Statistics, 22(1), 79-86. This paper introduces the Kullback-Leibler divergence, which is closely related to the concept of weight of evidence.\n\n5. MacKay, D. J. C. (2003). \"Information Theory, Inference, and Learning Algorithms\". Cambridge University Press. This book provides an introduction to information theory with a focus on inference and learning algorithms, which are relevant to the concept of weight of evidence.\n\nThese references should provide a solid foundation for understanding the weight of evidence in hypothesis confirmation and its connection to mutual information."}]})
print(sum([len(cq['response']) for cq in context_dict['responses']]))
print([len(cq['response'].split(' ')) > 850/1.75 for cq in context_dict['responses']])

cqs = [ComputeQuery(**cq_dict) for cq_dict in context_dict['responses']]
total_chars = sum([ len(cq.response) for cq in cqs ])
if total_chars > 20000:
    cqs_truncated = []
    for cq in cqs:
        response_split = cq.response.split(' ')
        if len(response_split) > 485: # If too long, truncate response
            print(f"\t\t... truncating id={cq.task_id} ...")
            response_split = response_split[:485] + ['...']
        cq.response = ' '.join(response_split)
        cqs_truncated.append(cq)
    cqs = cqs_truncated
total_chars_new = sum([ len(cq.response) for cq in cqs ])
print({'total_chars': total_chars,'total_chars_new': total_chars_new})

In [ ]:
assessment_bayesian_taxonomy_new =  await assess_proposition2(
    proposition = 'A Conceptual Taxonomy of abstracted concepts ties together all of the [CONCEPTS] into a cohesive field of scientific inquiry.', 
    short_name = 'bayesian_taxonomy_new', 
    search_cache = search_cache, 
    added_instructions = """
Derive a query plan that results in a Conceptual Taxonomy for the [CONCEPTS] given below.
Be sure to leverage the `search_query` field of `Query` nodes by assigning `search_query` to a list that has a query string for each of the [CONCEPTS].

Explore the following [CONCEPTS], being sure to cite the literature references that explain them:

==================================

[CONCEPTS]

* Bayesian blocks, e.g., Scargle, (explored as an example of dynamic programming)
* Optimal stopping rules (explored with respect to derivation from Bayesian principles and as an example of dynamic programming)
* Kalman filters and smoothing (explored with respect to derivation from Bayesian principles and as an example of dynamic programming)
* Robotics SLAM (Simultaneous Localization and Mapping) with particle filters (explored with respect to derivation from Bayesian principles)
* Bayesian Optimization of expensive, uncertain objective functions as applied to experimental design
* Good-Turing estimation (explored with respect to derivation from Bayesian principles)
* Bayesian hypothesis confirmation theory, e.g., Branden Fitelsen
* Weight of Evidence in hypothesis confirmation, e.g., I.J. Good, and connection to mutual information by expectation (averaging)

==================================

""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    force_run = False,
    report_to_file = True,
    kb_label_wrap = 30,
    print_level = 2
)
display(assessment_bayesian_taxonomy_new['kb_dot'])
if assessment_bayesian_taxonomy_new.get('plan_result'):
    _=print_summary(assessment_bayesian_taxonomy_new)

In [ ]:
assessment_bayesian_taxonomy_improve =  await assess_proposition2(
    proposition = 'A Conceptual Taxonomy of abstracted concepts ties together all of the [CONCEPTS] into a cohesive field of scientific inquiry.', 
    short_name = 'bayesian_taxonomy_improve', 
    improvement_round = True,
    search_cache = search_cache, 
    added_instructions = """
Derive a query plan that results in a Conceptual Taxonomy for the [CONCEPTS] given below.
Be sure to leverage the `search_query` field of `Query` nodes by assigning `search_query` to a list that has a query string for each of the [CONCEPTS].

Explore the following [CONCEPTS], being sure to cite the literature references that explain them:

==================================

[CONCEPTS]

* Bayesian blocks, e.g., Scargle, (explored as an example of dynamic programming)
* Optimal stopping rules (explored with respect to derivation from Bayesian principles and as an example of dynamic programming)
* Kalman filters and smoothing (explored with respect to derivation from Bayesian principles and as an example of dynamic programming)
* Robotics SLAM (Simultaneous Localization and Mapping) with particle filters (explored with respect to derivation from Bayesian principles)
* Bayesian Optimization of expensive, uncertain objective functions as applied to experimental design
* Good-Turing estimation (explored with respect to derivation from Bayesian principles)
* Bayesian hypothesis confirmation theory, e.g., Branden Fitelsen
* Weight of Evidence in hypothesis confirmation, e.g., I.J. Good, and connection to mutual information by expectation (averaging)

==================================

""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    force_run = False,
    report_to_file = True,
    kb_label_wrap = 30,
    print_level = 2
)
display(assessment_bayesian_taxonomy_improve['kb_dot'])
if assessment_bayesian_taxonomy_improve.get('plan_result'):
    _=print_summary(assessment_bayesian_taxonomy_improve)

#### Explain I.J. Good's Weight of Evidence

In [ ]:
assessment_explain_wt_evidence =  await assess_proposition2(
    proposition =  "Explain I.J. Good's Weight of Evidence for Hypothesis Confirmation.", 
    short_name = 'explain_wt_evidence', 
    search_cache = search_cache, 
    added_instructions = """
Do so in a manner that a 7th-grader can understand.
Provide many examples in a clear and concise manner.

Ensure that all Query nodes lead directly or indirectly back to the main proposition. 
So, the query graph has a single "sink" node capturing the main proposition that generates the ultimate conclusions.

Be sure to leverage internet searches while investigating the following:
* Definition of I.J. Good's Weight of Evidence.
* The connection of Good's Weight of Evidence to other concepts: Bayes Factor, Likelihood Ratio, Kullback-Leibler Divergence, etc.
* Related issues like Branden Fitelsen's work on Hypothesis Confirmation Theory.
* Applications like Yuan et al. "Most Relevant Explanation in Bayesian Networks".
* Alternative approaches to estimating the strength of evidence with respect to a hypothesis.
""", 
    client = client,
    model = 'gpt-3.5-turbo', #"gpt-4-turbo-preview", 
    #use_plan = assessment_explain_wt_evidence['plan'],
    dynamic = True,
    force_run = True,
    report_to_file = True,
    kb_label_wrap = 20,
    print_level = 2
)
display(assessment_explain_wt_evidence['kb_dot'])
_=print_summary(assessment_explain_wt_evidence)

In [ ]:
assessment_explain_wt_evidence_gpt4 =  await assess_proposition2(
    proposition =  "Explain I.J. Good's Weight of Evidence.", 
    short_name = 'explain_wt_evidence_gpt4', 
    search_cache = search_cache, 
    added_instructions = """
Do so in a manner that a 7th-grader can understand.
Provide many examples in a clear and concise manner.

Be sure to leverage internet searches while investigating the following:

* Definition of I.J. Good's Weight of Evidence.
* The connection of Good's Weight of Evidence to other concepts: Bayes Factor, Likelihood Ratio, Kullback-Leibler Divergence, etc.
* Related issues like Branden Fitelsen's work on Hypothesis Confirmation Theory.
* Applications like Yuan et al. "Most Relevant Explanation in Bayesian Networks".
* Alternative approaches to estimating the strength of evidence with respect to a hypothesis.
""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    #use_plan = assessment_explain_wt_evidence['plan'],
    force_run = False,
    report_to_file = True,
    kb_label_wrap = 24,
    print_level = 2
)
display(visualize_knowledge_graph(convert_to_graph(assessment_explain_wt_evidence_gpt4['plan0'],proposition='copilotkt_gamechanger')))
display(assessment_explain_wt_evidence_gpt4['kb_dot'])
if assessment_explain_wt_evidence_gpt4.get('plan_result'):
    _ = print_summary(assessment_explain_wt_evidence_gpt4)

In [ ]:
assessment_explain_wt_evidence_gpt4['plan']

In [ ]:
rprt = generate_report(assessment_explain_wt_evidence_gpt4['plan'].query_plan_outline,list(assessment_explain_wt_evidence_gpt4['plan_result'].values())[-1],assessment_explain_wt_evidence_gpt4['plan_result'],{q.id: q.dependencies for q in assessment_explain_wt_evidence_gpt4['plan'].query_graph},search_cache,'explain_woe', True,False)

In [ ]:
print(rprt)

In [ ]:
outline_processing(assessment_explain_wt_evidence_gpt4['plan'],True)

In [ ]:
assessment_explain_wt_evidence['kb_dot']

#### Explain Real Analysis

In [ ]:
assessment_explain_real_analysis =  await assess_proposition2(
    proposition =  "Explain Real Analysis.", 
    short_name = 'explain_real_analysis', 
    search_cache = search_cache, 
    added_instructions = """
Do so in a manner that a 7th-grader can understand.
Provide many examples in a clear and concise manner.

Ensure that all Query nodes lead directly or indirectly back to the main proposition. 
So, the query graph has a single "sink" node capturing the main proposition that generates the ultimate conclusions.

Investigate textbooks, such as 'Baby Rudin'.
Investigate alternative explanations for each subproposition.
Investigate evidence for and evidence against each subproposition.

""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    dynamic = True,
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_explain_real_analysis['kb_dot']

#### Explain Hilbert Spaces

In [ ]:
assessment_hilbert_spaces =  await assess_proposition2(
    proposition =  "Explain Hilbert spaces.", 
    short_name = 'hilbert_spaces', 
    search_cache = search_cache, 
    added_instructions = """
Do so in a manner that a 7th-grader can understand.
Provide many examples in a clear and concise manner.

Ensure that all Query nodes lead directly or indirectly back to the main proposition. 
So, the query graph has a single "sink" node capturing the main proposition that generates the ultimate conclusions.

Investigate alternative explanations for each subproposition.
Investigate evidence for and evidence against each subproposition.

""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    dynamic = True,
    report_to_file = True,
    kb_label_wrap = 20
)


In [ ]:
assessment_hilbert_spaces['kb_dot']

#### Comparison -- Gemini Ultra vs GPT-4

In [ ]:
assessment_compare_llm =  await assess_proposition2(
    proposition =  "GPT-4 outperforms Gemini Ultra.", 
    short_name = 'compare_llm', 
    search_cache = search_cache, 
    improvement_round=True,
    force_run=False,
    added_instructions = """
Accept the <<GIVEN FACTS>> as truths that needn't be questioned. DO NOT create Query nodes for these facts.
=========

<<GIVEN FACTS>>:
* OpenAI developed GPT-4.
* Google developed Gemini Ultra.

==========""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20,
    print_level=2
)
display(assessment_compare_llm['kb_dot'])
if assessment_compare_llm.get('plan_result'):
    print_summary(assessment_compare_llm)

#### Gravity & Reality

In [ ]:
assessment_gravity_reality =  await assess_proposition2(
    proposition =  "Gravity, when at the extremes of mass, force, and speed, bends reality in a way such that humans perceive the world as magical phenomena -- incomprehensible in the context of everyday life.", 
    short_name = 'gravity_reality', 
    use_plan = None,
    search_cache = search_cache, 
    added_instructions = """
Ensure that all Query nodes either lead directly or indirectly back to the main proposition. 
So, the query graph has a single "sink" node capturing the main proposition that generates the ultimate conclusions.

Investigate alternative explanations for each subproposition.
Investigate evidence for and evidence against each subproposition.

Accept the <<GIVEN FACTS>> as truths that are not to be questioned. DO NOT create Query nodes for any of the <<GIVEN FACTS>>.

=========

<<GIVEN FACTS>>:
* Einstein's Theory of Relativity.

==========
""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    dynamic = True,
    force_run = False,
    save_pickle = True,
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_gravity_reality['kb_dot']

#### Obesity & Diabetes

In [ ]:
assessment_obesity_diabetes =  await assess_proposition(
    proposition = "The best management of obesity and diabetes -- in terms of being efficacious and likely to be sustained as a long-term lifestyle -- is a low-carb diet and mild exercise.", 
    short_name = 'obesity_diabetes', 
    search_cache = search_cache, 
    added_instructions = '', 
    client = client,
    model = "gpt-4-turbo-preview",
    dynamic = True,
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_obesity_diabetes['kb_dot']

#### Japan in Generative AI

In [ ]:
assessment_japan_ai =  await assess_proposition(
    proposition = "Japan is far behind America, China, and Europe in the race to deploy Generative AI.", 
    short_name = 'japan_ai', 
    search_cache = search_cache, 
    added_instructions = '', 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_japan_ai['kb_dot']

#### Generative AI is different

In [ ]:
assessment_generative_ai_different =  await assess_proposition2(
    proposition =  "Generative AI will have impacts on society that are fundamentally different and more profound than previous technologies because it surpasses human creativity.", 
    short_name = 'generative_ai_different', 
    #use_plan = ai_plan,
    search_cache = search_cache, 
    added_instructions = """
Ensure that all Query nodes lead directly or indirectly back to the main proposition. 
So, the query graph has a single "sink" node capturing the main proposition that generates the ultimate conclusions.

Investigate alternative explanations for each subproposition.
Investigate evidence against as well as for each subproposition.
""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    dynamic = True,
    report_to_file = True,
    kb_label_wrap = 20,
    force_run = False
)

In [ ]:
assessment_generative_ai_different = pickle_it({'short_name' : 'generative_ai_different'}, save = False)

In [ ]:
assessment_generative_ai_different2 =  await assess_proposition2(
    proposition =  "Generative AI will have impacts on society that are fundamentally different and more profound than previous technologies because it surpasses human creativity.", 
    short_name = 'generative_ai_different2', 
    search_cache = search_cache, 
    added_instructions = "", 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 30,
    force_run = False,
    print_level = 2
)
display(assessment_generative_ai_different2['kb_dot'])
if assessment_generative_ai_different2.get('plan_result'):
    _ = print_summary( assessment_generative_ai_different2)

In [ ]:
import re

# Define the pattern to match the context with possible nested occurrences
pattern = r"Context: ##`(.+)`##"

def extract_dictionaries(text):
    # List to hold all dictionary representations
    dicts = []
    
    # Find all matches of the pattern in the text
    matches = re.finditer(pattern, text)
    for match in matches:
        # Extract the dictionary string
        dict_str = match.group(1)
        print(dict_str)
        
        # Check if the dictionary string itself contains nested patterns
        if re.search(pattern, dict_str):
            # If nested patterns exist, recurse and append the result
            print(50*'=')
            dicts.append(extract_dictionaries(dict_str))
        else:
            # If no further nesting, evaluate the string as a dictionary
            dicts.append(eval(dict_str))
    
    return dicts

# Your string here
text = "Context: ##`{'key': 'value', 'nested': 'Context: ##`{\"nested_key\": \"nested_value\"}`##'}`##"

# Extract and print the nested dictionaries
nested_dicts = extract_dictionaries(text)
print(nested_dicts)


## Business

#### El Che Steakhouse, Chicago

In [ ]:
assessment_el_che_chicago =  await assess_proposition2(
    proposition =  "El Che Steakhouse is the best steakhouse in Chicago.", 
    short_name = 'el_che_chicago', 
    improvement_round=True,
    search_cache = search_cache, 
    added_instructions = "Be sure to entertain alternative hypotheses and to obtain evidence for and against the root proposition.", 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 35,
    print_level = 2,
    force_run = True
)
display(assessment_el_che_chicago['kb_dot'])
if assessment_el_che_chicago.get('plan_result'):
    print_summary(assessment_el_che_chicago)

#### Zeiss Holocam

In [ ]:
assessment_zeiss_holocam =  await assess_proposition2(
    proposition =  "Privacy has been forever lost due to Holocam by Zeiss.", 
    short_name = 'zeiss_holocam', 
    search_cache = search_cache, 
    added_instructions = """
Ensure that all Query nodes lead directly or indirectly back to the main proposition. 
So, the query graph has a single "sink" node capturing the main proposition that generates the ultimate conclusions.

Accept the <<GIVEN FACTS>> as truths that needn't be questioned. DO NOT create Query nodes for these facts.
=========

<<GIVEN FACTS>>:
* Zeiss' "Holocam" is a transparent camera powered by holography that turns glass windows into cameras (https://www.digitalcameraworld.com/news/this-holographic-camera-turns-any-window-into-an-invisible-camera).

==========

""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    dynamic = False,
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_zeiss_holocam['kb_dot']

#### Rabbit r1

In [ ]:
assessment_rabbit_r1 =  await assess_proposition2(
    proposition = "The Rabbit r1 is a revolutionary new product.", 
    short_name = 'rabbit_r1', 
    search_cache = search_cache, 
    client = client,
    model = "gpt-4-turbo-preview",
    report_to_file = True,
    kb_label_wrap = 20,
    print_level=2
)

display(assessment_rabbit_r1['kb_dot'])
if assessment_rabbit_r1.get('plan_result'):
    _=print_summary(assessment_rabbit_r1)

#### KUOE Kyoto watches

In [ ]:
assessment_kuoe_watches =  await assess_proposition2(
    proposition =  "KUOE Kyoto is the best brand currently available for field/military automatic watches under $1000", 
    short_name = 'kuoe_watches', 
    use_plan = None,
    search_cache = search_cache, 
    added_instructions = """
Ensure that all Query nodes either lead directly or indirectly back to the main proposition. 
So, the query graph has a single "sink" node capturing the main proposition that generates the ultimate conclusions.

Investigate alternative explanations for each subproposition.
Investigate evidence for and evidence against each subproposition.

Accept the <<GIVEN FACTS>> as truths that needn't be questioned. DO NOT create Query nodes for these given facts.

=========

<<GIVEN FACTS>>:
* KUOE Kyoto is a watch brand headquartered in Kyoto, Japan.
* They make field/military watches.

==========
""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    dynamic = True,
    force_run = False,
    save_pickle = True,
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_kuoe_watches['kb_dot']

In [ ]:
assessment_kuoe_watches_improve =  await assess_proposition2(
    proposition =  "KUOE Kyoto is the best brand currently available for field/military automatic watches under $1000", 
    short_name = 'kuoe_watches_improve',  
    improvement_round = True,
    search_cache = search_cache, 
    added_instructions = """
Ensure that all Query nodes either lead directly or indirectly back to the main proposition. 
So, the query graph has a single "sink" node capturing the main proposition that generates the ultimate conclusions.

Investigate alternative explanations for each subproposition.
Investigate evidence for and evidence against each subproposition.

Accept the <<GIVEN FACTS>> as truths that needn't be questioned. DO NOT create Query nodes for these given facts.

=========

<<GIVEN FACTS>>:
* KUOE Kyoto is a watch brand headquartered in Kyoto, Japan.
* They make field/military watches.

==========
""", 
    client = client,
    model = "gpt-4-turbo-preview",
    force_run = True,
    save_pickle = True,
    report_to_file = True,
    kb_label_wrap = 20,
    print_level = 2
)
display(assessment_kuoe_watches_improve['kb_dot'])
if assessment_kuoe_watches_improve.get('plan_result'):
    _=print_summary(assessment_kuoe_watches_improve)

#### from ChatGPT4

* Prompt: What does 'impeccable record' mean in the context of wealth management services?

* Response:

In the context of wealth management services, an "impeccable record" usually refers to a consistently excellent performance history and a reputation for integrity and reliability. Here's what it typically encompasses:

1. **Performance**: It indicates a history of strong returns on investments or consistent achievement of financial goals tailored to clients' needs. The firm or individual has demonstrated the ability to perform well across different market conditions while managing risks effectively.

2. **Regulatory Compliance**: Maintaining an impeccable record means adhering strictly to all regulatory requirements and industry standards. This includes being transparent in dealings, providing accurate reporting, and engaging in ethical practices.

3. **Client Satisfaction**: High levels of client satisfaction and retention, positive testimonials, and referrals. This implies that the service has successfully met or exceeded client expectations over time.

4. **Expertise and Stability**: Longevity and stability in the industry, along with a team of professionals who are experienced and highly qualified. This suggests a depth of knowledge and a breadth of experience in handling various aspects of wealth management.

5. **Error and Dispute Handling**: A minimal history of errors, disputes, or legal issues, and an effective, fair approach to resolving any that do arise. An impeccable record would indicate that any issues are resolved promptly and satisfactorily.

In summary, an impeccable record in wealth management signifies trustworthiness and a proven track record of success and professionalism. It's a powerful statement of credibility and quality that can be a deciding factor for potential clients looking for wealth management services.

#### Walker Bros. Original Pancake House, Wilmette, IL

In [ ]:
assessment_walker_bros =  await assess_proposition2(
    proposition = "Walker Bros. Original Pancake House is the best pancake place in the entire Chicago metropolitan area.", 
    short_name = 'walker_bros', 
    search_cache = search_cache, 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    force_run = False,
    save_pickle = True
)
display(assessment_walker_bros['kb_dot'])
_=print_summary(assessment_walker_bros)

#### Tesla Model Y

In [ ]:
assessment_tesla_y =  await assess_proposition2(
    proposition = "Tesla Model Y is the best electric vehicle currently available.", 
    short_name = 'tesla_y', 
    search_cache = search_cache, 
    added_instructions = '', 
    client = client,
    model ="gpt-3.5-turbo", 
    report_to_file = False,
    kb_label_wrap = 20
)

In [ ]:
assessment_tesla_y['kb_dot']

In [ ]:
prop = re.sub(r"[^']*('[^\n]+')[^?]+\?\n.+",r"\1",list(assessment_tesla_y['plan_result'].values())[-1].query,flags=re.DOTALL)
mrkdwn_header = f"""---
title: "Proposition: {prop}"
author: "GPT-4, with Michael L. Thompson"
date: "`r Sys.Date()`"
output:
  pdf_document:
    toc: yes
    toc_depth: 4
linkcolor: red
urlcolor: blue
---

```{{r setup, include=FALSE}}
knitr::opts_chunk$set(echo = TRUE)
```
\newpage

"""
filename_stem = assessment_tesla_y['short_name']
mrkdwn_file_path = f'{filename_stem}.Rmd'
with open(mrkdwn_file_path, 'w') as file:
    file.write(mrkdwn_header + to_ascii(assessment_tesla_y['report']))

In [ ]:
print(list(assessment_tesla_y['plan_result'].values())[-1].response)

#### SpaceX vs. NASA

In [ ]:
assessment_spacex_nasa =  await assess_proposition(
    proposition =  "SpaceX is much more innovative than NASA.", 
    short_name = 'spacex_nasa', 
    search_cache = search_cache, 
    added_instructions = '', 
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_spacex_nasa['kb_dot']

#### SpaceX vs. NASA, currently & specificly in launch technology

In [ ]:
assessment_spacex_nasa2 =  await assess_proposition(
    proposition =  "Currently and specificly in launch and space flight technology, SpaceX is much more innovative than NASA.", 
    short_name = 'spacex_nasa2', 
    search_cache = search_cache, 
    added_instructions = '', 
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_spacex_nasa2['kb_dot']

#### Tech startups DEI

In [ ]:
client = instructor.patch(OpenAI())
assessment_startup_dei =  await assess_proposition(
    proposition = "American tech startups have killed DEI in 2023 and forward.", 
    short_name = 'startup_dei', 
    search_cache = search_cache, 
    added_instructions = 'Ignore gains in DEI prior to 2023. The proposition is about 2023 and the future.', 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20,
    force_run = True
)

In [ ]:
assessment_startup_dei['kb_dot']

In [ ]:
client = instructor.patch(OpenAI())
assessment_startup_dei2 =  await assess_proposition2(
    proposition = "American tech startups have killed DEI in 2023 and forward.", 
    short_name = 'startup_dei2', 
    search_cache = search_cache, 
    added_instructions = 'Ignore gains in DEI prior to 2023. The proposition is about 2023 and the future.', 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20,
    force_run = True
)

In [ ]:
assessment_startup_dei2['kb_dot']

In [ ]:
assessment_hamilton_goat['kb_dot']

## Politics/Issues

#### Fairness of current Supreme Court

In [ ]:
assessment_scotus_fairness =  await assess_proposition2(
    proposition =  "The current Supreme Court of the United States will deliver a fair ruling on Donald Trump's eligibility for the 2024 Presidential election.", 
    short_name = 'scotus_fairness', 
    improvement_round=True,
    search_cache = search_cache, 
    added_instructions = "", 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 25,
    force_run = False,
    print_level = 2
)
display(assessment_scotus_fairness['kb_dot'])
if assessment_scotus_fairness.get('plan_result'):
    print_summary(assessment_scotus_fairness)

In [ ]:
assessment_scotus_fairness2 =  await assess_proposition2(
    proposition =  "The current Supreme Court of the United States will deliver a fair ruling on Donald Trump's eligibility for the 2024 Presidential election.", 
    short_name = 'scotus_fairness2', 
    improvement_round=True,
    search_cache = search_cache, 
    added_instructions = """Especially, consider the following issues:

* Recent ethics challenges of, monetary influence on, and failures to recuse themselves by the current U.S. Supreme Court justices.
* Recent ideologically-based rulings of the current U.S. Supreme Court justices.
* Partisan indebtedness of and political pressure on the current U.S. Supreme Court Justices.
""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 25,
    force_run = False,
    print_level = 2
)
display(assessment_scotus_fairness2['kb_dot'])
if assessment_scotus_fairness2.get('plan_result'):
    print_summary(assessment_scotus_fairness2)

In [ ]:
assessment_scotus_fairness3 =  await assess_proposition2(
    proposition =  "The current Supreme Court of the United States will deliver a fair ruling on Donald Trump's eligibility for the 2024 Presidential election.", 
    short_name = 'scotus_fairness3', 
    use_plan = assessment_scotus_fairness2['plan0'], # try the previously generated plan prior to the 'improvement round'.
    improvement_round=False,
    search_cache = search_cache, 
    added_instructions = """Especially, consider the following issues:

* Recent ethics challenges of, monetary influence on, and failures to recuse themselves by the current U.S. Supreme Court justices.
* Recent ideologically-based rulings of the current U.S. Supreme Court justices.
* Partisan indebtedness of and political pressure on the current U.S. Supreme Court Justices.
""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 25,
    force_run = False,
    print_level = 2
)
display(assessment_scotus_fairness3['kb_dot'])
if assessment_scotus_fairness3.get('plan_result'):
    print_summary(assessment_scotus_fairness3)

#### Voting Rights Act

In [ ]:
assessment_voting_rights =  await assess_proposition2(
    proposition =  "The Voting Rights Act is still necessary and effective despite attacks by states, Congress, and the Supreme Court.", 
    short_name = 'voting_rights', 
    improvement_round=True,
    search_cache = search_cache, 
    added_instructions = "", 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 25,
    force_run = False,
    print_level = 2
)
display(assessment_voting_rights['kb_dot'])
if assessment_voting_rights.get('plan_result'):
    print_summary(assessment_voting_rights)

#### Presidential conviction

In [ ]:
assessment_president_conviction =  await assess_proposition2(
    proposition =  "It is impossible to convict a former or sitting U.S. President of a felony without starting a civil war.", 
    short_name = 'president_conviction', 
    improvement_round=True,
    search_cache = search_cache, 
    added_instructions = "", 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 25,
    force_run = True,
    print_level = 2
)
display(assessment_president_conviction['kb_dot'])
if assessment_president_conviction.get('plan_result'):
    print_summary(assessment_president_conviction)

#### Biden & Inflation

In [ ]:
assessment_biden_inflation =  await assess_proposition2(
    proposition =  "Inflation in the U.S. economy is because of failures in the Biden administration economic policy.", 
    short_name = 'biden_inflation', 
    search_cache = search_cache, 
    client = client,
    model = "gpt-4-turbo-preview", 
    force_run = False,
    save_pickle = True,
    report_to_file = True,
    print_level = 2
)
display(assessment_biden_inflation['kb_dot'])
if assessment_biden_inflation.get('plan_result'):
    _=print_summary(assessment_biden_inflation)

#### Biden economy

In [ ]:
assessment_biden_economy =  await assess_proposition2(
    proposition =  "The Biden administration economic policy has failed America.", 
    short_name = 'biden_economy', 
    search_cache = search_cache, 
    client = client,
    model = "gpt-4-turbo-preview", 
    force_run = False,
    save_pickle = True,
    report_to_file = True,
    print_level = 2
)
display(assessment_biden_economy['kb_dot'])
if assessment_biden_economy.get('plan_result'):
    _=print_summary(assessment_biden_economy)

#### Trump economy

In [ ]:
assessment_trump_econom_getplan =  await assess_proposition2(
    proposition =  "The Trump administration economic policy failed America.", 
    short_name = 'trump_economy_getplan', 
    plan_only = True,
    search_cache = search_cache, 
    client = client,
    model = "gpt-4-turbo-preview", 
    force_run = False,
    save_pickle = False,
    report_to_file = False,
    print_level = 2
)

In [ ]:
assessment_trump_econom_getplan['kb_dot']

In [ ]:
assessment_trump_economy =  await assess_proposition2(
    proposition =  "The Trump administration economic policy failed America.", 
    short_name = 'trump_economy', 
    use_plan = assessment_trump_econom_getplan['plan'],
    search_cache = search_cache, 
    client = client,
    model = "gpt-4-turbo-preview", 
    force_run = False,
    save_pickle = True,
    report_to_file = True,
    print_level = 2
)
display(assessment_trump_economy['kb_dot'])
if assessment_trump_economy.get('plan_result'):
    _=print_summary(assessment_trump_economy)

#### Rents are too high

In [ ]:
assessment_high_rents =  await assess_proposition2(
    proposition =  "More renters than ever now spend too much of their incomes on housing.", 
    short_name = 'high_rents', 
    search_cache = search_cache, 
    added_instructions = """
Pursue this issue through the 2024 study by the Joint Center for Housing Studies of Harvard University.

Investigate alternative explanations for each subproposition.
Investigate evidence for and evidence against each subproposition.
""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    force_run = False,
    save_pickle = True,
    report_to_file = True,
    print_level = 2
)
display(assessment_high_rents['kb_dot'])
_=print_summary(assessment_high_rents)

#### American Democracy

In [ ]:
short_name = 'american_democracy' #'long_proposition'
assessment_results = pickle_it(assessment_results = {'short_name': short_name}, save = False)

In [ ]:
#file_path = 'results_american_democracy.pkl'
#with open(file_path, 'rb') as state_file:
#    json_string = pickle.load(state_file)
#print(json_string)

In [ ]:
assessment_american_democracy =  await assess_proposition2(
    proposition =  "American democracy is stronger than it has ever been.", 
    short_name = 'american_democracy', 
    improvement_round=True,
    search_cache = search_cache, 
    added_instructions = """
Ensure that all Query nodes either lead directly or indirectly back to the main proposition. 
So, the query graph has a single "sink" node capturing the main proposition that generates the ultimate conclusions.

Investigate alternative explanations for each subproposition.
Investigate evidence for and evidence against each subproposition.
""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    force_run = False,
    save_pickle = True,
    report_to_file = True,
    print_level = 2
)
display(assessment_american_democracy['kb_dot'])
_=print_summary(assessment_american_democracy)

In [ ]:
results_text = search_GoogleSerperAPI(topic = 'Ohio Nov. 2023 Ballot Issue 1') #(topic = "Yes on Ohio Issue 1 is good or bad?")

In [ ]:
print(results_text)

In [ ]:
client = instructor.patch(OpenAI())
plan_ohio0 = query_planner( "Given proposition 'Yes on Ohio Nov. 2023 Ballot Issue 1 is good', first identify all subpropositions and their subpropositions, and so on, that need to be true to support it, then identify all factual axiomatic statements as evidence that support or refute each and every one of the propositions. Finally, Is Yes on Ohio Issue 1 good?",client)
pprint(plan_ohio0.model_dump())

In [ ]:
graph: KnowledgeGraph = convert_to_graph(plan_ohio0,'Yes on Ohio Nov. 2023 Ballot Issue 1 is good',label_wrap=20)
kb_dot = visualize_knowledge_graph(graph,filename_stem='plan_ohio0',flip=True, aspect_ratio= 'auto')
kb_dot

In [ ]:
plan_ohio0_result = await plan_ohio0.execute(client)

In [ ]:
#import re
last_node_ohio0 = list(plan_ohio0_result.values())[-1]
print('QUERY: ' + re.sub(r'\\"response(s)*\\"',r'\n"response\1"',last_node_ohio0.query.replace("\\n","\n")).replace('"query"','\n"query"').replace(',"response"',',\n\n* "response"') + '\n\n')
print('RESPONSE: ' + last_node_ohio0.response)

In [ ]:
client = instructor.patch(OpenAI())
parent_proposition = 'Voting "Yes" on the "Ohio Nov. 2023 Ballot Issue 1" was best for Ohio.'
context = "Investigate the Ohio November 2023 Ballot Issue 1 related to Reproductive Rights. DO NOT investigate any other Issue 1's, e.g., related to threshold for constitutional ammendments."
messages = [
    {
        "role": "system",
        "content": "You are one of two players in the World Championship of a game called \"Deep Planning to Investigate a Hypothesis!\" \
As such, you are a world-class hypothesis confirmation algorithm capable of breaking apart a proposition into its subpropositions, their sub-subpropositions, \
and so on. This creates a <<QUERY PLAN>>, which is a graph of subpropositions that go from bottom-level axiomatic primitives and logically lead up to the original top-level proposition. \
You also identify key factual evidence that supports or refutes each and every subproposition such that an argument, which is objective, thorough, and logical, \
can be made for or against each subproposition. When logically assessing the evidence, you consider all alternative explanations for the existence of that evidence and not only those \
subpropositions that lead back to the top-level proposition. \
The player that generates the (1) deepest, (2) most thorough & complete, (3) most accurate as verifiable by evidence from the internet, and (4) most insightful research plan into the hypothesis \
will be crowned the World Champion Hypothesis Analyzer!!! \
\
So, think step-by-step in a logical reasoning manner to get a better understanding of the problem. Then generate this inverted 'network' of logical dependencies leading from \
the parent top-level proposition at 'root' to the 'branch' subpropositions and to the 'leaf' pieces of evidence supporting or refuting each subproposition.\
\
Before you respond with your finalized plan, consider that your opponent player will defeat you if they have found additional subpropositions for any proposition that is in common with yours \
or if they have a more thorough search of the internet than you do to gather evidence. So, make sure that your plan is both deep in terms of levels of subpropositions and bushy in terms of \
lots of leaf nodes to uncover evidence through internet searches. \
",
    },
    {
        "role": "user",
        "content": f"""CONTEXT: '{context}'
--------------------------------------------------------------
TOP-LEVEL PROPOSITION: {parent_proposition}
--------------------------------------------------------------
TASK: Generate the correct QUERY PLAN -- i.e., 
the network of top-level proposition, branches of 
subpropositions and their sub-subpropositions, 
and so on, and finally, the total set of leaves of evidence.
--------------------------------------------------------------
RESPONSE: <<QUERY PLAN>>""",
    },
]

plan_ohio1 = client.chat.completions.create(
    model='gpt-4-turbo-preview',
    temperature=0,
    response_model=QueryPlan,
    messages=messages,
    max_retries = 2,
    max_tokens=3000,
)

In [ ]:
pprint(plan_ohio1.model_dump())

In [ ]:
graph: KnowledgeGraph = convert_to_graph(plan_ohio1,'Yes on "Ohio Nov. 2023 Ballot Issue 1" is good',label_wrap=20)
kb_dot = visualize_knowledge_graph(graph,filename_stem='plan_ohio1',flip=True, aspect_ratio= 'auto')
kb_dot

**Execute the plan using GPT-3.5 Turbo...**

In [ ]:
plan_ohio1_result = await plan_ohio1.execute(client)

In [ ]:
#import re
last_node_ohio1 = list(plan_ohio1_result.values())[-1]
print('QUERY: ' + re.sub(r'\\"response(s)*\\"',r'\n"response\1"',last_node_ohio1.query.replace("\\n","\n"),flags=re.DOTALL).replace('"query"','\n"query"').replace(',"response"',',\n\n* "response"') + '\n\n')
print('RESPONSE: ' + last_node_ohio1.response)

#### American Stature During Biden Administration

In [ ]:
proposition = "America's stature in the world has worsened during the Biden administration in comparison to what it was under the Trump administration."
key_question = f"""Given proposition '{proposition}'. First identify all sub-propositions and their sub-propositions, and so on, that need to be true to support it.
Then identify all factual axiomatic statements as evidence that support or refute each and every one of the propositions. 
Finally, answer this question: Is '{proposition}' the truth?"""
plan = query_planner( key_question )
pprint(plan.model_dump())
plan_biden_result = await plan.execute() # Execute using GPT-3.5 Turbo

In [ ]:
print('QUERY: ' + re.sub(r'\\"response(s)*\\"',r'\n"response\1"',plan_biden_result.query.replace("\\n","\n")).replace('"query"','\n"query"').replace(',"response"',',\n\n* "response"') + '\n\n')
print('RESPONSE: ' + plan_biden_result.response)

#### Trump Corruption

In [ ]:
assessment_trump_corruption =  await assess_proposition2(
    proposition = "Donald Trump is no more or less corrupt than any typical U.S. President.", 
    short_name = 'trump_corruption', 
    improvement_round=True,
    search_cache = search_cache, 
    client = client,
    model = "gpt-4-turbo-preview",
    report_to_file = True,
    kb_label_wrap = 25,
    print_level=2
)
display(assessment_trump_corruption['kb_dot'])
if assessment_trump_corruption.get('plan_result'):
    _=print_summary(assessment_trump_corruption)

In [ ]:
proposition = "Donald Trump is no more or less corrupt than any typical U.S. President."
key_question = f"""Given proposition '{proposition}'. First identify all sub-propositions and their sub-propositions, and so on, that need to be true to support it.
Then identify all factual axiomatic statements as evidence that support or refute each and every one of the propositions. 
Finally, answer this question: Is '{proposition}' the truth?"""
plan = query_planner( key_question )
pprint(plan.model_dump())
plan_trump_result = await plan.execute() # Execute using GPT-3.5 Turbo

In [ ]:
print('QUERY: ' + re.sub(r'\\"response(s)*\\"',r'\n"response\1"',plan_trump_result.query.replace("\\n","\n")).replace('"query"','\n"query"').replace(',"response"',',\n\n* "response"') + '\n\n')
print('RESPONSE: ' + plan_trump_result.response)

#### Biden Corruption

In [ ]:
qp_dict = eval(assessment_trump_corruption['plan'].model_dump_json().replace("Donald Trump","Joe Biden"))
qp_dict['context'] = None
qp = QueryPlan(**qp_dict)

In [ ]:
qp

In [ ]:
assessment_biden_corruption =  await assess_proposition2(
    proposition = "Joe Biden is no more or less corrupt than any typical U.S. President.", 
    short_name = 'biden_corruption', 
    use_plan = qp,
    search_cache = search_cache, 
    client = client,
    model = "gpt-4-turbo-preview",
    report_to_file = True,
    kb_label_wrap = 25,
    print_level=2
)
display(assessment_biden_corruption['kb_dot'])
if assessment_biden_corruption.get('plan_result'):
    _=print_summary(assessment_biden_corruption)

In [ ]:
proposition = "Joe Biden is no more or less corrupt than any typical U.S. President."
key_question = f"""Given proposition '{proposition}'. First identify all sub-propositions and their sub-propositions, and so on, that need to be true to support it.
Then identify all factual axiomatic statements as evidence that support or refute each and every one of the propositions. 
In generating the query plan, identify these specific sub-propositions and the specific pieces of evidence needed to confirm and refute them.
Finally, answer this question: Is '{proposition}' the truth?"""
plan = query_planner( key_question )
pprint(plan.model_dump())
plan_biden2_result = await plan.execute() # Execute using GPT-3.5 Turbo

In [ ]:
print('QUERY: ' + re.sub(r'\\"response(s)*\\"',r'\n"response\1"',plan_biden2_result.query.replace("\\n","\n")).replace('"query"','\n"query"').replace(',"response"',',\n\n* "response"') + '\n\n')
print('RESPONSE: ' + plan_biden2_result.response)

#### Presidential immunity

In [ ]:
assessment_president_immune =  await assess_proposition2(
    proposition =  "Is the U.S. President immune from prosecution for any and all actions they commit during their term?", 
    short_name = 'president_immune', 
    use_plan = None,
    search_cache = search_cache, 
    added_instructions = """
Ensure that all Query nodes either lead directly or indirectly back to the main proposition. 
So, the query graph has a single "sink" node capturing the main proposition that generates the ultimate conclusions.

Investigate alternative explanations for each subproposition.
Investigate evidence for and evidence against each subproposition.
""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    dynamic = True,
    force_run = True,
    save_pickle = True,
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_president_immune['kb_dot']

#### Florida Director of Health on Vaccines

In [ ]:
assessment_florida_vaccines =  await assess_proposition2(
    proposition =  "The current Director of Health for the state of Florida is right about vaccines.", 
    short_name = 'florida_vaccines', 
    use_plan = None,
    search_cache = search_cache, 
    added_instructions = """
Ensure that all Query nodes either lead directly or indirectly back to the main proposition. 
So, the query graph has a single "sink" node capturing the main proposition that generates the ultimate conclusions.

Investigate alternative explanations for each subproposition.
Investigate evidence for and evidence against each subproposition.
""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    dynamic = True,
    force_run = True,
    save_pickle = True,
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_florida_vaccines['kb_dot']

The query graph shows that the three leafe nodes involving evidence should actually be parents of the node for the ultimate question. So, we need to add them to the `dependencies` list of the ultimate question node.

In [ ]:
print(assessment_florida_vaccines['plan'])
plan_florida_vaccines.query_graph

In [ ]:
ultimate_question_node = None
assessment_florida_vaccines['plan'].query_graph[0].dependencies = list(set(assessment_florida_vaccines['plan'].query_graph[0].dependencies))
plan_florida_vaccines = QueryPlan(**(assessment_florida_vaccines['plan'].model_dump()))
ultimate_question = [iq for iq in enumerate(plan_florida_vaccines.query_graph) if 'right about vaccines' in iq[1].question][0]
i_ultimate,ultimate_question_node  = ultimate_question
print(i_ultimate)
evidence_nodes = [q.id for q in plan_florida_vaccines.query_graph if 'evidence' in q.question]
evidence_nodes2 = [q.id for q in plan_florida_vaccines.query_graph if 'alternative explanations' in q.question]
ultimate_question_node.dependencies.extend(evidence_nodes + evidence_nodes2)
ultimate_question_node.dependencies = list(set(ultimate_question_node.dependencies))
plan_florida_vaccines.query_graph[i_ultimate] = ultimate_question_node
graph_florida_vaccines: KnowledgeGraph = convert_to_graph(plan_florida_vaccines,"The current Director of Health for the state of Florida is right about vaccines.",label_wrap=20)
kb_dot_florida_vaccines = visualize_knowledge_graph(graph_florida_vaccines,flip=True, aspect_ratio= 'auto')
kb_dot_florida_vaccines

In [ ]:
assessment_florida_vaccines2 =  await assess_proposition2(
    proposition =  "The current Director of Health for the state of Florida is right about vaccines.", 
    short_name = 'florida_vaccines2', 
    use_plan = plan_florida_vaccines,
    search_cache = search_cache, 
    added_instructions = """
Ensure that all Query nodes either lead directly or indirectly back to the main proposition. 
So, the query graph has a single "sink" node capturing the main proposition that generates the ultimate conclusions.

Investigate alternative explanations for each subproposition.
Investigate evidence for and evidence against each subproposition.
""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    dynamic = True,
    force_run = True,
    save_pickle = True,
    report_to_file = True,
    kb_label_wrap = 20
)

#### Vaccination risk-benefit

In [ ]:
assessment_antivax2 =  await assess_proposition2(
    proposition = "Vaccinations do more harm than good", 
    short_name = 'antivax2', 
    search_cache = search_cache, 
    added_instructions = '', 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20,
    print_level = 1
)

In [ ]:
display(assessment_antivax2['kb_dot'])
if assessment_antivax2.get('plan_result'):
    _ = print_summary(assessment_antivax2)

In [ ]:
print(assessment_antivax2['plan'].model_dump_json(indent=2))

In [ ]:
assessment_antivax3 =  await assess_proposition(
    proposition = "Vaccinations do more harm than good", 
    short_name = 'antivax3', 
    search_cache = search_cache, 
    added_instructions = '', 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20,
    print_level = 1
)
display(assessment_antivax3['kb_dot'])
if assessment_antivax3.get('plan_result'):
    _ = print_summary(assessment_antivax3)

In [ ]:
assessment_antivax2['kb_dot']

#### Belichick-Brady Dynasty

In [ ]:
assessment_nfl_dynasty =  await assess_proposition2(
    proposition = "Currently, the Belichick-Brady Era New England Patriots are the greatest NFL dynasty of all time.", 
    short_name = 'nfl_dynasty', 
    added_instructions = """
Build a `query_plan_outline` of the following form:

Is [TARGET ENTITY] better than [OTHER ENTITIES] wrt [TRAIT]? (for N other entities)

* What are the indicators of [TRAIT]? (find M indicators)
* How do [ENTITY]'s statistics on these indicators compare to the statistics of the comparison entities? 
  * [INDICATOR_1] 
    * Tabulate the statistics of [OTHER ENTITIES] on [INDICATOR_1]
      * What is [INDICATOR_1] for [TARGET ENTITY]?
      * What is [INDICATOR_1] for [OTHER ENTITY_1]?
      * What is [INDICATOR_1] for [OTHER ENTITY_2]?
      ...
      * What is [INDICATOR_1] for [OTHER ENTITY_N]?
  * [INDICATOR_2] 
    * Tabulate the statistics of [OTHER ENTITIES] on [INDICATOR_2]
      * What is [INDICATOR_2] for [TARGET ENTITY]?
      * What is [INDICATOR_2] for [OTHER ENTITY_1]?
      * What is [INDICATOR_2] for [OTHER ENTITY_2]?
      ...
      * What is [INDICATOR_2] for [OTHER ENTITY_N]?
  ...
  * [INDICATOR_M] 
    * Tabulate the statistics of [OTHER ENTITIES] on [INDICATOR_M]
      * What is [INDICATOR_M] for [TARGET ENTITY]?
      * What is [INDICATOR_M] for [OTHER ENTITY_1]?
      * What is [INDICATOR_M] for [OTHER ENTITY_2]?
     ...
      * What is [INDICATOR_M] for [OTHER ENTITY_N]?


In this case:
[TARGET ENTITY] is Belichick-Brady Era New England Patriots
[TRAIT] is great NFL dynasty
[OTHER ENTITIES] are at least N=3 other great NFL dynasties, which you must identify.
You must determine the appropriate number, N, and the names of [OTHER ENTITY_1], [OTHER ENTITY_2],..., [OTHER ENTITY_M].

Based upon the top proposition, determine the relevant indicators for [TRAIT]: [INDICATOR_1], [INDICATOR_2], ..., [INDICATOR_M].
You must determine the appropriate number, M, and the names of [INDICATOR_1], [INDICATOR_2],..., [INDICATOR_M].

In the `query_graph` of the query plan, for `Query` nodes corresponding to "Tabulate the statistics...", set the `question` field 
to that tabulation statement, i.e., "Tabulate the statistics of ..." so that a table is returned for those nodes.

""", 
    client = client,
    search_cache = search_cache, 
    model = "gpt-4-turbo-preview", 
    force_run=False,
    report_to_file = True,
    print_level=2
)
display(assessment_nfl_dynasty['kb_dot'])
if assessment_nfl_dynasty.get('plan_result'):
    _ = print_summary(assessment_nfl_dynasty)

In [ ]:
assessment_nfl_dynasty2 =  await assess_proposition2(
    proposition = "Currently, the Belichick-Brady Era New England Patriots are the greatest NFL dynasty of all time.", 
    short_name = 'nfl_dynasty2', 
    added_instructions = """
Build a `query_plan_outline` of the following form:

Is [TARGET ENTITY] better than [OTHER ENTITIES] wrt [TRAIT]? (for N other entities)

* What are the indicators of [TRAIT]? (find M indicators)
  * Be sure to create a `Query` node that searches for these indicators and that then is included in the `dependencies` field of the other `Query` nodes that need [INDICATOR_1], [INDICATOR_2], ..., [INDICATOR_M].
* What are the [OTHER ENTITIES] in contention wrt [TRAIT]? (Identify specific entities by name as [OTHER ENTITY_1], [OTHER ENTITY_2],..., [OTHER ENTITY_N].)
  * Be sure to create a `Query` node that searches for these other entities and that then is included in the `dependencies` field of the other `Query` nodes that need [OTHER ENTITY_1], [OTHER ENTITY_2], ..., [OTHER ENTITY_N].
* How do [ENTITY]'s statistics on these indicators compare to the statistics of the comparison entities? 
  * [INDICATOR_1] 
    * Tabulate the statistics of [OTHER ENTITIES] on [INDICATOR_1]
      * What is [INDICATOR_1] for [TARGET ENTITY]?
      * What is [INDICATOR_1] for [OTHER ENTITY_1]?
      * What is [INDICATOR_1] for [OTHER ENTITY_2]?
      ...
      * What is [INDICATOR_1] for [OTHER ENTITY_N]?
  * [INDICATOR_2] 
    * Tabulate the statistics of [OTHER ENTITIES] on [INDICATOR_2]
      * What is [INDICATOR_2] for [TARGET ENTITY]?
      * What is [INDICATOR_2] for [OTHER ENTITY_1]?
      * What is [INDICATOR_2] for [OTHER ENTITY_2]?
      ...
      * What is [INDICATOR_2] for [OTHER ENTITY_N]?
  ...
  * [INDICATOR_M] 
    * Tabulate the statistics of [OTHER ENTITIES] on [INDICATOR_M]
      * What is [INDICATOR_M] for [TARGET ENTITY]?
      * What is [INDICATOR_M] for [OTHER ENTITY_1]?
      * What is [INDICATOR_M] for [OTHER ENTITY_2]?
     ...
      * What is [INDICATOR_M] for [OTHER ENTITY_N]?


In this case:

* [TARGET ENTITY] = Belichick-Brady Era New England Patriots
* [TRAIT] = great NFL dynasty
* [OTHER ENTITIES] = at least N=3 other great NFL dynasties, which you must identify by specific names.
  * You must determine the appropriate number, N, and the names of [OTHER ENTITY_1], [OTHER ENTITY_2],..., [OTHER ENTITY_M].

Based upon the top proposition, determine the relevant indicators for [TRAIT]: [INDICATOR_1], [INDICATOR_2], ..., [INDICATOR_M].
You must determine the appropriate number, M, and the names of [INDICATOR_1], [INDICATOR_2],..., [INDICATOR_M].

In the `query_graph` of the query plan, for `Query` nodes corresponding to "Tabulate the statistics...", set the `question` field 
to that tabulation statement, i.e., "Tabulate the statistics of ..." so that a table is returned for those nodes.

Make sure that you assign the `Query` nodes for indicators and for other entities as members of the `dependencies` field list of other `Query` nodes that require them.


""", 
    client = client,
    search_cache = search_cache, 
    model = "gpt-4-turbo-preview", 
    force_run=False,
    report_to_file = True,
    print_level=2
)
display(assessment_nfl_dynasty2['kb_dot'])
if assessment_nfl_dynasty2.get('plan_result'):
    _ = print_summary(assessment_nfl_dynasty2)

#### Brady vs. Others

In [ ]:
assessment_brady_vs_others =  await assess_proposition2(
    proposition = "Tom Brady is the greatest NFL player of all time.", 
    short_name = 'brady_vs_others', 
    added_instructions = """
Build a `query_plan_outline` of the following form:

Is [TARGET ENTITY] better than [OTHER ENTITIES] wrt [TRAIT]? (for N other entities)

* What are the indicators of [TRAIT]? (find M indicators)
* Who are the [OTHER ENTITIES] in contention wrt [TRAIT]? (Identify specific entities by name as [OTHER ENTITY_1], [OTHER ENTITY_2],..., [OTHER ENTITY_M].)
* How do [ENTITY]'s statistics on these indicators compare to the statistics of the comparison entities? 
  * [INDICATOR_1] 
    * Tabulate the statistics of [OTHER ENTITIES] on [INDICATOR_1]
      * What is [INDICATOR_1] for [TARGET ENTITY]?
      * What is [INDICATOR_1] for [OTHER ENTITY_1]?
      * What is [INDICATOR_1] for [OTHER ENTITY_2]?
      ...
      * What is [INDICATOR_1] for [OTHER ENTITY_N]?
  * [INDICATOR_2] 
    * Tabulate the statistics of [OTHER ENTITIES] on [INDICATOR_2]
      * What is [INDICATOR_2] for [TARGET ENTITY]?
      * What is [INDICATOR_2] for [OTHER ENTITY_1]?
      * What is [INDICATOR_2] for [OTHER ENTITY_2]?
      ...
      * What is [INDICATOR_2] for [OTHER ENTITY_N]?
  ...
  * [INDICATOR_M] 
    * Tabulate the statistics of [OTHER ENTITIES] on [INDICATOR_M]
      * What is [INDICATOR_M] for [TARGET ENTITY]?
      * What is [INDICATOR_M] for [OTHER ENTITY_1]?
      * What is [INDICATOR_M] for [OTHER ENTITY_2]?
     ...
      * What is [INDICATOR_M] for [OTHER ENTITY_N]?


In this case:

* [TARGET ENTITY] is Tom Brady
* [TRAIT] is great NFL player
* [OTHER ENTITIES] are at least N=3 other great NFL players, which you must identify.
  * You must determine the appropriate number, N, and the names of [OTHER ENTITY_1], [OTHER ENTITY_2],..., [OTHER ENTITY_M].

Based upon the top proposition, determine the relevant indicators for [TRAIT]: [INDICATOR_1], [INDICATOR_2], ..., [INDICATOR_M].
You must determine the appropriate number, M, and the names of [INDICATOR_1], [INDICATOR_2],..., [INDICATOR_M].

In the `query_graph` of the query plan, for `Query` nodes corresponding to "Tabulate the statistics...", set the `question` field 
to that tabulation statement, i.e., "Tabulate the statistics of ..." so that a table is returned for those nodes.

""", 
    client = client,
    search_cache = search_cache, 
    model = "gpt-4-turbo-preview", 
    force_run=False,
    report_to_file = True,
    print_level=2
)
display(assessment_brady_vs_others['kb_dot'])
if assessment_brady_vs_others.get('plan_result'):
    _ = print_summary(assessment_brady_vs_others)

In [ ]:
assessment_brady_vs_others2 =  await assess_proposition2(
    proposition = "Tom Brady is the greatest NFL player of all time.", 
    short_name = 'brady_vs_others2', 
    added_instructions = """
Build a `query_plan_outline` of the following form:

Is [TARGET ENTITY] better than [OTHER ENTITIES] wrt [TRAIT]? (for N other entities)

* What are the indicators of [TRAIT]? (find M indicators)
  * Be sure to create a `Query` node that searches for these indicators and that then is included in the `dependencies` field of the other `Query` nodes that need [INDICATOR_1], [INDICATOR_2], ..., [INDICATOR_M].
* Who are the [OTHER ENTITIES] in contention wrt [TRAIT]? (Identify specific entities by name as [OTHER ENTITY_1], [OTHER ENTITY_2],..., [OTHER ENTITY_M].)
  * Be sure to create a `Query` node that searches for these other entities and that then is included in the `dependencies` field of the other `Query` nodes that need [OTHER ENTITY_1], [OTHER ENTITY_2], ..., [OTHER ENTITY_N].
* How do [ENTITY]'s statistics on these indicators compare to the statistics of the comparison entities? 
  * [INDICATOR_1] 
    * Tabulate the statistics of [OTHER ENTITIES] on [INDICATOR_1]
      * What is [INDICATOR_1] for [TARGET ENTITY]?
      * What is [INDICATOR_1] for [OTHER ENTITY_1]?
      * What is [INDICATOR_1] for [OTHER ENTITY_2]?
      ...
      * What is [INDICATOR_1] for [OTHER ENTITY_N]?
  * [INDICATOR_2] 
    * Tabulate the statistics of [OTHER ENTITIES] on [INDICATOR_2]
      * What is [INDICATOR_2] for [TARGET ENTITY]?
      * What is [INDICATOR_2] for [OTHER ENTITY_1]?
      * What is [INDICATOR_2] for [OTHER ENTITY_2]?
      ...
      * What is [INDICATOR_2] for [OTHER ENTITY_N]?
  ...
  * [INDICATOR_M] 
    * Tabulate the statistics of [OTHER ENTITIES] on [INDICATOR_M]
      * What is [INDICATOR_M] for [TARGET ENTITY]?
      * What is [INDICATOR_M] for [OTHER ENTITY_1]?
      * What is [INDICATOR_M] for [OTHER ENTITY_2]?
     ...
      * What is [INDICATOR_M] for [OTHER ENTITY_N]?


In this case:

* [TARGET ENTITY] is Tom Brady
* [TRAIT] is great NFL player
* [OTHER ENTITIES] are at least N=3 other great NFL players, which you must identify.
  * You must determine the appropriate number, N, and the names of [OTHER ENTITY_1], [OTHER ENTITY_2],..., [OTHER ENTITY_M].

Based upon the top proposition, determine the relevant indicators for [TRAIT]: [INDICATOR_1], [INDICATOR_2], ..., [INDICATOR_M].
You must determine the appropriate number, M, and the names of [INDICATOR_1], [INDICATOR_2],..., [INDICATOR_M].

In the `query_graph` of the query plan, for `Query` nodes corresponding to "Tabulate the statistics...", set the `question` field 
to that tabulation statement, i.e., "Tabulate the statistics of ..." so that a table is returned for those nodes.

Make sure that you assign the `Query` nodes for indicators and for other entities as members of the `dependencies` field list of other `Query` nodes that require them.

""", 
    client = client,
    search_cache = search_cache, 
    model = "gpt-4-turbo-preview", 
    force_run=False,
    report_to_file = True,
    print_level=2
)
display(assessment_brady_vs_others2['kb_dot'])
if assessment_brady_vs_others2.get('plan_result'):
    _ = print_summary(assessment_brady_vs_others2)

#### Critical Race Theory in K-12 schools

In [ ]:
assessment_crt =  await assess_proposition(
    proposition = 'Critical Race Theory is not actually taught in American K-12 schools.', 
    short_name = 'crt', 
    search_cache = search_cache, 
    added_instructions = '', 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_crt['kb_dot']

#### 2020 election stolen

In [ ]:
assessment_election_2020 =  await assess_proposition(
    proposition = "The 2020 U.S. Presidential election was stolen.", 
    short_name = 'election_2020', 
    search_cache = search_cache, 
    added_instructions = '', 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_election_2020['kb_dot']

In [ ]:
assessment_election_2020_max =  await assess_proposition(
    proposition = "The 2020 U.S. Presidential election was stolen.", 
    short_name = 'election_2020_max', 
    search_cache = search_cache, 
    added_instructions = '', 
    model = "gpt-4-turbo-preview", 
    dynamic = True,
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_election_2020_max['kb_dot']

#### 2016 election stolen

In [ ]:
assessment_election_2016 =  await assess_proposition(
    proposition = "The 2016 U.S. Presidential election was stolen.", 
    short_name = 'election_2016', 
    search_cache = search_cache, 
    added_instructions = '', 
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_election_2016['kb_dot']

#### Russian interference, 2016 election

In [ ]:
assessment_election_2016a =  await assess_proposition(
    proposition = "Russian interference swayed the outcome of the 2016 U.S. Presidential election.", 
    short_name = 'election_2016a', 
    search_cache = search_cache, 
    added_instructions = '', 
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_election_2016a['kb_dot']

#### NRA and 2nd Amendment

In [ ]:
assessment_nra_2nd =  await assess_proposition(
    proposition = "The 2nd Amendment of the U.S. Constitution protects a narrower set of rights than the NRA claims that the 2nd Amendment does.", 
    short_name = 'nra_2nd', 
    search_cache = search_cache, 
    added_instructions = '', 
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_nra_2nd['kb_dot']

#### College Value Proposition

In [ ]:
assessment_college_value =  await assess_proposition(
    proposition =  "A college education is a bad value proposition for the majority of graduating high school seniors in America today.", 
    short_name = 'college_value', 
    search_cache = search_cache, 
    added_instructions = '', 
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_college_value['kb_dot']

In [ ]:
assessment_college_value2 =  await assess_proposition(
    proposition =  "A college education is a good value proposition for the majority of graduating high school seniors in America today.", 
    short_name = 'college_value2', 
    search_cache = search_cache, 
    added_instructions = '', 
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_college_value2['kb_dot']

#### Elon Musk

In [ ]:
assessment_musk =  await assess_proposition(
    proposition =  "Elon Musk's current success is due more to his wealth and autonomy in decision-making than due to his brilliance and judgement.", 
    short_name = 'musk', 
    search_cache = search_cache, 
    added_instructions = '', 
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_musk['kb_dot']

#### Elon Musk, hate-monger?

In [ ]:
assessment_musk2 =  await assess_proposition(
    proposition =  "Elon Musk's current success is due more to his wealth and autonomy in decision-making than due to his intellect and judgement and despite his engagement in and enablement of hate speech on X (nee Twitter).", 
    short_name = 'musk2', 
    search_cache = search_cache, 
    added_instructions = '', 
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_musk2['kb_dot']

In [ ]:
assessment_musk3 =  await assess_proposition2(
    proposition = "Elon Musk is a hatemonger because he actively retweets hateful content and enables hate speech on X (Twitter).", 
    short_name = 'musk3',
    search_cache = search_cache, 
    added_instructions =  """
Ensure that all Query nodes lead directly or indirectly back to the main proposition. 
So, the query graph has a single "sink" node capturing the main proposition that generates the ultimate conclusions.

Accept the <<GIVEN FACTS>> as truths that needn't be questioned. DO NOT create Query nodes for these facts.
=========

<<GIVEN FACTS>>:
* Elon Musk owns X (Twitter).

==========

""", 
    client = client,
    model = "gpt-4-turbo-preview",
    dynamic = True,
    report_to_file = True,
    kb_label_wrap = 20,
    force_run = True,
    save_pickle = True
)

In [ ]:
assessment_musk3['kb_dot']

#### Populism in American politics

In [ ]:
assessment_populism =  await assess_proposition(
    proposition =  "'Populism' ('ordinary people vs. elites') in current American political discourse is a misnomer for 'xenophobia' and 'racism' -- where 'ordinary people' connotes ordinary white, native-born Americans and 'elites' connotes all non-white people as well as white elites.", 
    short_name = 'populism', 
    search_cache = search_cache, 
    added_instructions = """
Be sure to include the <<CONTEXT>> of <<'in current American politics'>> in all searches and queries that you pose.
=====

<<CONTEXT>>: In current American politics.

=====
""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_populism['kb_dot']

#### Ultra-wealthy hegemony

In [ ]:
assessment_wealth_hegemony =  await assess_proposition(
    proposition =  "Currently, ultra-wealthy American individuals wield the economic hegemony and broader powers that allow them to operate outside of the laws of nations.", 
    short_name = 'wealth_hegemony', 
    search_cache = search_cache, 
    added_instructions = '', 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    kb_label_wrap = 20
)

#### America post-2024-presidential-election

In [ ]:
assessment_america_2024 =  await assess_proposition(
    proposition =  "America will be a great place to live after the 2024 U.S. Presidential Election.", 
    short_name = 'america_2024', 
    search_cache = search_cache, 
    added_instructions = '', 
    model = "gpt-4-turbo-preview", 
    dynamic = True,
    force_run = False,
    save_pickle = True,
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_america_2024['kb_dot']

#### America desperately needs a younger president

In [ ]:
assessment_younger_president =  await assess_proposition(
    proposition =  "America desperately needs a candidate younger than 60-years-old to win the 2024 U.S. Presidential Election.", 
    short_name = 'younger_president', 
    search_cache = search_cache, 
    added_instructions = '', 
    client = client,
    model = "gpt-4-turbo-preview", 
    dynamic = True,
    force_run = False,
    save_pickle = True,
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_younger_president['kb_dot']

#### Guns for personal safety

In [ ]:
assessment_get_a_gun =  await assess_proposition(
    proposition =  "I must carry a gun for personal safety because it is visibly obvious that I a member of an ethnic group that is currently often threatened by hate crimes.", 
    short_name = 'get_a_gun', 
    search_cache = search_cache, 
    added_instructions = "", 
    client = client,
    model = "gpt-4-turbo-preview", 
    dynamic = True,
    force_run = False,
    save_pickle = True,
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_get_a_gun['kb_dot']

In [ ]:
assessment_get_a_gun2 =  await assess_proposition(
    proposition =  "I must carry a gun for personal safety because it is visibly obvious that I a member of an ethnic group that is currently often threatened by hate crimes.", 
    short_name = 'get_a_gun2', 
    search_cache = search_cache, 
    added_instructions = """
Accept the <<GIVEN FACTS>> as truths that needn't be questioned.
=========

<<GIVEN FACTS>>:
* I am Jewish.
* It is visibly obvious that I am Jewish.
* I live in New York City.

==========

""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    force_run = True,
    save_pickle = True,
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_get_a_gun2['kb_dot']

#### Harvard & Asian admissions

In [ ]:
assessment_harvard_asian =  await assess_proposition2(
    proposition =  "During the years 2002 to 2022, Harvard explicitly applied racial and ethnic quotas to limit the number of Asians admitted to the university.", 
    short_name = 'harvard_asian', 
    search_cache = search_cache, 
    added_instructions = """
Ensure that all Query nodes lead directly or indirectly back to the main proposition. 
So, the query graph has a single "sink" node capturing the main proposition that generates the ultimate conclusions.

Investigate alternative explanations for each subproposition.
Investigate evidence against as well as for each subproposition.
""", 
    client = client,
    model = "gpt-4-turbo-preview", 
    force_run = False,
    save_pickle = True,
    report_to_file = True,
    kb_label_wrap = 20
)

In [ ]:
assessment_harvard_asian['kb_dot']

#### Harvard antisemitism

In [ ]:
assessment_harvard_antisemitism =  await assess_proposition2(
    proposition =  "Harvard has a serious anti-semitism problem.", 
    short_name = 'harvard_antisemitism', 
    search_cache = search_cache, 
    added_instructions = "", 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    print_level = 2
)
display(assessment_harvard_antisemitism['kb_dot'])
_=print_summary(assmnt=assessment_harvard_antisemitism)

In [ ]:
a = QueryPlan(**(eval(re.sub("anti-semitism","anti-Arab racism", assessment_harvard_antisemitism['plan'].model_dump_json()))))
visualize_knowledge_graph(convert_to_graph(a,proposition = "Harvard has a serious anti-Arab racism problem."))
assessment_harvard_antiarabracism =  await assess_proposition2(
    proposition =  "Harvard has a serious anti-Arab racism problem.", 
    short_name = 'harvard_antiarabracism', 
    use_plan = a,
    search_cache = search_cache, 
    added_instructions = "", 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    print_level = 2
)
#display(assessment_harvard_antiarabracism['kb_dot'])
_=print_summary(assmnt=assessment_harvard_antiarabracism)

In [ ]:
assessment_harvard_racism =  await assess_proposition2(
    proposition =  "Harvard has a serious problem with racism.", 
    added_instructions = """
Separately perform deep investigations into racism against each of the following individual groups:

* Jews
* Arabs
* African-Americans
* Latinos
* Asians

Compile the findings into a thorough report on the proposition.

""",
    short_name = 'harvard_racism', 
    search_cache = search_cache, 
    client = client,
    model = "gpt-4-turbo-preview", 
    report_to_file = True,
    print_level = 2
)
display(assessment_harvard_racism['kb_dot'])
_=print_summary(assmnt=assessment_harvard_racism)

#### Tesla Model Y

In [ ]:
proposition = "Tesla Model Y is the best electric vehicle currently available."

key_question = f"""# Mandate:
Given proposition '{proposition}'. 
1. Enumerate all sub-propositions, and their sub-propositions and so on, that need to be true to support it.
2. Enumerate all factual axiomatic statements as evidence that support or refute each and every one of the propositions. 
3. Record these propositions and evidence as a textual outline (in 'query_plan_outline') describing the query plan.
3. Generate the query plan as a tree of query nodes (in 'query_graph').
   - Assign a separate query node for each specific sub-proposition.
   - Assign a separate query node for each specific piece of evidence needed to confirm and refute each sub-proposition. 
   - Be sure to specify SEARCH-type query nodes when needed to gather the latest factual information as evidence.
"""

instructions = """
## Two-Way Contingency Table: 
This is a tool to maximize thoroughness and objectivity in analyzing each and every proposition.
In considering each proposition H and each piece of evidence E, assess the two-way contingency table with 2x2=4 probabilities P(H=True,E=True), P(H=True,E=False), P(H=False,E=True), and P(H=False,E=False).
From these joint probabilities, consider the conditional probalities P(E=True | H=True) and P(E=True | H=False), which are the Likelihoods of the evidence given the proposition.
Then assuming the observed evidence E=True, compute (1) the Bayes Factor, BF(H;E) = P(E=True | H=True) / P(E=True | H=False), which is a measure of the strength of evidence supporting (if > 1) or refuting (if < 1) the proposition;
and (2) the Posterior Probability, P( H=True | E=True ) = P(E=True | H=True) x P(H=True) / P(E=True), where P(E=True) =  P(E=True | H=True) x P(H=True) +  P(E=True | H=False) x P(H=False).

### Note: 
The Weight of Evidence, measured in decibans, WoE(H;E) = 10 x log10 BF(H;E), is considered substantial if |WoE(H;E)| >= 5 decibans.  
Also, BF(H;E) = Odds(H=True|E=True)/Odds(H=True), where Odds(x) = P(x)/(1 - P(x)). So, P( H=True | E=True ) = BF(H;E)/(BF(H;E) + 1/Odds(H=True)).
In sum, BF & WoE measure how strongly E moves H from its prior beliefs P(H) to its posterior beliefs P(H|E); and P(H|E) & Odds(H|E) measure how strongly beliefs about H ends up in light of E.

## Contextual Knowledge -- Assigning Numerical Probabilities
For any proposition H = X -> Y (like, X=Next_car, and Y=Is_red), it is often easier to arrive at values for the probability P(H) by considering the chances of different 
alternative propositions $H_alt_i = X & \neg Y_i (like, Y_1=Is_blue, Y_2=Is_green, Y_M=Is_any_other_color). Then we can assign the probabilities of the alternative relative to the central proposition
of interest: R(H_alt_i) = P(H_alt_i)/P(H)$.  Given that, we arrive at the probability P(H) by $P(H) = 1/(1 + sum_{i}{R(H_alt_i)}$.
And this is also useful for assigning conditional probabilities like for Likelihoods P(E|H), consider $R(E|H_alt_i)=P(E|H_alt_i)/P(E|H)$. Then $P(E|H) = 1/(1 + sum_{i}{R(E|H_alt_i)}$.
To assign numerical values , consider the probability ratio R(...) by half-orders of magnitude: which is approximately 1/1000, 1/300, 1/100, 1/30, 1/10, 1/3, 1, 3, 10, 30, 100, 300, 1000.
Pick the one most suitable based upon knowledge of the subject and any information given in context or found by search.
"""

mandate = f"""
# The Question for which we require a Query Plan:
Generate a query plan to answer this question: Is '{proposition}' the truth?"""

plan_tesla2 = query_planner( key_question + mandate )
#plan = query_planner( key_question + instructions + mandate )
pprint(plan_tesla2.model_dump())
plan_tesla2_result = await plan_tesla2.execute()

In [ ]:
last_node_result = list(plan_tesla2_result.values())[-1]
print('QUERY: ' + re.sub(r'\\"response(s)*\\"',r'\n"response\1"',last_node_result.query.replace("\\n","\n")).replace('"query"','\n"query"').replace(',"response"',',\n\n* "response"') + '\n\n')
print('RESPONSE: ' + last_node_result.response)

In [ ]:
tesla2_search_keys = [q.search_query for q in plan_tesla2.query_graph if len(q.search_query)>0]
tesla2_search = {key: summarize_raw_search(eval(search_cache[key])) for key in tesla2_search_keys if search_cache.get(key,None) is not None}

report_markdown = make_report(computed_query = last_node_result,search_dict=tesla2_search)

mrkdwn_file_path = 'plan_tesla2.Rmd'
with open(mrkdwn_file_path, 'w') as file:
    file.write(to_ascii(report_markdown))

#### Universal Basic Income

In [ ]:
proposition = "Universal Basic Income is the most likely result, and a necessary result, of widespread adoption of Generative AI."

key_question = f"""# Mandate:
Given proposition '{proposition}'. 
1. Enumerate all sub-propositions, and their sub-propositions and so on, that need to be true to support it.
2. Enumerate all factual axiomatic statements as evidence that support or refute each and every one of the propositions. 
3. Record these propositions and evidence as a textual outline (in 'query_plan_outline') describing the query plan.
3. Generate the query plan as a tree of query nodes (in 'query_graph').
   - Assign a separate query node for each specific sub-proposition.
   - Assign a separate query node for each specific piece of evidence needed to confirm and refute each sub-proposition. 
   - Be sure to specify search_flag=True for a query node when needed to gather the latest factual information as evidence.
   - Demand that actual specific data and statistics are gathered as evidence and that the answers to the propositions are stated in terms of these data.
"""

mandate = f"""
# The Question for which we require a Query Plan:
Is '{proposition}' the truth?"""

plan_ubiai = query_planner( key_question + mandate )
#plan = query_planner( key_question + instructions + mandate )

pprint(plan_ubiai.model_dump())

In [ ]:
graph: KnowledgeGraph = convert_to_graph(plan_ubiai,proposition,label_wrap=20)
kb_dot = visualize_knowledge_graph(graph,flip=True, aspect_ratio= 'auto')
kb_dot

In [ ]:
plan_ubiai_result = await plan_ubiai.execute(model='gpt-4-turbo-preview')

In [ ]:
last_node_ubiai_result_gpt4 = list(plan_ubiai_result.values())[-1]
print('QUERY: ' + re.sub(r'\\"response(s)*\\"',r'\n"response\1"',last_node_ubiai_result_gpt4.query.replace("\\n","\n")).replace('"query"','\n"query"').replace(',"response"',',\n\n* "response"') + '\n\n')
print('RESPONSE: ' + last_node_ubiai_result_gpt4.response)

In [ ]:
list(search_cache.keys())

In [ ]:

ubiai_search_keys = [
 'economic impacts of Generative AI',
 'Generative AI impact on employment and income distribution',
 'alternative results of widespread adoption of Generative AI',
 'precedents of technological advancements leading to policy changes',
 'societal and economic conditions that necessitate UBI',
 'alternatives to UBI for addressing impacts of Generative AI',
 'current implementations or considerations of UBI in response to AI',
 'expert opinions on Generative AI and UBI',
 'arguments against UBI necessity in AI context',
 'expert opinions against Generative AI and UBI']

ubiai_search = {key: summarize_raw_search(eval(search_cache[key])) for key in ubiai_search_keys if search_cache.get(key,None) is not None}

In [ ]:
report_markdown_ubiai_gpt4 = generate_report(
    plan_outline = plan_ubiai.query_plan_outline,
    lastnode=last_node_ubiai_result_gpt4,
    computed_queries=plan_ubiai_result,
    search_dict=ubiai_search, 
    filename_stem='plan_ubiai_gpt4', 
    write_to_file=True
)

#### Republican State Legislatures vs. Democrat State Legislatures

In [ ]:
#proposition = "Currently, the American citizens of states with Republican-controlled legislatures are prospering greater than the citizens of Democratic-controlled legislatures."
proposition = "Currently, the citizens of Texas are prospering greater than the citizens of California."
key_question = f"""# Mandate:
Given proposition '{proposition}'. 
1. Enumerate all sub-propositions, and their sub-propositions and so on, that need to be true to support it.
2. Enumerate all factual axiomatic statements as evidence that support or refute each and every one of the propositions. 
3. Record these propositions and evidence as a textual outline (in 'query_plan_outline') describing the query plan.
3. Generate the query plan as a tree of query nodes (in 'query_graph').
   - Assign a separate query node for each specific sub-proposition.
   - Assign a separate query node for each specific piece of evidence needed to confirm and refute each sub-proposition. 
   - Be sure to specify SEARCH-type query nodes when needed to gather the latest factual information as evidence.
   - Demand that actual specific data and statistics are gathered as evidence and that the answers to the propositions are stated in terms of these data.
"""

mandate = f"""
# Question for which we require a Query Plan:
Generate a query plan to answer this question: 
Is '{proposition}' the truth?"""

plan_states = query_planner( key_question + mandate )
#plan = query_planner( key_question + instructions + mandate )

pprint(plan_states.model_dump())

In [ ]:
graph: KnowledgeGraph = convert_to_graph(plan_states,proposition,label_wrap=20)
kb_dot = visualize_knowledge_graph(graph,flip=True, aspect_ratio= 'auto')
kb_dot

In [ ]:
print(plan_states.query_plan_outline)

In [ ]:
plan_state_govt_result = await plan_states.execute()

In [ ]:
last_node_state_govt_result = list(plan_state_govt_result.values())[-1]
print('QUERY: ' + re.sub(r'\\"response(s)*\\"',r'\n"response\1"',last_node_state_govt_result.query.replace("\\n","\n")).replace('"query"','\n"query"').replace(',"response"',',\n\n* "response"') + '\n\n')
print('RESPONSE: ' + last_node_state_govt_result.response)

In [ ]:
plan_state_govt_result_gpt4 = await plan_states.execute(model="gpt-4-turbo-preview")

In [ ]:
last_node_state_govt_result_gpt4 = list(plan_state_govt_result_gpt4.values())[-1]
print('QUERY: ' + re.sub(r'\\"response(s)*\\"',r'\n"response\1"',last_node_state_govt_result_gpt4.query.replace("\\n","\n")).replace('"query"','\n"query"').replace(',"response"',',\n\n* "response"') + '\n\n')
print('RESPONSE: ' + last_node_state_govt_result_gpt4.response)

In [ ]:
first_graph = """
1. Is 'Currently, the citizens of Texas are prospering greater than the citizens of California.' the truth?
   A. What does it mean for citizens to be prospering?
   B. Are the citizens of Texas prospering?
      i. What are the economic indicators of prosperity in Texas?
         a. What is the current unemployment rate in Texas?
         b. What is the current GDP per capita in Texas?
         c. What is the current median household income in Texas?
      ii. What are the social indicators of prosperity in Texas?
         a. What is the current education level in Texas?
         b. What is the current health status in Texas?
   C. Are the citizens of California prospering?
      i. What are the economic indicators of prosperity in California?
         a. What is the current unemployment rate in California?
         b. What is the current GDP per capita in California?
         c. What is the current median household income in California?
      ii. What are the social indicators of prosperity in California?
         a. What is the current education level in California?
         b. What is the current health status in California?
   D. Comparison of prosperity between Texas and California
      i. Compare economic indicators of prosperity between Texas and California
      ii. Compare social indicators of prosperity between Texas and California
"""
revised_graph = """
1. Is 'Currently, the citizens of Texas are prospering greater than the citizens of California.' the truth?
   A. What does it mean for citizens to be prospering?
   B. Are the citizens of Texas prospering?
      i. What are the economic indicators of prosperity in Texas?
         a. What is the current unemployment rate in Texas?
         b. What is the current GDP per capita in Texas?
         c. What is the current median household income in Texas?
      ii. What are the social indicators of prosperity in Texas?
         a. What is the current education level in Texas?
         b. What is the current health status in Texas?
   C. Are the citizens of California prospering?
      i. What are the economic indicators of prosperity in California?
         a. What is the current unemployment rate in California?
         b. What is the current GDP per capita in California?
         c. What is the current median household income in California?
      ii. What are the social indicators of prosperity in California?
         a. What is the current education level in California?
         b. What is the current health status in California?
   D. Comparison of prosperity between Texas and California
      i. Compare economic indicators of prosperity between Texas and California
      ii. Compare social indicators of prosperity between Texas and California
"""

result2_queryplan: QueryPlan = client.chat.completions.create(
    model="gpt-4-turbo-preview", temperature=0, messages=[{'role':'system', 'content':system_prompt},{'role':'user', 'content':query}], max_tokens=2000,
    response_model = QueryPlan
    #model="gpt-3.5-turbo", temperature=0, messages=[{'role':'user', 'content':query}], max_tokens=10000
)

print(result2_queryplan.query_plan_outline)

In [ ]:
list(search_cache.keys())

In [ ]:
state_govt_search_keys = [  'current unemployment rate in Texas',
 'current GDP per capita in Texas',
 'current median household income in Texas',
 'current education level in Texas',
 'current health status in Texas',
 'current unemployment rate in California',
 'current GDP per capita in California',
 'current median household income in California',
 'current education level in California',
 'current health status in California'
]

state_govt_search = {key: summarize_raw_search(eval(search_cache[key])) for key in state_govt_search_keys if search_cache.get(key,None) is not None}

In [ ]:
report_markdown_state_govt = make_report(computed_query = last_node_state_govt_result,search_dict=state_govt_search)

mrkdwn_file_path = 'plan_state_govt.Rmd'
with open(mrkdwn_file_path, 'w') as file:
    file.write(to_ascii(report_markdown_state_govt))

In [ ]:
report_markdown_state_govt_gpt4 = make_report(computed_query = last_node_state_govt_result_gpt4,search_dict=state_govt_search)

mrkdwn_file_path = 'plan_state_govt_gpt4.Rmd'
with open(mrkdwn_file_path, 'w') as file:
    file.write(to_ascii(report_markdown_state_govt_gpt4))

In [ ]:
report_markdown_state_govt_gpt4b = generate_report(
    plan_outline = plan_states.query_plan_outline,
    lastnode=last_node_state_govt_result_gpt4,
    computed_queries=plan_state_govt_result_gpt4,
    search_dict=state_govt_search, 
    filename_stem='plan_state_govt_gpt4b', 
    write_to_file=True
)

#### Discrimination

In [ ]:
proposition = "In America today, white Christians are under greater persecution by governmental and private organizations that any Americans of other ethnicities and religions."
key_question = f"""# Mandate:
Given proposition '{proposition}'. 
1. Enumerate all sub-propositions, and their sub-propositions and so on, that need to be true to support it.
2. Enumerate all factual axiomatic statements as evidence that support or refute each and every one of the propositions. 
3. Record these propositions and evidence as a textual outline (in 'query_plan_outline') describing the query plan.
3. Generate the query plan as a tree of query nodes (in 'query_graph').
   - Assign a separate query node for each specific sub-proposition.
   - Assign a separate query node for each specific piece of evidence needed to confirm and refute each sub-proposition. 
   - Be sure to specify SEARCH-type query nodes when needed to gather the latest factual information as evidence.
"""

instructions = """
## Two-Way Contingency Table: 
This is a tool to maximize thoroughness and objectivity in analyzing each and every proposition.
In considering each proposition H and each piece of evidence E, assess the two-way contingency table with 2x2=4 probabilities P(H=True,E=True), P(H=True,E=False), P(H=False,E=True), and P(H=False,E=False).
From these joint probabilities, consider the conditional probalities P(E=True | H=True) and P(E=True | H=False), which are the Likelihoods of the evidence given the proposition.
Then assuming the observed evidence E=True, compute (1) the Bayes Factor, BF(H;E) = P(E=True | H=True) / P(E=True | H=False), which is a measure of the strength of evidence supporting (if > 1) or refuting (if < 1) the proposition;
and (2) the Posterior Probability, P( H=True | E=True ) = P(E=True | H=True) x P(H=True) / P(E=True), where P(E=True) =  P(E=True | H=True) x P(H=True) +  P(E=True | H=False) x P(H=False).

### Note: 
The Weight of Evidence, measured in decibans, WoE(H;E) = 10 x log10 BF(H;E), is considered substantial if |WoE(H;E)| >= 5 decibans.  
Also, BF(H;E) = Odds(H=True|E=True)/Odds(H=True), where Odds(x) = P(x)/(1 - P(x)). So, P( H=True | E=True ) = BF(H;E)/(BF(H;E) + 1/Odds(H=True)).
In sum, BF & WoE measure how strongly E moves H from its prior beliefs P(H) to its posterior beliefs P(H|E); and P(H|E) & Odds(H|E) measure how strongly beliefs about H ends up in light of E.

## Contextual Knowledge -- Assigning Numerical Probabilities
For any proposition H = X -> Y (like, X=Next_car, and Y=Is_red), it is often easier to arrive at values for the probability P(H) by considering the chances of different 
alternative propositions $H_alt_i = X & \neg Y_i (like, Y_1=Is_blue, Y_2=Is_green, Y_M=Is_any_other_color). Then we can assign the probabilities of the alternative relative to the central proposition
of interest: R(H_alt_i) = P(H_alt_i)/P(H)$.  Given that, we arrive at the probability P(H) by $P(H) = 1/(1 + sum_{i}{R(H_alt_i)}$.
And this is also useful for assigning conditional probabilities like for Likelihoods P(E|H), consider $R(E|H_alt_i)=P(E|H_alt_i)/P(E|H)$. Then $P(E|H) = 1/(1 + sum_{i}{R(E|H_alt_i)}$.
To assign numerical values , consider the probability ratio R(...) by half-orders of magnitude: which is approximately 1/1000, 1/300, 1/100, 1/30, 1/10, 1/3, 1, 3, 10, 30, 100, 300, 1000.
Pick the one most suitable based upon knowledge of the subject and any information given in context or found by search.
"""

mandate = f"""
# The Question for which we require a Query Plan:
Finally, derive a query plan to answer this question: Is '{proposition}' the truth?"""

plan = query_planner( key_question + mandate )
#plan = query_planner( key_question + instructions + mandate )

pprint(plan.model_dump())
plan_discrimination_result = await plan.execute()

In [ ]:
print('QUERY: ' + re.sub(r'\\"response(s)*\\"',r'\n"response\1"',plan_discrimination_result.query.replace("\\n","\n")).replace('"query"','\n"query"').replace(',"response"',',\n\n* "response"') + '\n\n')
print('RESPONSE: ' + plan_discrimination_result.response)

In [ ]:
print(plan.query_plan_outline)

In [ ]:
list(search_cache.keys())

In [ ]:
discrimination_search_keys = [ 'Identify the governmental and private organizations in America.',
 'Search for recent incidents of persecution against white Christians.',
 'Search for policies or actions by governmental and private organizations targeting white Christians.',
 'Search for recent incidents of persecution against other ethnicities and religions.',
 'Search for policies or actions by governmental and private organizations targeting other ethnicities and religions.'
]

discrimination_search = {key: summarize_raw_search(eval(search_cache[key])) for key in discrimination_search_keys if search_cache.get(key,None) is not None}

report_markdown = make_report(computed_query = plan_discrimination_result,search_dict=discrimination_search)

mrkdwn_file_path = 'plan_discrimination.Rmd'
with open(mrkdwn_file_path, 'w') as file:
    file.write(to_ascii(report_markdown))

#### Google Unreliable Source of Factual News

In [ ]:
proposition = "Google searches are an unreliable source of factual news."
key_question = f"""# Mandate:
Given proposition '{proposition}'. 
1. Enumerate all sub-propositions, and their sub-propositions and so on, that need to be true to support it.
2. Enumerate all factual axiomatic statements as evidence that support or refute each and every one of the propositions. 
3. Record these propositions and evidence as a textual outline (in 'query_plan_outline') describing the query plan.
3. Generate the query plan as a tree of query nodes (in 'query_graph').
   - Assign a separate query node for each specific sub-proposition.
   - Assign a separate query node for each specific piece of evidence needed to confirm and refute each sub-proposition. 
   - Be sure to specify SEARCH-type query nodes when needed to gather the latest factual information as evidence.
"""


instructions = """
## Two-Way Contingency Table: 
This is a tool to maximize thoroughness and objectivity in analyzing each and every proposition.
In considering each proposition H and each piece of evidence E, assess the two-way contingency table with 2x2=4 probabilities P(H=True,E=True), P(H=True,E=False), P(H=False,E=True), and P(H=False,E=False).
From these joint probabilities, consider the conditional probalities P(E=True | H=True) and P(E=True | H=False), which are the Likelihoods of the evidence given the proposition.
Then assuming the observed evidence E=True, compute (1) the Bayes Factor, BF(H;E) = P(E=True | H=True) / P(E=True | H=False), which is a measure of the strength of evidence supporting (if > 1) or refuting (if < 1) the proposition;
and (2) the Posterior Probability, P( H=True | E=True ) = P(E=True | H=True) x P(H=True) / P(E=True), where P(E=True) =  P(E=True | H=True) x P(H=True) +  P(E=True | H=False) x P(H=False).

### Note: 
The Weight of Evidence, measured in decibans, WoE(H;E) = 10 x log10 BF(H;E), is considered substantial if |WoE(H;E)| >= 5 decibans.  
Also, BF(H;E) = Odds(H=True|E=True)/Odds(H=True), where Odds(x) = P(x)/(1 - P(x)). So, P( H=True | E=True ) = BF(H;E)/(BF(H;E) + 1/Odds(H=True)).
In sum, BF & WoE measure how strongly E moves H from its prior beliefs P(H) to its posterior beliefs P(H|E); and P(H|E) & Odds(H|E) measure how strongly beliefs about H ends up in light of E.

## Contextual Knowledge -- Assigning Numerical Probabilities
For any proposition H = X -> Y (like, X=Next_car, and Y=Is_red), it is often easier to arrive at values for the probability P(H) by considering the chances of different 
alternative propositions $H_alt_i = X & \neg Y_i (like, Y_1=Is_blue, Y_2=Is_green, Y_M=Is_any_other_color). Then we can assign the probabilities of the alternative relative to the central proposition
of interest: R(H_alt_i) = P(H_alt_i)/P(H)$.  Given that, we arrive at the probability P(H) by $P(H) = 1/(1 + sum_{i}{R(H_alt_i)}$.
And this is also useful for assigning conditional probabilities like for Likelihoods P(E|H), consider $R(E|H_alt_i)=P(E|H_alt_i)/P(E|H)$. Then $P(E|H) = 1/(1 + sum_{i}{R(E|H_alt_i)}$.
To assign numerical values , consider the probability ratio R(...) by half-orders of magnitude: which is approximately 1/1000, 1/300, 1/100, 1/30, 1/10, 1/3, 1, 3, 10, 30, 100, 300, 1000.
Pick the one most suitable based upon knowledge of the subject and any information given in context or found by search.
"""

mandate = f"""
# The Question for which we require a Query Plan:
Finally, derive a query plan to answer this question: Is '{proposition}' the truth?"""

plan = query_planner( key_question + mandate )
#plan = query_planner( key_question + instructions + mandate )

pprint(plan.model_dump())
plan_google_result = await plan.execute()

In [ ]:
print('QUERY: ' + re.sub(r'\\"response(s)*\\"',r'\n"response\1"',plan_google_result.query.replace("\\n","\n")).replace('"query"','\n"query"').replace(',"response"',',\n\n* "response"') + '\n\n')
print('RESPONSE: ' + plan_google_result.response)

In [ ]:
print(plan.query_plan_outline)

In [ ]:
list(search_cache.keys())

In [ ]:
google_search_keys = [ 'Identify the governmental and private organizations in America.',
 'Search for recent incidents of persecution against white Christians.',
 'Search for policies or actions by governmental and private organizations targeting white Christians.',
 'Search for recent incidents of persecution against other ethnicities and religions.',
 'Search for policies or actions by governmental and private organizations targeting other ethnicities and religions.'
]

google_search = {key: summarize_raw_search(eval(search_cache[key])) for key in google_search_keys if search_cache.get(key,None) is not None}

report_markdown = make_report(computed_query = plan_google_result,search_dict=google_search)

mrkdwn_file_path = 'plan_google.Rmd'
with open(mrkdwn_file_path, 'w') as file:
    file.write(to_ascii(report_markdown))

#### College Admissions

In [ ]:
#proposition = "Elite American universities unfairly penalize applicants who are not Black or Latino."
proposition = """When evaluating an applicant, admissions policies at elite American universities should consider the applicant's 
(a) uniqueness -- i.e., unlikeliness of such an applicant achieving what they have in life given the conditional circumstances from which they come -- 
(b) value to on-campus community -- i.e., unique perspective & contributions to discourse and ideas in academic community given their circumstances -- 
(c) value to world after graduation -- i.e., role model in community from which applicant came; unique perspective & contributions to professional community within which graduate will work -- and 
(d) other standard criteria for college admission -- i.e., ability to excel in an academically demanding environment."""

proposition = """When evaluating an applicant, admissions policies at elite American universities should consider the applicant's 
(a) unlikeliness of such an applicant achieving what they have in life given the conditional circumstances from which they come -- 
(b) unique perspective & contributions to discourse and ideas in the on-campus academic community given their circumstances -- 
(c) future as a role model in the neighborhood/community from which applicant came; and unique perspective & contributions to professional community within which graduate will work -- and 
(d) ability to excel in an academically demanding environment."""

key_question = f"""# Mandate:
Given proposition '{proposition}'. 
1. Enumerate all sub-propositions, and their sub-propositions and so on, that need to be true to support it.
2. Enumerate all factual axiomatic statements as evidence that support or refute each and every one of the propositions. 
3. Record these propositions and evidence as a textual outline (in 'query_plan_outline') describing the query plan.
3. Generate the query plan as a tree of query nodes (in 'query_graph').
   - Assign a separate query node for each specific sub-proposition.
   - Assign a separate query node for each specific piece of evidence needed to confirm and refute each sub-proposition. 
   - Be sure to specify SEARCH-type query nodes when needed to gather the latest factual information as evidence. Assess and comment on the bias and agenda of all sources of search query responses.
   - Demand that actual specific data and statistics are gathered as evidence and that the answers to the propositions are stated in terms of these data.
"""

mandate = f"""
# Question for which we require a Query Plan
Generate a query plan to answer this question: 
Is this true: '{proposition}' ?"""



plan_college = query_planner( key_question + mandate )
#plan = query_planner( key_question + instructions + mandate )

pprint(plan_college.model_dump())

In [ ]:
graph: KnowledgeGraph = convert_to_graph(plan_college,proposition,label_wrap=40)
kb_dot = visualize_knowledge_graph(graph,flip=True, aspect_ratio= 'auto')
kb_dot

In [ ]:
plan_college2_result = await plan_college.execute(model="gpt4-1106-preview")

In [ ]:
last_node_college2_result = list(plan_college2_result.values())[-1]
print('QUERY: ' + re.sub(r'\\"response(s)*\\"',r'\n"response\1"',last_node_college2_result.query.replace("\\n","\n")).replace('"query"','\n"query"').replace(',"response"',',\n\n* "response"') + '\n\n')
print('RESPONSE: ' + last_node_college2_result.response)

In [ ]:
list(search_cache.keys())

In [ ]:
college2_search_keys = ['current admissions policies at elite American universities 2023',
 "impact of considering applicant's background on success in university admissions",
 'benefits of diverse perspectives in academic communities',
 'impact of role models from diverse backgrounds in communities and professional settings',
 'predictors of academic success in elite university settings']

college2_search = {key: summarize_raw_search(eval(search_cache[key])) for key in college2_search_keys if search_cache.get(key,None) is not None}

In [ ]:
report_markdown_college2 = generate_report(
    plan_outline = plan_college.query_plan_outline,
    lastnode=last_node_college2_result,
    computed_queries=plan_college2_result,
    search_dict=college2_search, 
    filename_stem='plan_college2', 
    write_to_file=True
)

#### College Value Proposition

In [ ]:
proposition = "A college education is a bad value proposition for the majority of graduating high school seniors in America today."
key_question = f"""# Mandate:
Given proposition '{proposition}'. First identify all sub-propositions and their sub-propositions, and so on, that need to be true to support it.
Then identify all factual axiomatic statements as evidence that support and refute each and every one of the propositions. 

In generating the query plan, identify these specific sub-propositions and the specific pieces of evidence needed to confirm and refute them.
"""

instructions = """
## Two-Way Contingency Table: 
This is a tool to maximize thoroughness and objectivity in analyzing each and every proposition.
In considering each proposition H and each piece of evidence E, assess the two-way contingency table with 2x2=4 probabilities P(H=True,E=True), P(H=True,E=False), P(H=False,E=True), and P(H=False,E=False).
From these joint probabilities, consider the conditional probalities P(E=True | H=True) and P(E=True | H=False), which are the Likelihoods of the evidence given the proposition.
Then assuming the observed evidence E=True, compute (1) the Bayes Factor, BF(H;E) = P(E=True | H=True) / P(E=True | H=False), which is a measure of the strength of evidence supporting (if > 1) or refuting (if < 1) the proposition;
and (2) the Posterior Probability, P( H=True | E=True ) = P(E=True | H=True) x P(H=True) / P(E=True), where P(E=True) =  P(E=True | H=True) x P(H=True) +  P(E=True | H=False) x P(H=False).

### Note: 
The Weight of Evidence, measured in decibans, WoE(H;E) = 10 x log10 BF(H;E), is considered substantial if |WoE(H;E)| >= 5 decibans.  
Also, BF(H;E) = Odds(H=True|E=True)/Odds(H=True), where Odds(x) = P(x)/(1 - P(x)). So, P( H=True | E=True ) = BF(H;E)/(BF(H;E) + 1/Odds(H=True)).
In sum, BF & WoE measure how strongly E moves H from its prior beliefs P(H) to its posterior beliefs P(H|E); and P(H|E) & Odds(H|E) measure how strongly beliefs about H ends up in light of E.

## Contextual Knowledge -- Assigning Numerical Probabilities
For any proposition H = X -> Y (like, X=Next_car, and Y=Is_red), it is often easier to arrive at values for the probability P(H) by considering the chances of different 
alternative propositions $H_alt_i = X & \neg Y_i (like, Y_1=Is_blue, Y_2=Is_green, Y_M=Is_any_other_color). Then we can assign the probabilities of the alternative relative to the central proposition
of interest: R(H_alt_i) = P(H_alt_i)/P(H)$.  Given that, we arrive at the probability P(H) by $P(H) = 1/(1 + sum_{i}{R(H_alt_i)}$.
And this is also useful for assigning conditional probabilities like for Likelihoods P(E|H), consider $R(E|H_alt_i)=P(E|H_alt_i)/P(E|H)$. Then $P(E|H) = 1/(1 + sum_{i}{R(E|H_alt_i)}$.
To assign numerical values , consider the probability ratio R(...) by half-orders of magnitude: which is approximately 1/1000, 1/300, 1/100, 1/30, 1/10, 1/3, 1, 3, 10, 30, 100, 300, 1000.
Pick the one most suitable based upon knowledge of the subject and any information given in context or found by search.
"""

mandate = f"""
# The Question for which we require a Query Plan:
Finally, derive a query plan to answer this question: Is '{proposition}' the truth?"""

plan = query_planner( key_question + mandate )
#plan = query_planner( key_question + instructions + mandate )

pprint(plan.model_dump())
plan_college2_result = await plan.execute()

In [ ]:
print('QUERY: ' + re.sub(r'\\"response(s)*\\"',r'\n"response\1"',plan_college2_result.query.replace("\\n","\n")).replace('"query"','\n"query"').replace(',"response"',',\n\n* "response"') + '\n\n')
print('RESPONSE: ' + plan_college2_result.response)

In [ ]:
proposition = "A college education is a bad value proposition for the majority of graduating high school seniors in America today."
key_question = f"""# Mandate:
Given proposition '{proposition}'. First identify all sub-propositions and their sub-propositions, and so on, that need to be true to support it.
Then identify all factual axiomatic statements as evidence that support and refute each and every one of the sub-propositions. Explicitly seek information and
pose queries for each piece of refuting evidence and each piece of supporting evidence for each sub-proposition.

In generating the query plan, identify these specific sub-propositions and the specific pieces of evidence needed to confirm and refute them.
"""

instructions = """
## Two-Way Contingency Table: 
This is a tool to maximize thoroughness and objectivity in analyzing each and every proposition.
In considering each proposition H and each piece of evidence E, assess the two-way contingency table with 2x2=4 probabilities P(H=True,E=True), P(H=True,E=False), P(H=False,E=True), and P(H=False,E=False).
From these joint probabilities, consider the conditional probalities P(E=True | H=True) and P(E=True | H=False), which are the Likelihoods of the evidence given the proposition.
Then assuming the observed evidence E=True, compute (1) the Bayes Factor, BF(H;E) = P(E=True | H=True) / P(E=True | H=False), which is a measure of the strength of evidence supporting (if > 1) or refuting (if < 1) the proposition;
and (2) the Posterior Probability, P( H=True | E=True ) = P(E=True | H=True) x P(H=True) / P(E=True), where P(E=True) =  P(E=True | H=True) x P(H=True) +  P(E=True | H=False) x P(H=False).

### Note: 
The Weight of Evidence, measured in decibans, WoE(H;E) = 10 x log10 BF(H;E), is considered substantial if |WoE(H;E)| >= 5 decibans.  
Also, BF(H;E) = Odds(H=True|E=True)/Odds(H=True), where Odds(x) = P(x)/(1 - P(x)). So, P( H=True | E=True ) = BF(H;E)/(BF(H;E) + 1/Odds(H=True)).
In sum, BF & WoE measure how strongly E moves H from its prior beliefs P(H) to its posterior beliefs P(H|E); and P(H|E) & Odds(H|E) measure how strongly beliefs about H ends up in light of E.

## Contextual Knowledge -- Assigning Numerical Probabilities
For any proposition H = X -> Y (like, X=Next_car, and Y=Is_red), it is often easier to arrive at values for the probability P(H) by considering the chances of different 
alternative propositions $H_alt_i = X & \neg Y_i (like, Y_1=Is_blue, Y_2=Is_green, Y_M=Is_any_other_color). Then we can assign the probabilities of the alternative relative to the central proposition
of interest: R(H_alt_i) = P(H_alt_i)/P(H)$.  Given that, we arrive at the probability P(H) by $P(H) = 1/(1 + sum_{i}{R(H_alt_i)}$.
And this is also useful for assigning conditional probabilities like for Likelihoods P(E|H), consider $R(E|H_alt_i)=P(E|H_alt_i)/P(E|H)$. Then $P(E|H) = 1/(1 + sum_{i}{R(E|H_alt_i)}$.
To assign numerical values , consider the probability ratio R(...) by half-orders of magnitude: which is approximately 1/1000, 1/300, 1/100, 1/30, 1/10, 1/3, 1, 3, 10, 30, 100, 300, 1000.
Pick the one most suitable based upon knowledge of the subject and any information given in context or found by search.
"""

mandate = f"""
# The Question for which we require a Query Plan:
Finally, derive a query plan to answer this question: Is '{proposition}' the truth?"""

plan = query_planner( key_question + mandate )
#plan = query_planner( key_question + instructions + mandate )

pprint(plan.model_dump())
plan_college2a_result = await plan.execute()

In [ ]:
print('QUERY: ' + re.sub(r'\\"response(s)*\\"',r'\n"response\1"',plan_college2a_result.query.replace("\\n","\n")).replace('"query"','\n"query"').replace(',"response"',',\n\n* "response"') + '\n\n')
print('RESPONSE: ' + plan_college2a_result.response)